Desarrollo

In [1]:
HISTORICO_LOG = "historico.log"
def printf(filename, text, add_newline=True):
    """
    Añade una cadena de texto a un archivo especificado. Si el archivo no existe, se crea.
    Opcionalmente, añade un salto de línea al final del texto dependiendo del parámetro add_newline.
    
    Parámetros:
    filename (str): El nombre del archivo al cual se desea añadir el texto.
    text (str): El texto que se desea añadir al archivo.
    add_newline (bool): Indica si se debe añadir un salto de línea al final del texto (por defecto es True).
    """
    try:
        # Abrir el archivo en modo de añadir ('append'). El modo 'a' asegura que si el archivo no existe, se crea.
        with open(filename, 'a') as file:
            if add_newline:
                file.write(text + "\n")  # Añade el texto y un salto de línea al final
            else:
                file.write(text)  # Añade solo el texto sin salto de línea
    except IOError as e:
        # Manejar posibles errores de entrada/salida, como permisos insuficientes
        print(f"Ocurrió un error al abrir o escribir en el archivo: {e}")
    except Exception as e:
        # Manejar cualquier otro tipo de errores
        print(f"Ocurrió un error inesperado: {e}")



# Ejemplo de uso de la función
printf(HISTORICO_LOG, "Hola, este es un texto de prueba.")


In [2]:
# %%writefile cargar_llama_cpp.py
# from ctransformers import AutoModelForCausalLM, AutoTokenizer

import os
# import accelerate

import subprocess

# Definir las variables de entorno y las rutas
BASE_FOLDER = "./"
REPO = "QuantFactory"
TYPE_MODEL = "Meta-Llama-3-8B-Instruct-GGUF"
MODEL = "Meta-Llama-3-8B-Instruct.Q8_0.gguf"
MODEL_PATH = os.path.join(BASE_FOLDER, MODEL)
CONTEXT_LENGTH = 8192

# Crear el directorio base si no existe
if not os.path.exists(BASE_FOLDER):
    os.mkdir(BASE_FOLDER)
    print("Creado directorio base:", BASE_FOLDER)

# Descargar el modelo si no existe
if not os.path.exists(MODEL_PATH):
    url = f"https://huggingface.co/{REPO}/{TYPE_MODEL}/resolve/main/{MODEL}?download=true"
    cmd = f'curl -L "{url}" -o "{MODEL_PATH}"'
    print("Descargando:", MODEL)
    try:
        result = subprocess.run(cmd, shell=True, check=True)
        print("Descarga completa.")
    except subprocess.CalledProcessError as e:
        print("Error al descargar el archivo:", e)

# %cd {BASE_FOLDER}

from llama_cpp import Llama

model=None
# eos_token_id=None

# Función para cargar el modelo si aún no está cargado
def load_model():
    global model
    # global tokenizer
    if model is None:  # Verifica si model está vacío o no parece ser un modelo válido
        print("Cargando modelo...")
        # model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map='auto', load_in_8bit=True, trust_remote_code=True)
        # model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map='auto', torch_dtype="auto", trust_remote_code=True)
        enable_gpu = True  # offload LLM layers to the GPU (must fit in memory)

        model = Llama(
            model_path=MODEL_PATH,
            n_gpu_layers=-1 if enable_gpu else 0,
            n_ctx=CONTEXT_LENGTH,
            # verbose=False,
        )
        model.verbose=False

        print("Modelo cargado.")
    else:
        print("Modelo ya estaba cargado.")


load_model()

import whisper
modelWhisper = whisper.load_model('medium')


from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface

# Carga el modelo y la configuración
models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/fastspeech2-en-ljspeech",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)

# Asegúrate de que models es una lista
if not isinstance(models, list):
    models = [models]

modelT2S = models[0]
modelT2S = modelT2S.to('cuda:0')

TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

# Aquí, asumimos que task.build_generator puede manejar correctamente el objeto cfg y model
generator = task.build_generator(models, cfg)

Cargando modelo...


ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4080, compute capability 8.9, VMM: yes
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ./Meta-Llama-3-8B-Instruct.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count

Modelo cargado.


2024-05-10 10:36:14 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

2024-05-10 10:36:16 | INFO | fairseq.tasks.speech_to_text | dictionary size (vocab.txt): 75
/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
2024-05-10 10:36:17 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/javier/.cache/fairseq/models--facebook--fastspeech2-en-ljspeech/snapshots/a3e3e5e2e62bb7ca7514b11aa469e9c5b01a20bf/hifigan.bin


Server

In [7]:
# %%writefile server_conversacional.py

#########################################
####    MODEL IN LLAMA_CPP
#########################################

#MODELO LO CARGAMOS A PARTE PORQUE TARDA EN CARGARSE

#########################################
####    CHATBOT
#########################################
# %%writefile modelo_llama3.py
# from cargar_llama_cpp import model
LOGGING = False
from threading import Lock

# global model
def encontrar_coincidencia(texto, cadena_busqueda="<|eot_id|>"):
    """
    Esta función busca la primera aparición de una cadena de búsqueda en un texto dado y devuelve el substring
    desde el principio del texto hasta el final de esta coincidencia (incluida).
    
    Parámetros:
    texto (str): El texto en el que se buscará la cadena.
    cadena_busqueda (str): La cadena de caracteres que se buscará en el texto.
    
    Retorna:
    str: El substring desde el inicio hasta el final de la primera coincidencia de la cadena buscada,
    incluyendo la coincidencia. Si no se encuentra ninguna coincidencia, devuelve una cadena vacía.
    """
    # Buscar la posición de la primera coincidencia de la cadena en el texto
    indice = texto.find(cadena_busqueda)
    
    if indice != -1:
        # Devolver el substring desde el inicio hasta el final de la coincidencia
        return texto[:indice + len(cadena_busqueda)]
    else:
        # Devolver una cadena vacía si no hay coincidencia
        return ""


# VENTANA DESLIZANTE
def ajustar_contexto(texto, max_longitud=15000, secuencia="<|start_header_id|>", system_end="<|eot_id|>"):
    system_prompt = encontrar_coincidencia(texto, system_end)
    # Comprobar si la longitud del texto es mayor que el máximo permitido
    if len(texto) > max_longitud:
        indice_secuencia = 0

        while True:
            # Buscar la secuencia de ajuste
            indice_secuencia = texto.find(secuencia, indice_secuencia + 1)

            # Si la secuencia no se encuentra o el texto restante es menor que la longitud máxima
            if indice_secuencia == -1 or len(system_prompt) + len(texto) - indice_secuencia <= max_longitud:
                break

        # Si encontramos una secuencia válida
        if indice_secuencia != -1:
            return system_prompt + texto[indice_secuencia:]

        else:
            # Si no se encuentra ninguna secuencia adecuada, tomar los últimos max_longitud caracteres
            return system_prompt + texto[-max_longitud + len(system_prompt):]
    else:
        return system_prompt + texto



generate_lock = Lock()

def pre_warm_chat(historico, max_additional_tokens=100, stop=["</s>","user:"], short_answer=True, streaming=False, printing=False):
 
    # if short_answer:
    #     # añade como stop el salto de linea
    #     stop.append("\n")

    outputs = ""

    with generate_lock:
        response=model(prompt=historico, max_tokens=max_additional_tokens, temperature=0, top_p=1,
                    top_k=0, repeat_penalty=1,
                    stream=True)


        respuesta = ""

        for chunk in response:
            trozo = chunk['choices'][0]['text']
            # trozo.replace("\n", "")
            # trozo.replace("<|EOT|>", "")
            # for caracter in trozo:
            #     cadena_con_codigos += f"{caracter}({ord(caracter)}) "
            respuesta += trozo
            print(trozo, end="", flush=True)
            # linea += trozo

            # if len(linea)>35:
                # print(linea, end="", flush=True)  # Impresión en consola

                # linea = ""


        outputs = historico + respuesta
        return historico, outputs



import threading

class EstadoGeneracion:
    def __init__(self):
        # lista de 100 partes del texto
        self.parts = [""]*100
        self.top = -1
        self.generando = False
        # self.lock = threading.Lock()


estado_generacion = {}
estado_generacion['anonimo'] = EstadoGeneracion()

# generate_lock = Lock()

def generate_in_file_parts(userID, historico, ai, user, input_text, max_additional_tokens=2000, short_answer=False, streaming=True, printing=True):
    global estado_generacion

    printf(HISTORICO_LOG, f"generando para USER:{userID}\nhistorico:{historico}\ninput_text:{input_text}")
    # global generate_lock
    if userID not in estado_generacion:
        estado_generacion[userID] = EstadoGeneracion()

    estado_generacion[userID].generando = True
    with generate_lock:
        estado_generacion[userID].top = -1
        print(f"Empezamos a generar ponemos el TOP a {estado_generacion[userID].top} para USER:{userID}!!:", input_text)
        indiceParte = 0
        # estado_generacion[userID].generando = True
        print(f"generando={estado_generacion[userID].generando}; Generando respuesta para USER:{userID}:", input_text)
        # estado_generacion.parts = []  # lista de partes del texto
        parte_actual = ""  # añade la primera parte

        if short_answer:
            # añade como stop el salto de linea
            stop.append("\n")


        prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

        final_prompt = historico + "\n" + prompt


        model_inputs = final_prompt

        outputs = ""
        print(f"{ai}:", end="")


        # outputs = ""
        colchon = (CONTEXT_LENGTH - max_additional_tokens)*3
        print("Longitud:",len(final_prompt), "Colchon:", colchon)
        if len(final_prompt)> colchon: #cuenta la vieja cada token son 3 caracteres (como poco)
            print("Ajustando contexto!!!")
            final_prompt = ajustar_contexto(final_prompt, max_longitud=colchon)
            print(final_prompt)
            #contexto ajustado imprimir los primeros 500 caracteres
            # print(final_prompt[:500])
            #imprimir los 500 últimos caracteres
            # print(final_prompt[-500:])

        response=model(prompt=final_prompt, max_tokens=max_additional_tokens, temperature=0, top_p=1,
                      top_k=0, repeat_penalty=1,
                      stream=True)


        respuesta = ""

        for chunk in response:
            trozo = chunk['choices'][0]['text']
            # trozo.replace("\n", "")
            # trozo.replace("<|EOT|>", "")
            # for caracter in trozo:
            #     cadena_con_codigos += f"{caracter}({ord(caracter)}) "
            respuesta += trozo
            print(trozo, end="", flush=True)

            outputs += trozo
            parte_actual += trozo
            if trozo in ",;:.?!" and len(parte_actual)>44 or trozo in ".?!" and len(parte_actual)>4:
                estado_generacion[userID].parts[indiceParte] = parte_actual
                estado_generacion[userID].top = indiceParte
                if LOGGING:
                    print(f"trozo generado para USER: {userID}:", parte_actual)
                    print("se ha generado para entrada de indiceParte (ahora TOP tb vale esto):", indiceParte)
                indiceParte += 1                
                # print("se incrementa indiceParte (pero TOP aun no) a:", indiceParte)
                parte_actual = ""


        if len(parte_actual)>1:
            estado_generacion[userID].parts[indiceParte] = parte_actual
            estado_generacion[userID].top = indiceParte

        all_text = model_inputs + outputs + "<|eot_id|>"
        estado_generacion[userID].generando = False
        if LOGGING:
            print(f"generando={estado_generacion[userID].generando}; Respuesta Terminada. El total generado para {user}:", outputs)

        return all_text, outputs


#########################################
####    SERVER
#########################################
from flask import Flask, request, jsonify, send_file, send_from_directory
from flask_cors import CORS
from threading import Lock
import threading
import os
import torch
from pydub import AudioSegment
import pandas as pd
import random

import time


# import whisper
# modelWhisper = whisper.load_model('medium')



# parts = []  # lista de partes del texto
# generando = False
# global model



# from modelo_llama3 import generate_in_file_parts, pre_warm_chat
if LOGGING:
    print("El modelo es:", model)

ai = "assistant"
user = "user"

contexto = """

"""

system_prompt = """
You are a kind and helpful assistan bot. You are here to help the user to find the best answer to his question.
"""

saludo = "Hello, I am ready to receive and process your input."

idioma = "en"

import sys

# Verifica si el comando tenía flag -s o --short
if "-s" in sys.argv or "--short" in sys.argv:
    short_answer = True
else:
    short_answer = False

# Si encuentra el flag -es cambia el idioma a español
if "-es" in sys.argv:
    idioma = "es"

# Filtra los argumentos para eliminar los flags
args = [arg for arg in sys.argv[1:] if arg not in ["-s", "--short", "-es"]]

# Asigna los valores a system_prompt y saludo basándose en los argumentos restantes
if len(args) > 0:
    system_prompt = args[0]
if len(args) > 1:
    saludo = args[1]


historico = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{saludo}<|eot_id|>"


# load model
# load_model()

if LOGGING:
    print(f"{ai}:", saludo)

# Crea un bloqueo para proteger el código contra la concurrencia a la hora de transcribir
transcribe_lock = Lock()


# generate_lock = Lock()

app = Flask(__name__)
# app.config['MAX_CONTENT_LENGTH'] = 30 * 1024 * 1024  # 30 MB

#import logging
#logging.basicConfig(level=logging.DEBUG)

CORS(app)

output = ""


@app.route('/alive')
def alive():
    return jsonify(True)



def eliminar_archivos_temp(nombre_inicio='temp_sync'):
    # Obtener una lista de todos los archivos en el directorio actual
    archivos = os.listdir('.')
    
    # Filtrar archivos que comienzan con 'temp_sync'
    archivos_temp = [archivo for archivo in archivos if archivo.startswith(nombre_inicio)]
    
    # Iterar sobre la lista de archivos y eliminarlos
    for archivo in archivos_temp:
        try:
            os.remove(archivo)
            print(f"Archivo eliminado: {archivo}")
        except Exception as e:
            print(f"No se pudo eliminar el archivo {archivo}. Razón: {e}")

@app.route('/inicio', methods=['POST'])
def print_strings():
    # global modelo
    # global historico
    eliminar_archivos_temp("received_audio")
    historico = ""

    # Lee el archivo CSV y selecciona un personaje de ficción al azar
    def elegir_personaje_aleatorio():
        df = pd.read_csv('Personajes_ficcion.csv')
        return random.choice(df.iloc[:, 0].tolist())

    # Obtiene los datos del cuerpo de la solicitud
    data = request.json

    # Extrae los strings del objeto JSON
    system_prompt = data.get('system_prompt')
    saludo = data.get('saludo')

    # Preprocesamiento para reemplazar "#personaje" con un personaje aleatorio
    if "#personaje" in system_prompt:
        personaje_aleatorio = elegir_personaje_aleatorio()
        system_prompt = system_prompt.replace("#personaje", personaje_aleatorio)

    # Preprocesamiento para reemplazar "#personaje" con un personaje aleatorio en el saludo
    if "#personaje" in saludo:
        saludo = saludo.replace("#personaje", personaje_aleatorio)

    # Imprime los strings en el log del servidor
    if LOGGING:
        print("INICIALIZANDO CONVERSACIÓN")
    conversation_file = 'conversacion.mp3'
    # si existe el archivo de conversación, lo elimina
    if os.path.exists(conversation_file):
        os.remove(conversation_file)

    if LOGGING:
        print(f"system: {system_prompt}, saludo: {saludo}")

    # if modelo == "mistral":
    #     historico = f"system\n{system_prompt}\nassistant\n{saludo}\n"
    # elif modelo == "zypher":
    #     historico = f"{system_prompt}</s>\n\n{saludo}</s>\n"
    historico = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{saludo}<|eot_id|>"

    pre_warm_chat(historico)

    # Retorna una respuesta para indicar que se recibieron y procesaron los datos
    return jsonify({"message": saludo, "historico": historico}), 200


@app.route('/get-translations-file', methods=['GET'])
def get_translations():
    return send_from_directory(directory='.', path='translations.csv', as_attachment=True)

import csv
import shutil
@app.route('/save-translations-file', methods=['POST'])
def save_translations():
    data = request.json  # Asume que el cliente envía los datos como JSON
    if not data:
        return jsonify({'error': 'No data provided'}), 400

    try:
        # Hace una copia de seguridad del archivo translations.csv antes de modificarlo
        shutil.copy('translations.csv', 'translations.csv.bak')

        # Abre el archivo translations.csv para escribir y actualiza con los datos recibidos
        with open('translations.csv', mode='w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile, delimiter='#')
            for row in data:
                writer.writerow(row)

        return jsonify({'message': 'File successfully saved'}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/all_conversation', methods=['GET'])
def all_conversation():
    # Asegúrate de que el path al archivo sea correcto para tu estructura de proyecto
    filepath = 'conversacion.mp3'
    if not os.path.exists(filepath):
        return jsonify(error="Archivo de conversación no encontrado"), 404

    # Leer el archivo y convertirlo a base64
    with open(filepath, 'rb') as audio_file:
        audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

    return jsonify(audio_base64=audio_base64)



import subprocess

def convert_ogg_to_mp3(source_ogg_path, target_mp3_path):
    """
    Utiliza ffmpeg para convertir un archivo .ogg a .mp3.
    """
    command = ['ffmpeg', '-y' ,'-i', source_ogg_path, target_mp3_path]
    process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Si el comando falla, imprime la salida de error
    if process.returncode != 0:
        print(f"Error al convertir {source_ogg_path} a {target_mp3_path}")
        print("Salida de error de ffmpeg:")
        print(process.stderr.decode())




def convert_wav_to_mp3(source_wav_path, target_mp3_path):
    command = ['ffmpeg', '-i', source_wav_path, target_mp3_path]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


def add_audio_to_conversation_async(source_path, convert_to_mp3=False):
    def task():
        if convert_to_mp3:
            # Convertir de WAV a MP3 si es necesario
            temp_mp3_path = source_path.replace('.wav', '.mp3')
            convert_wav_to_mp3(source_path, temp_mp3_path)
            final_path = temp_mp3_path
        else:
            final_path = source_path

        # Añadir al archivo de conversación
        sound = AudioSegment.from_file(final_path)
        conversation_file = 'conversacion.mp3'
        if os.path.exists(conversation_file):
            conversation_audio = AudioSegment.from_mp3(conversation_file)
            combined_audio = conversation_audio + sound
        else:
            combined_audio = sound
        combined_audio.export(conversation_file, format='mp3')

        # Limpiar archivos temporales
        os.remove(source_path)
        if convert_to_mp3:
            os.remove(temp_mp3_path)

    thread = threading.Thread(target=task)
    thread.start()



def generate_chat_background(userID, entrada, phistorico, ai, user, short_answer):
    # global output  # Indicar que se utilizará la variable global 'output'
    if LOGGING:
        print("OJOOOOOOOO!!!!!!  generate_chat_background USERID:", userID, "entrada:", entrada)
    start_generation_time = time.time()
    # Ejecutar la generación de chat en un hilo aparte
    historico_local, output_local = generate_in_file_parts(userID, phistorico, ai, user, input_text=entrada, max_additional_tokens=2048, short_answer=short_answer, streaming=True, printing=False)
    end_generation_time = time.time()
    generation_duration = end_generation_time - start_generation_time
    if LOGGING:
        print(f"Generación completada en {generation_duration} segundos")

    # Actualizar las variables globales con los resultados obtenidos
    # global historico
    # historico = historico_local
    # output = output_local

@app.route('/transcribe', methods=['POST'])
def transcribe_audio():
    if LOGGING:
        print("Transcribiendo audio...")
    global user
    global ai


    if 'userID' not in request.form:
        return jsonify(error="No se proporcionó userID"), 400
    userID = int(request.form['userID'])

    if 'historico' not in request.form:
        return jsonify(error="No se proporcionó historico"), 400
    historico = request.form['historico']  # Asumiendo que se envía como JSON y necesitará ser parseado en Python

    # Extraer el archivo
    if 'file' not in request.files:
        return jsonify(error="No se proporcionó file"), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify(error="No selected file"), 400

    timestamp = int(time.time() * 1000)
    ogg_filepath = f"received_audio_{timestamp}.ogg"
    file.save(ogg_filepath)

    start_transcribe_time = time.time()
    if LOGGING:
        print("antes del transcribe lock, userID:", userID)
    with transcribe_lock:
        if LOGGING:
            print("después del transcribe lock, userID:", userID)
        transcripcion = modelWhisper.transcribe(ogg_filepath, fp16=False, language=idioma)
        transcripcion = transcripcion["text"]
    end_transcribe_time = time.time()
    transcribe_duration = end_transcribe_time - start_transcribe_time
    if LOGGING:
        print(f"Transcripción completada en {transcribe_duration} segundos")
        print("transcripción:", transcripcion)

    # Iniciar la generación de chat en un hilo aparte
    thread = threading.Thread(target=generate_chat_background, args=(userID, transcripcion, historico, ai, user, short_answer))
    thread.start()

    # Inicia el proceso de adición del audio .ogg en segundo plano, considerando su conversión a .mp3
    add_audio_to_conversation_async(ogg_filepath)

    # La respuesta ya no incluirá 'output' porque se generará en segundo plano
    prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{transcripcion}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    historico += prompt    
    return jsonify(entrada=transcripcion, prompt=prompt, entrada_traducida="")
    # return jsonify(entrada=transcripcion, historico=historico, entrada_traducida="")





@app.route('/only_transcribe', methods=['POST'])
def only_transcribe_audio():
    if LOGGING:
        print("Transcribiendo audio...")
    # global historico
    # global user
    # global ai

    if 'file' not in request.files:
        print("No file part")
        return jsonify(error="No file part"), 400

    file = request.files['file']
    if file.filename == '':
        print("No selected file")
        return jsonify(error="No selected file"), 400

    if LOGGING:
        print("creando fichero ogg audio antes de transcripción...")
    timestamp = int(time.time() * 1000)
    ogg_filepath = f"received_audio_{timestamp}.ogg"
    file.save(ogg_filepath)

    start_transcribe_time = time.time()
    if LOGGING:
        print("antes del transcribe lock")
    with transcribe_lock:
        if LOGGING:
            print("después del transcribe lock")
        transcripcion = modelWhisper.transcribe(ogg_filepath, fp16=False, language=idioma)
        transcripcion = transcripcion["text"]
    end_transcribe_time = time.time()
    transcribe_duration = end_transcribe_time - start_transcribe_time
    if LOGGING:
        print(f"Transcripción completada en {transcribe_duration} segundos")
        print("transcripción:", transcripcion)

    # Iniciar la generación de chat en un hilo aparte
    #thread = threading.Thread(target=generate_chat_background, args=(transcripcion, historico, ai, user, short_answer))
    #thread.start()

    # Inicia el proceso de adición del audio .ogg en segundo plano, considerando su conversión a .mp3
    add_audio_to_conversation_async(ogg_filepath)

    # La respuesta ya no incluirá 'output' porque se generará en segundo plano
    return jsonify(entrada=transcripcion, entrada_traducida="")



@app.route('/get_next_part', methods=['GET'])
def get_next_part():
    global estado_generacion
    if LOGGING:
        print("LAS CLAVES de usuario y sus TOP:")
        for clave in estado_generacion.keys():
            print(clave," TOP:", estado_generacion[clave].top)

    userID = request.args.get('userID', default=0, type=int)
    # Obtener el índice de la solicitud. Si no se proporciona, por defecto es None
    index = request.args.get('index', default=0, type=int)

    if LOGGING:
        print(f"userID:{userID} partes: {estado_generacion[userID].parts}, generando: {estado_generacion[userID].generando}, index: {index}, estado_generacion[userID].top: {estado_generacion[userID].top}")

    while True:
        # if estado_generacion.parts:
            # Verificar si el índice es válido
        if index is not None and index >= 0 and index <= estado_generacion[userID].top:

            part = estado_generacion[userID].parts[index]
            estado_generacion[userID].parts[index] = ""  # Elimina el elemento en el índice dado
            if LOGGING:
                print("con index:", index, "estado_generacion[userID].top:", estado_generacion[userID].top)    
                print(f"Enviando parte: {part}")
            return jsonify(output=part)
            # else:
            #     print("Índice inválido o fuera de límites")
            #     return jsonify(error="Índice inválido o fuera de límites"), 400
        elif estado_generacion[userID].generando:
            if LOGGING:
                print("Esperando a que se generen más partes...")
            time.sleep(0.1)  # Espera 0.1 segundos antes de volver a verificar
        else:
            if LOGGING:
                print("No hay más partes para enviar", "index:", index, "estado_generacion[userID].top:", estado_generacion[userID].top)
            return jsonify(output="") # Si 'generando' es False y 'parts' está vacía, devuelve una cadena vacía



@app.route('/texto', methods=['POST'])
def process_text():
    # global historico
    global user
    global ai

    # Recibe el texto directamente del cuerpo de la solicitud
    data = request.json
    if not data or 'texto' not in data:
        return jsonify(error="No se proporcionó texto"), 400

    texto = data['texto']

    if 'historico' not in data:
        return jsonify(error="No se proporcionó historico"), 400

    historico = data['historico']

    if 'userID' not in data:
        return jsonify(error="No se proporcionó userID"), 400

    userID = data['userID']

    if LOGGING:
        print("HISTORICO!!!:", historico)

    # Utiliza la variable 'idioma' declarada globalmente
    global idioma


    # Generación de respuesta basada en el texto proporcionado
    thread = threading.Thread(target=generate_chat_background, args=(userID, texto, historico, ai, user, short_answer))
    thread.start()


    # si el idioma es español, traduce la respuesta al español

    prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{texto}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    historico += prompt
    return jsonify(entrada=texto, historico=historico, entrada_traducida="")



import torch  # Importamos PyTorch para poder usar la función `to()`

# Función para mover recursivamente todos los tensores en una estructura anidada a un dispositivo
def move_to_device(obj, device):
    if isinstance(obj, torch.Tensor):
        return obj.to(device)
    elif isinstance(obj, dict):
        return {key: move_to_device(value, device) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [move_to_device(item, device) for item in obj]
    else:
        return obj


# PREPARAMOS INSTANCIAS FAIRSEQ

if idioma == "en":
    print("idioma ingles") # ESTO HAY QUE CAMBIARLO 
    # from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    # from fairseq.models.text_to_speech.hub_interface import TTSHubInterface

    # # Carga el modelo y la configuración
    # models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    #     "facebook/fastspeech2-en-ljspeech",
    #     arg_overrides={"vocoder": "hifigan", "fp16": False}
    # )

    # # Asegúrate de que models es una lista
    # if not isinstance(models, list):
    #     models = [models]

    # modelT2S = models[0]
    # modelT2S = modelT2S.to('cuda:0')

    # TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # # Aquí, asumimos que task.build_generator puede manejar correctamente el objeto cfg y model
    # generator = task.build_generator(models, cfg)



elif idioma == "es":
    # El modelo no entiende de números aritméticos. Esta función los convierte a palabras.
    import re
    from num2words import num2words

    def number_to_words(num_str):
        try:
            num = int(num_str)
            return num2words(num, lang='es')
        except ValueError:
            return "Por favor, introduzca un número válido."

    def process_numbers_in_line(line):
        def replace_with_words(match):
            return number_to_words(match.group())

        return re.sub(r'\b\d+\b', replace_with_words, line)

    # Ejemplo de uso
    line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
    new_line = process_numbers_in_line(line)
    print(new_line)
    # Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."


    # Diccionario con las traducciones

    def process_abrev(line):
        translations = {
        'Dr': 'doctor',
        'Sr': 'señor',
        'Sra': 'señora',
        # Añade más traducciones aquí
    }
        for abbr, full in translations.items():
            line = line.replace(f'{abbr}.', full)
            line = line.replace(f'{abbr} ', f'{full} ')
        return line

    def otras_traducciones(line):

        translations = {
        '-': ',',
        '—': ',',
        '%': ' por ciento '
        # Añade más traducciones aquí
        }

        for old, new in translations.items():
            line = line.replace(old, new)
        return line


    def preprocesado_al_modelo(line):
        line_with_numbers = process_numbers_in_line(line)
        line_with_both = process_abrev(line_with_numbers)
        line_with_all = otras_traducciones(line_with_both)
        return line_with_all

    import os
    os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
    print(os.environ['TF_ENABLE_ONEDNN_OPTS'])


    from pydub import AudioSegment, silence

    def quitar_silencios(input_filepath, output_filepath, min_silence_len=1500, new_silence_len=750, silence_thresh=-60):
        """
        Elimina silencios largos de un archivo de audio.

        Parámetros:
        - input_filepath: ruta al archivo de audio de entrada (MP3).
        - output_filepath: ruta al archivo de audio de salida (MP3).
        - min_silence_len: duración mínima del silencio a eliminar (en milisegundos).
        - new_silence_len: duración de los nuevos segmentos de silencio (en milisegundos).
        - silence_thresh: umbral de silencio (en dB).
        """

        # Cargar el archivo de audio
        audio_segment = AudioSegment.from_wav(input_filepath)

        # Encuentra los segmentos de audio separados por silencios
        segments = silence.split_on_silence(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

        # Crear un nuevo segmento de audio con silencios ajustados
        new_audio_segment = AudioSegment.empty()
        silence_chunk = AudioSegment.silent(duration=new_silence_len)  # Chunk de silencio de la nueva duración

        # Añade cada segmento de audio al nuevo audio, intercalando con los nuevos segmentos de silencio
        for segment in segments:
            new_audio_segment += segment + silence_chunk

        # Removemos el último chunk de silencio añadido
        new_audio_segment = new_audio_segment[:-new_silence_len]

        # Guarda el nuevo archivo de audio
        new_audio_segment.export(output_filepath, format="wav")


    # Cargamos el modelo generador fairseq
    from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
    import IPython.display as ipd





    # Cargamos el modelo y la configuración desde el modelo preentrenado de Hugging Face
    models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
        "facebook/tts_transformer-es-css10",
        arg_overrides={"vocoder": "hifigan", "fp16": False}
    )
    modelT2S = models[0]

    # Movemos el modelo al dispositivo GPU
    modelT2S = modelT2S.to('cuda:0')

    # Actualizamos la configuración con los datos del task
    TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # Creamos el generador
    generator = task.build_generator([modelT2S], cfg)


    import torchaudio

    import re

    def dividir_texto_con_minimo_palabras(texto, min_palabras=8):
        partes = re.split(r'([.,;:?!])', texto)
        partes_filtradas = [parte.strip() for parte in partes if parte.strip()]
        partes_combinadas = []
        parte_actual = ''

        for parte in partes_filtradas:
            if parte in '.,;:?!':
                parte_actual += parte
                if len(parte_actual.split()) >= min_palabras:
                    partes_combinadas.append(parte_actual)
                    parte_actual = ''
                else:
                    parte_actual += ' '
            else:
                parte_actual += parte + ' '

        if len(parte_actual.strip()) > 10:
            partes_combinadas.append(parte_actual.strip())

        return partes_combinadas

    def combinar_audios(audios_temporales):
        audio_combinado = "audio_combinado.wav"
        # Cargar el primer archivo de audio para inicializar la concatenación
        wav_total, rate = torchaudio.load(audios_temporales[0])

        # Iterar sobre los archivos restantes y concatenarlos
        for archivo in audios_temporales[1:]:
            wav, _ = torchaudio.load(archivo)
            wav_total = torch.cat((wav_total, wav), 1)

        # Guardar el audio combinado en un archivo final
        torchaudio.save(audio_combinado, wav_total, rate)

        return audio_combinado

    def voz_sintetica_spanish(text):
        text = preprocesado_al_modelo(text)

        lista_dividida = dividir_texto_con_minimo_palabras(text)

        audios_temporales = []

        for parte in lista_dividida:
            # Preparamos los datos de entrada para el modelo
            sample = TTSHubInterface.get_model_input(task, parte)

            # Movemos los datos al dispositivo GPU
            sample = move_to_device(sample, 'cuda:0')

            # Realizamos la predicción
            wav, rate = TTSHubInterface.get_prediction(task, modelT2S, generator, sample)

            if len(wav.shape) == 1:
                wav = wav.unsqueeze(0)

            # temp_file_name = "Temporal.wav"
            temp_file_name = f"temporal_{parte[:10]}.wav"
            torchaudio.save(temp_file_name, wav.to('cpu'), rate)
            audios_temporales.append(temp_file_name)

            combinado = combinar_audios(audios_temporales)
            sin_silencios = "sin_silencios.wav"
            # quitamos silencios
            quitar_silencios(combinado, sin_silencios, min_silence_len=1500, new_silence_len=750, silence_thresh=-60)


        with open(sin_silencios, "rb") as audio_file:
            audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

        return audio_base64



import base64
@app.route('/audio', methods=['POST'])
def generate_audio():
    texto = request.json.get('texto')
    pausa = request.json.get('pausa')
    if LOGGING:
        print('PAUSA!!!!!!!!:', pausa)
        print('TEXTO!!!!!!!!:', texto)
    if not texto:
        return jsonify(error="No se proporcionó texto"), 400

    if idioma == "en":
        audio_base64 = voz_sintetica_english(texto, pausa)
        return jsonify(audio_base64=audio_base64)
    elif idioma == "es":
        audio_base64 = voz_sintetica_spanish(texto)
        return jsonify(audio_base64=audio_base64)

import base64
import io
import soundfile as sf


def add_comma_after_punctuation(text: str) -> str:
    # Lista de caracteres después de los cuales se debe agregar una coma
    punctuation_marks = ['.', '!', '?', '(', ')', ':', '\n']

    # Recorre cada marca de puntuación y añade una coma después de cada ocurrencia
    for mark in punctuation_marks:
        text = text.replace(mark, mark + ',...,')

    return text

# Ejemplo de uso de la función
#example_text = "Hello! How are you? I hope you're doing well. Let's meet tomorrow."
#modified_text = add_comma_after_punctuation(example_text)
#print(modified_text)

import io
import base64
import soundfile as sf
import os
import threading
from pydub import AudioSegment
import subprocess


def add_silence_to_audio(audio_path, duration_ms=3000):
    """Añade un segmento de silencio al final de un archivo de audio."""
    # Carga el audio
    sound = AudioSegment.from_wav(audio_path)
    # Genera el silencio
    silence = AudioSegment.silent(duration=duration_ms)
    # Concatena el audio con el silencio
    combined = sound + silence
    # Guarda el nuevo archivo
    combined.export(audio_path, format='wav')


def voz_sintetica_english(texto, pausa="true"):
    if pausa == "true":
        texto = add_comma_after_punctuation(texto)
    # Preparamos los datos de entrada para el modelo
    sample = TTSHubInterface.get_model_input(task, texto)

    # Movemos los datos al dispositivo GPU
    sample = move_to_device(sample, 'cuda:0')

    # Realizamos la predicción
    wav, rate = TTSHubInterface.get_prediction(task, modelT2S, generator, sample)


        # Convertimos el tensor wav a un buffer de audio en memoria y luego a un archivo temporal
    temp_wav_path = f"temp_synth_audio_{int(time.time() * 1000)}.wav"
    with io.BytesIO() as audio_buffer:
        sf.write(audio_buffer, wav.cpu().numpy(), rate, format='WAV')
        audio_buffer.seek(0)  # Regresamos al inicio del buffer para leerlo
        # Guardar en un archivo temporal
        with open(temp_wav_path, 'wb') as f:
            f.write(audio_buffer.read())

  

    # if len(texto) <= 20:
    #     # Añade silencio al final del archivo de audio
    #     add_silence_to_audio(temp_wav_path, 1000)  # Añade 1 segundo de silencio para que no de problemas en audios cortos

    if len(texto) <= 30:
        print("Añadiendo 700 milisegundos de silencio")
        add_silence_to_audio(temp_wav_path, 700)

    elif len(texto) <= 44:
        print("Añadiendo 0.5 segundos de silencio")
        add_silence_to_audio(temp_wav_path, 500)

    # Añadir el audio al archivo de conversación en segundo plano
    add_audio_to_conversation_async(temp_wav_path, convert_to_mp3=True)  # Asegúrate de implementar la conversión dentro de esta función si es necesario


    # Convertir el buffer a base64 para retornar
    with open(temp_wav_path, 'rb') as f:
        audio_base64 = base64.b64encode(f.read()).decode('utf-8')


    return audio_base64


def print_routes(app):
    print("Endpoints disponibles:")
    for rule in app.url_map.iter_rules():
        methods = ','.join(sorted(rule.methods))
        print(f"{rule.endpoint}: {rule.rule} [{methods}]")


if __name__ == '__main__':
    print_routes(app)
    app.run(host='0.0.0.0', port=5500, threaded=True)
    # app.run(host='0.0.0.0', port=5500, threaded=True, ssl_context=('cert.pem', 'key.pem'))




idioma ingles
Endpoints disponibles:
static: /static/<path:filename> [GET,HEAD,OPTIONS]
alive: /alive [GET,HEAD,OPTIONS]
print_strings: /inicio [OPTIONS,POST]
get_translations: /get-translations-file [GET,HEAD,OPTIONS]
save_translations: /save-translations-file [OPTIONS,POST]
all_conversation: /all_conversation [GET,HEAD,OPTIONS]
transcribe_audio: /transcribe [OPTIONS,POST]
only_transcribe_audio: /only_transcribe [OPTIONS,POST]
get_next_part: /get_next_part [GET,HEAD,OPTIONS]
process_text: /texto [OPTIONS,POST]
generate_audio: /audio [OPTIONS,POST]
 * Serving Flask app '__main__'
 * Debug mode: off


2024-05-10 11:27:07 | INFO | werkzeug | WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5500
 * Running on http://172.25.241.247:5500
2024-05-10 11:27:07 | INFO | werkzeug | Press CTRL+C to quit


2024-05-10 11:27:15 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:27:15] "GET /alive HTTP/1.1" 200 -
2024-05-10 11:27:16 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:27:16] "OPTIONS /inicio HTTP/1.1" 200 -


assistant

Nice to meet you! I'm Sofie, your English teacher. How are you today?

2024-05-10 11:27:16 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:27:16] "POST /inicio HTTP/1.1" 200 -
2024-05-10 11:27:17 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:27:17] "OPTIONS /audio HTTP/1.1" 200 -
2024-05-10 11:27:17 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:27:17] "POST /audio HTTP/1.1" 200 -


...

2024-05-10 11:27:48 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:27:48] "OPTIONS /audio HTTP/1.1" 200 -
2024-05-10 11:27:49 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:27:49] "POST /audio HTTP/1.1" 200 -
2024-05-10 11:27:49 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:27:49] "GET /get_next_part?index=13&userID=299691 HTTP/1.1" 200 -
2024-05-10 11:27:49 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:27:49] "OPTIONS /audio HTTP/1.1" 200 -
2024-05-10 11:27:50 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:27:50] "POST /audio HTTP/1.1" 200 -
2024-05-10 11:27:50 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:27:50] "GET /get_next_part?index=14&userID=299691 HTTP/1.1" 200 -
2024-05-10 11:27:50 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:27:50] "OPTIONS /audio HTTP/1.1" 200 -
2024-05-10 11:27:51 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:27:51] "POST /audio HTTP/1.1" 200 -
2024-05-10 11:27:51 | INFO | werkzeug | 172.25.240.1 - - [1

Añadiendo 0.5 segundos de silencio


2024-05-10 11:28:00 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:28:00] "GET /get_next_part?index=26&userID=299691 HTTP/1.1" 200 -
2024-05-10 11:28:01 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:28:01] "OPTIONS /audio HTTP/1.1" 200 -
2024-05-10 11:28:01 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:28:01] "POST /audio HTTP/1.1" 200 -
2024-05-10 11:28:01 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:28:01] "GET /get_next_part?index=27&userID=299691 HTTP/1.1" 200 -


Añadiendo 700 milisegundos de silencio


2024-05-10 11:28:30 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:28:30] "GET /alive HTTP/1.1" 200 -
2024-05-10 11:29:00 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:29:00] "GET /alive HTTP/1.1" 200 -
2024-05-10 11:29:30 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:29:30] "GET /alive HTTP/1.1" 200 -
2024-05-10 11:29:35 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:29:35] "GET /all_conversation HTTP/1.1" 200 -
2024-05-10 11:30:00 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:30:00] "GET /alive HTTP/1.1" 200 -
2024-05-10 11:30:30 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:30:30] "GET /alive HTTP/1.1" 200 -
2024-05-10 11:31:00 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:31:00] "GET /alive HTTP/1.1" 200 -
2024-05-10 11:31:30 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:31:30] "GET /alive HTTP/1.1" 200 -
2024-05-10 11:32:00 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 11:32:00] "GET /alive HTTP/1.1" 200 -


# Cliente

In [26]:
%%writefile Llama3JuegoClienteUserID.html

<head>
  <style>


    body {
        font-family: Arial, sans-serif; /* Mejora la tipografía general */
    }


    #system_prompt {
        height: 150px;
    }


    #inicioForm {
        max-width: 1000px; /* Limita el ancho del formulario */
        margin: 20px auto; /* Centra el formulario */
        padding: 20px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Añade un sombreado ligero */
    }

    #inicioForm div {
        margin-bottom: 15px; /* Añade más espacio entre los campos */
    }

    #inicioForm label {
        font-weight: bold; /* Hace que las etiquetas sean más notables */
        display: block; /* Asegura que la etiqueta esté encima del input */
        margin-bottom: 5px; /* Espacio entre la etiqueta y el campo */
    }

    #inicioForm select, #inicioForm textarea, #inicioForm button {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
    }

    #inicioForm select {
        cursor: pointer; /* Indica que es un elemento interactivo */
        font-size: 16px; /* Aumenta el tamaño del texto */
    }

    #inicioForm textarea {
        resize: vertical; /* Permite al usuario ajustar la altura verticalmente */
    }

    #inicioForm button {
        background-color: #007bff; /* Color de fondo */
        color: white; /* Color del texto */
        border: none; /* Elimina el borde */
        padding: 10px 15px; /* Añade relleno */
        font-size: 18px; /* Aumenta el tamaño del texto */
        cursor: pointer; /* Indica que es un elemento interactivo */
        border-radius: 5px; /* Bordes redondeados */
    }

    #inicioForm button:hover {
        background-color: #0056b3; /* Oscurece el botón al pasar el mouse */
    }

#audioPlayerContainer {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#recordButton {
    background-color: #f44336; /* Color rojo para la grabación */
    color: white;
    border: none;
    padding: 10px 0;
    font-size: 18px;
    border-radius: 5px;
    cursor: pointer;
}

#recordButton:hover {
    background-color: #d32f2f; /* Oscurece el botón al pasar el mouse */
}

/* Estilos para el área de texto y botón de envío */
/* Contenedor del área de texto y el botón */
div#textButtonContainer {
    display: flex; /* Establece el contenedor para usar flexbox */
    justify-content: space-between; /* Espacia los elementos uniformemente */
    align-items: center; /* Alinea los elementos verticalmente en el centro */
}

/* Área de texto */
#textInput {
    flex-grow: 1; /* Permite que el área de texto crezca para ocupar el espacio disponible */
    margin-right: 10px; /* Añade un margen a la derecha para separarlo del botón */
    border: 1px solid #ccc; /* Establece un borde sutil */
    border-radius: 5px; /* Bordes redondeados */
    padding: 8px; /* Añade padding interno */
}

/* Botón */
#sendTextButton {
    padding: 8px 15px; /* Ajusta el padding para dimensionar el botón */
    background-color: #4CAF50; /* Color de fondo */
    color: white; /* Color del texto */
    border: none; /* Elimina el borde */
    border-radius: 5px; /* Bordes redondeados */
    cursor: pointer; /* Cambia el cursor a mano al pasar sobre el botón */
}

#sendTextButton:hover {
    background-color: #388E3C; /* Oscurece el botón al pasar el mouse */
}


#responseText {
    height: 250px;
    margin-top: 20px;
    border: 1px solid #ddd;
    padding: 10px;
    overflow-y: auto; /* Asegura el desplazamiento vertical */
    background-color: #f9f9f9; /* Fondo claro para resaltar el área */
}



</style>
</head>

<script>

window.intervalId = window.intervalId || null; // Asegura que intervalId sea global y única

function startInterval() {
    if (window.intervalId === null) {
        window.intervalId = setInterval(() => {
            fetch('http://localhost:5500/alive')
                .then(response => response.json())
                .then(data => console.log('Alive:', data))
                .catch(error => console.error('Error fetching alive status:', error));
        }, 30000);
        console.log('Intervalo iniciado.');
    } else {
        console.log('Ya existe un intervalo en ejecución.');
    }
}

function stopInterval() {
    if (window.intervalId !== null) {
        clearInterval(window.intervalId);
        console.log('Intervalo detenido.');
        window.intervalId = null;
    } else {
        console.log('No hay un intervalo para detener.');
    }
}


// startInterval()

// document.getElementById('btnAccesoMic').addEventListener('click', async () => {
//     try {
//         const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
//         // Procesa el stream aquí
//         console.log('Acceso al micrófono concedido');
//     } catch (error) {
//         console.error('Acceso al micrófono denegado:', error);
//     }
// });

document.addEventListener('DOMContentLoaded', function() {
    window.scrollTo(0, 0); // Asegura que la página comience en la parte superior
    document.getElementById('ejercicios').focus(); // Luego establece el foco en el selector
});



// Objeto para mapear los ejercicios a sus strings correspondientes
const ejerciciosStrings = {
    "guessing_game1": {
        systemPrompt: `This is a conversational game in which you have to think in this famous character: #personaje, and the user have to guess this character. You should aswer the user questions about the character. Be concise but give some clues. NEVER say the name of the character until the end. If the user guesses the character then you will say: "Congratulations, you got it right, the character was #personaje". If user give up you will say: "what a shame!, the character was #personaje
Game success example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he #personaje?
assistant: Congratulations, you were right, it was #personaje.
Game give up example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he Benito Perez?
assistant: No, it has nothing to do with.
user: I give up. Who is it?
assistant: what a shame!, the character was #personaje
`,
        saludo: "I'm thinking of a famous fictional character, guess which one it is."
    },
    "guessing_game2": {
        systemPrompt: "You are #personaje, the fictional character. You have to take on the personality of that character and engage in conversations about your events and experiences.",
        saludo: "I'm #personaje, the fictional character. Ask me anything you want to know about me."
    },
    "guessing_game3": {
        systemPrompt: "This is a conversational game in which you have to guess a famous character. You should make questions to the user in order to guess the character that the user have choosen.",
        saludo: "Do you want to play? I will guess your choosen character by asking about it."
    },
    "yes_no_game": {
        systemPrompt: `IMPORTANT: You only can answer "yes" or "no" (nothing more!).
This is a conversational game between you and a the user. The game consists that only at the beggininig you tell the user only a piece of the context and the user having to guess the "key point" of the context from "yes or no questions". The User could ask anything about the story (context) to guess the "key point" of the context but Assistant could only answer "Yes" or "not". If the user asks a question that does not lend itself or cannot be answered with a "yes" or "no" such as "What is the man's name?" then Assistant will respond: "Only "yes or no" questions. When the user guesses the key point of the story you will say: "Congratulations, you have guessed the key to the story.
Game context: A man named Edgar is the lighthouse keeper of Águilas for 30 years. He always turns on the lighthouse at dusk and shortly after sleeps in a small room next to its large lamp. On Edgar's birthday, at dusk after lighting the lighthouse, he decided to go to dinner with an old friend to celebrate his birthday. During dinner he drank more than necessary and they both got drunk. Afther the dinner Edgar accompanied his friend to her house and then went to his house, the lighthouse, where he sleeps every night. Upon entering the lighthouse and going up to his room, due to his drunkenness and the fact that he was very sleepy, he decided to turn off the light (which was actually the light from the main lamp of the lighthouse) to sleep off the drunkenness and did it without realizing or knowing it was dangerous. During the early hours of the morning, a cruise ship full of passengers crashed into the cliff that the lighthouse protected because, when it was turned off, neither the lookout, nor the captain, nor the rest of the crew nor the passengers could see that they were heading against the cliff. An hour later Edgar wakes up, it hasn't dawned yet but you can hear sirens and a lot of noise from the rescuers who are trying to rescue the shipwrecked. Edgar turns on the lamp to illuminate the scene where hundreds of dead shipwrecked people continually crash against the cliff due to the waves. Faced with this heartbreaking reality and his feeling of guilt, Edgar decides to commit suicide by jumping from the top of the lighthouse. The key point of the story that the user must find out is: "Edgar commits suicide because he was the LIGHTHOUSE keeper." or similar but always emphasizing that he was the lighthouse keeper.
IMPORTANT: You only can answer "yes" or "no" or "Only yes or no questions"
Examples of correct answers:
user: What is Edgar's job?
Assistant: Only yes or no questions.
user: Is Edgar a man?
Assistant: yes.`,
        saludo: `This is I can show about the hidden story: Edgar was dazed and comes to his room, turns off the light and lies down on his bed. He wakes up a few hours later, turns on the light, looks out the window and is so horrified that he ends up jumping out of the window and committing suicide.
Guess what happened.
IMPORTANT: From now on I can only answer you "yes" or "no" and nothing more.`
    },
    "English_teacher": {
        systemPrompt: `You are Sofie, an english teacher 29 years old. You will have simple dialogues with the student in your charge. you will only have concise conversations with short sentences so that the student is encouraged to converse.
Also you can offer translations exercises but only from spainish to enlgish. if you do translations exercises about a topic you must always say the sentence to translate in spanish: How do you say 'me gustaría viajar a París'? and then de user will be able response in english. You will not ask for doing translates from english to spanish, only from spanish to english.`,
        saludo: "Good Morning. What is your name?"
    }
    // Añade más ejercicios según sea necesario
};
</script>


<form id="inicioForm">
    <div>
        <label for="system_prompt">System Prompt:</label><br>
        <textarea id="system_prompt" name="system_prompt" rows="10" cols="100"></textarea>
    </div>
    <div>
        <label for="saludo">Saludo:</label><br>
        <textarea id="saludo" name="saludo" rows="4" cols="100"></textarea>
    </div>
    <div>
        <label for="ejercicios">Juegos:</label><br>
        <select id="ejercicios" name="ejercicios">
            <option value="">Selecciona un ejercicio</option>
            <option value="English_teacher">English teacher</option>
            <option value="guessing_game1">You Guess fictional character</option>
            <option value="guessing_game2">You speak with fictional character</option>
            <option value="guessing_game3">Chatbot guess your fictional character</option>
            <!-- <option value="yes_no_game">yes no game</option> -->
        </select>
    </div>
    <button type="submit">EMPEZAR!</button>

</form>


<div style="margin-bottom: 20px; display: flex; align-items: center;">
    <button id="downloadButton" style="background-color: #4CAF50; /* Color de fondo */
                                       color: white; /* Color del texto */
                                       padding: 15px 32px; /* Padding alrededor del texto */
                                       text-align: center; /* Alinea el texto al centro */
                                       text-decoration: none; /* Elimina la decoración del texto */
                                       display: inline-block; /* Hace que el botón sea un bloque en línea */
                                       font-size: 16px; /* Tamaño del texto */
                                       margin: 4px 2px; /* Margen alrededor del botón */
                                       cursor: pointer; /* Cambia el cursor a un puntero */
                                       border: none; /* Elimina el borde */
                                       border-radius: 8px; /* Redondea las esquinas del botón */
    ">Descargar Conversación</button>
    <div id="audioPlayerAllContainer"></div>
</div>


<script>
    document.getElementById('downloadButton').addEventListener('click', function() {
        obtenerYReproducirAll(); // Llama a la función en vez de redirigir
    });
</script>


<script>
historico = ""
const userID = Math.floor(Math.random() * (999999));
console.log("userID:" + userID);


document.getElementById('ejercicios').addEventListener('change', function() {
    var selectedKey = this.value; // La clave seleccionada del objeto
    if (selectedKey) {
        // Actualiza los textareas con los valores correspondientes
        document.getElementById('system_prompt').value = ejerciciosStrings[selectedKey].systemPrompt;
        document.getElementById('saludo').value = ejerciciosStrings[selectedKey].saludo;
    }
});


document.getElementById('inicioForm').addEventListener('submit', function(e) {
    // Prevenir el comportamiento predeterminado del formulario
    e.preventDefault();
    fetch('http://localhost:5500/alive')
      .then(response => response.json())
      .then(data => console.log('Alive:', data))
      .catch(error => console.error('Error fetching alive status:', error));
    startInterval()
    // Obtener los valores de los campos del formulario
    const system_prompt = document.getElementById('system_prompt').value;
    const saludo = document.getElementById('saludo').value;

    document.getElementById('responseText').innerText = ""
    //obtenerYReproducirAudio(saludo)
    //updateResponseText(saludo + "\n")

    // Crear el cuerpo de la solicitud
    const data = { system_prompt, saludo };

    // Realizar la llamada al servicio Flask
    fetch('http://localhost:5500/inicio', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json',
        },
        body: JSON.stringify(data),
    })
    .then(response => response.json())
    .then(data => {
        let saludo = data.message;
        historico = data.historico
        obtenerYReproducirSaludo(saludo)
        updateResponseText(saludo + "\n")

    })
    .catch((error) => {
        console.error('Error:', error);
        alert('VUELVE A DAR AL PLAY!');
    });
    alert('ESPERE UNOS SEGUNDOS HASTA QUE EMPIECE LA CONVERSACIÓN');
});
</script>

<!-- <button id="btnAccesoMic">Permitir acceso al micrófono</button> -->


<div id="audioPlayerContainer"></div>
<button id="recordButton" style="width: 100%; height: 50px;">Pulsa para grabar/detener</button>


<!-- Estilos para textarea y botón de envío -->
<div id="textButtonContainer" style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4"></textarea>
    <button id="sendTextButton">Enviar Texto</button>
</div>



<div id="responseText" style="height: 250px; margin-top: 20px; border: 1px solid #ddd; padding: 10px; overflow-y:auto;"></div>
<script>
let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        console.log('mediaRecorder Detenido!!');
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};


async function enviarAudioAlServidor(blob) {
        let formData = new FormData();
        formData.append('file', blob, 'grabacion.ogg');
        formData.append('historico', JSON.stringify(historico)); // Se asegura de enviar como cadena JSON
        formData.append('userID', userID);

        try {
            const response = await fetch('http://localhost:5500/transcribe', {
                method: 'POST',
                body: formData, // Solo se envía formData
            });
            const data = await response.json();
            //return data; // Devuelve los datos procesados
       
            updateResponseText("\nyo: " + data.entrada + "\n\n" + "\n***********************************\n" + "respuesta: ");

            // Reinicia el buffer de audio de manera más eficiente
            audioBuffer = Array(currentAudioIndex).fill({});
            currentAudioIndex = 0;

            historico += data.prompt
            await obtenerPartes();
            } catch (error) {
                console.error('Error al enviar el audio:', error);
                alert("VUELVE A DAR AL PLAY! (después puedes continuar la conversación)");
            }
}

async function obtenerPartes(indice = 0) {
    try {
        const response = await fetch(`http://localhost:5500/get_next_part?index=${indice}&userID=${userID}`);
        const partData = await response.json();
        if (partData.output !== "") {
            let trozo = partData.output;
            updateResponseText(trozo);
            historico += trozo

            await obtenerYReproducirAudio(trozo, indice);
            await obtenerPartes(indice + 1); // Llamada recursiva con el índice incrementado
        }
        else{
            historico += "<|eot_id|>"
        }
    } catch (error) {
        console.error('Error al obtener la siguiente parte:', error);
    }
}

// Asumiendo que obtenerYReproducirAudio ya fue optimizado como se mostró anteriormente




// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTextoAlServidor(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};



let audioBuffer = [];
// inicializa audioBuffer con 100 elementos vacíos
for (let i = 0; i < 100; i++) {
    audioBuffer.push({});
}
let milisegundosInicio = Date.now();

let currentAudioIndex = 0; // Para controlar el orden de reproducción


async function enviarTextoAlServidor(texto) {
    try {
        const response = await fetch('http://localhost:5500/texto', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({ texto: texto, historico: historico, userID: userID })
        });
        const data = await response.json();
        updateResponseText("\nyo: " + data.entrada + "\n\n" + "\n***********************************\n" + "respuesta: ");

        historico += data.prompt

        // Reinicia el buffer de audio de manera más eficiente
        audioBuffer = Array(currentAudioIndex).fill({});
        currentAudioIndex = 0;

        await obtenerPartes();
    } catch (error) {
        console.error('Error al enviar el texto:', error);
        alert("VUELVE A DAR AL PLAY! (después puedes continuar la conversación)")
    }
}


async function obtenerYReproducirAudio(texto, index) {
    try {
        const response = await fetch('http://localhost:5500/audio', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({ texto: texto, pausa: 'true'})
        });
        const data = await response.json();
        if (data.audio_base64) {
            console.log('Enviando audio al buffer de reproducción, tamaño:', data.audio_base64.length, 'índice:', index);
            addAudioClipToBuffer(data.audio_base64, index);
        }
    } catch (error) {
        console.error('Error al obtener el audio:', error);
    }
}



function updateResponseText(text) {
    // document.getElementById('responseText').innerText += text;
    textarea = document.getElementById('responseText')
    textarea.innerText += text;
    textarea.scrollTop = textarea.scrollHeight;
}



function obtenerYReproducirAll() {
    fetch('http://localhost:5500/all_conversation', {
        method: 'GET',
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            createAudioAllPlayer(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}


function createAudioAllPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerAllContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

function obtenerYReproducirSaludo(texto) {
    console.log('Obteniendo audio para:', texto, 'microsegundos:', Date.now() - milisegundosInicio);
    fetch('http://localhost:5500/audio', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto, pausa: 'true'})
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            console.log('Obteniendo audio, tamaño:', data.audio_base64.length, 'SALUDO', 'microsegundos:', Date.now()-milisegundosInicio);
            playAudio(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}





function addAudioClipToBuffer(base64Audio, index) {
    audioBuffer[index] = { audio: base64Audio};
    let audioPlayer = document.getElementById('audioPlayerContainer').querySelector('audio');
    console.log('Despues de meter en Buffer, VIENDO si ejecuto play con indice:', index, 'pausa:',audioPlayer.paused, 'currentAudioIndex', currentAudioIndex,'microsegundos:', Date.now()-milisegundosInicio);
    if (audioPlayer.paused && currentAudioIndex == index) {
        playNextAudioClip();
    }
}

//function esObjetoVacio(obj) {
//  return Object.keys(obj).length === 0;
//}
function esObjetoVacio(obj) {
  try {
    // Intenta obtener las claves del objeto y verifica si su longitud es 0
    return Object.keys(obj).length === 0;
  } catch (error) {
    // Si ocurre un error (por ejemplo, si obj no es un objeto), devuelve true
    //console.error("Se produjo un error porque en esa posición del buffer está vacía: ", error);
    return true;
  }
}



function playNextAudioClip() {
    console.log('Entrando en playNextAudioClip para ver si reproducimos audio con currentAudioIndex:', currentAudioIndex, 'microsegundos:', Date.now()-milisegundosInicio, 'esObjetoVacio:', esObjetoVacio(audioBuffer[currentAudioIndex]));
    if (!esObjetoVacio(audioBuffer[currentAudioIndex])) {
        console.log('Iniciando variables para Reproducion audio concurrentAudioIndex:', currentAudioIndex, 'tamaño:', audioBuffer[currentAudioIndex].audio.length, 'microsegundos:', Date.now()-milisegundosInicio);
        const audioData = audioBuffer[currentAudioIndex];
        audioBuffer[currentAudioIndex] = {}; // Limpia el elemento actual
        currentAudioIndex++;

        let audioContainer = document.getElementById('audioPlayerContainer');
        audioContainer.innerHTML = ''; // Limpia el contenedor

        let audioSrc = `data:audio/wav;base64,${audioData.audio}`;
        let audioPlayer = document.createElement('audio');
        audioPlayer.src = audioSrc;
        audioPlayer.controls = true;
        audioPlayer.autoplay = true;

        console.log('Justo antes de APPENDCHILD Reproduciendo audio currentAudioIndex:', currentAudioIndex, 'microsegundos:', Date.now()-milisegundosInicio);
        audioContainer.appendChild(audioPlayer);

        audioPlayer.onended = playNextAudioClip;
    }
}

function playAudio(audioBase64) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    audioContainer.innerHTML = ''; // Limpia el contenedor

    let audioSrc = `data:audio/wav;base64,${audioBase64}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;

    console.log('Reproduciendo saludo! Justo antes de APPENDCHILD', 'microsegundos:', Date.now()-milisegundosInicio);
    audioContainer.appendChild(audioPlayer);
}

function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

obtenerYReproducirSaludo("Wellcom to Conversational Games!")



</script>

Overwriting Llama3JuegoClienteUserID.html


# CLiente traducción

In [ ]:
%%writefile juegoTraduccion2.html

<head>
  <style>


    body {
        font-family: Arial, sans-serif; /* Mejora la tipografía general */
    }



    #inicioForm {
        max-width: 1000px; /* Limita el ancho del formulario */
        margin: 20px auto; /* Centra el formulario */
        padding: 20px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Añade un sombreado ligero */
    }

    #inicioForm div {
        margin-bottom: 15px; /* Añade más espacio entre los campos */
    }

    #inicioForm label {
        font-weight: bold; /* Hace que las etiquetas sean más notables */
        display: block; /* Asegura que la etiqueta esté encima del input */
        margin-bottom: 5px; /* Espacio entre la etiqueta y el campo */
    }

    #inicioForm select, #inicioForm textarea, #inicioForm button {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
    }

    #inicioForm select {
        cursor: pointer; /* Indica que es un elemento interactivo */
        font-size: 16px; /* Aumenta el tamaño del texto */
    }

    #inicioForm textarea {
        resize: vertical; /* Permite al usuario ajustar la altura verticalmente */
    }

    #inicioForm button {
        background-color: #007bff; /* Color de fondo */
        color: white; /* Color del texto */
        border: none; /* Elimina el borde */
        padding: 10px 15px; /* Añade relleno */
        font-size: 18px; /* Aumenta el tamaño del texto */
        cursor: pointer; /* Indica que es un elemento interactivo */
        border-radius: 5px; /* Bordes redondeados */
    }

    #inicioForm button:hover {
        background-color: #0056b3; /* Oscurece el botón al pasar el mouse */
    }

#audioPlayerContainer {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#audioPlayerContainerSol {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#recordButton {
    background-color: #f44336; /* Color rojo para la grabación */
    color: white;
    border: none;
    padding: 10px 0;
    font-size: 18px;
    border-radius: 5px;
    cursor: pointer;
}

#recordButton:hover {
    background-color: #d32f2f; /* Oscurece el botón al pasar el mouse */
}

/* Estilos para el área de texto y botón de envío */
/* Contenedor del área de texto y el botón */
div#textButtonContainer {
    display: flex; /* Establece el contenedor para usar flexbox */
    justify-content: space-between; /* Espacia los elementos uniformemente */
    align-items: center; /* Alinea los elementos verticalmente en el centro */
}

/* Área de texto */
#textInput {
    flex-grow: 1; /* Permite que el área de texto crezca para ocupar el espacio disponible */
    margin-right: 10px; /* Añade un margen a la derecha para separarlo del botón */
    border: 1px solid #ccc; /* Establece un borde sutil */
    border-radius: 5px; /* Bordes redondeados */
    padding: 8px; /* Añade padding interno */
}

/* Botón */
#sendTextButton {
    padding: 8px 15px; /* Ajusta el padding para dimensionar el botón */
    background-color: #4CAF50; /* Color de fondo */
    color: white; /* Color del texto */
    border: none; /* Elimina el borde */
    border-radius: 5px; /* Bordes redondeados */
    cursor: pointer; /* Cambia el cursor a mano al pasar sobre el botón */
}

#sendTextButton:hover {
    background-color: #388E3C; /* Oscurece el botón al pasar el mouse */
}


#responseText {
    height: 250px;
    margin-top: 20px;
    border: 1px solid #ddd;
    padding: 10px;
    overflow-y: auto; /* Asegura el desplazamiento vertical */
    background-color: #f9f9f9; /* Fondo claro para resaltar el área */
}


#nuevoGrupoSelect, #ejercicioNuevoGrupoSelect {
    width: 100%; /* Aprovecha todo el ancho disponible */
    padding: 8px; /* Añade un relleno para mayor comodidad */
    margin-top: 4px; /* Espacio mínimo superior para separación */
    cursor: pointer; /* Indica que es un elemento interactivo */
    font-size: 16px; /* Aumenta el tamaño del texto */
}

    
        /* Estilo básico para el botón de refrescar */
        .btn-refrescar {
            background-color: #f5f5f5;
            border: none;
            cursor: pointer;
            padding: 10px;
            border-radius: 5px;
            font-size: 16px;
        }
        /* Icono de refrescar utilizando una entidad HTML */
        .btn-refrescar:before {
            content: "\21BB"; /* Símbolo de flecha circular */
            margin-right: 5px;
        }

#campoEspañol, #campoIngles, #campoGrupo {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
}

</style>
</head>

<input type="file" id="fileInput" />
<button onclick="uploadFile()">Subir Fichero de Ejercicios</button>
<button onclick="descargarCSV()">Guardar cambios y Descargar</button>


<script>
function uploadFile() {
  const fileInput = document.getElementById('fileInput');
  if (!fileInput.files.length) {
    alert('Por favor, selecciona un archivo para subir.');
    return;
  }

  const file = fileInput.files[0];
  const reader = new FileReader();

  reader.onload = function(e) {
    // Una vez que el archivo ha sido leído, su contenido se almacena en la variable 'text'.
    const text = e.target.result;
    procesarCSV(text)

    // Aquí puedes hacer lo que necesites con el texto. Por ahora, solo lo mostraremos en una alerta.
    alert("Contenido del archivo cargado:\n" + text.substring(0, 200) + "..."); // Muestra los primeros 200 caracteres
  };

  reader.onerror = function() {
    alert('Hubo un error al leer el archivo');
  };

  // Leer el contenido del archivo como un string.
  reader.readAsText(file);
}


function generarTextoCSV(registros) {
    // Unir cada registro array a un string, separando las columnas con '#', y cada fila con un salto de línea
    return registros.map(fila => fila.join('#')).join('\n');
}

function descargarCSV() {
    const csvText = generarTextoCSV(registros);
    const blob = new Blob([csvText], { type: 'text/csv;charset=utf-8;' });
    const url = URL.createObjectURL(blob);

    // Crear un enlace para la descarga
    const downloadLink = document.createElement("a");
    downloadLink.href = url;
    downloadLink.download = "datos_exportados.csv";

    // Simular un clic en el enlace para iniciar la descarga y luego limpiar
    document.body.appendChild(downloadLink);
    downloadLink.click();
    document.body.removeChild(downloadLink);
    URL.revokeObjectURL(url);
    //guardarCambios();  //para guardar también en el servidor
}

</script>

<script>
let registros = []; // Almacena aquí todos los registros procesados

  // Función para cargar y procesar el archivo CSV
  function cargarCSVDesdeServidor() {
    fetch('http://localhost:5500/get-translations-file')
      .then(response => response.text())
      .then(text => procesarCSV(text))
      .catch(error => console.error('Error al cargar el archivo CSV:', error));
  }

  // Función para procesar el contenido del CSV y almacenar los registros
  function procesarCSV(csvText) {
    const filas = csvText.split('\n');
    registros = filas.map(fila => {
        const columnas = fila.replace(/\r/g, '').split('#');
        return columnas.map(columna => columna.trim()); 
    });
    const tipos = new Set();
    //tipos.add('All'); // Añadir una opción predeterminada
    tipos.add('repaso'); // Añadir una opción siempre existente

    registros.forEach(registro => {
      if (registro.length > 2) {
        registro[2].split('/').forEach(tipo => tipos.add(tipo));
      }
    });
    llenarSelector(Array.from(tipos), 'nuevoGrupoSelect'); // Actualiza para llenar el <select>
    llenarSelector(Array.from(tipos), 'ejercicioNuevoGrupoSelect'); // Actualiza para llenar el <select>
    //añadir al prinpio de la lista de tipos el valor 'All'
    tipos.add('All'); // Añadir una opción predeterminada
    llenarSelector(Array.from(tipos), 'ejercicios');

  }


//  function guardarCambios() {
//    // Supongamos que 'registros' contiene los datos modificados que queremos guardar
//    fetch('http://localhost:5500/save-translations-file', {
//        method: 'POST',
//        headers: {
//            'Content-Type': 'application/json',
//        },
//        body: JSON.stringify(registros)  // 'registros' debe ser el array de tus datos
//    })
//    .then(response => response.json())
//    .then(data => {
//        console.log(data.message); // Mensaje de éxito
//       alert(data.message);
//    })
//    .catch((error) => {
//        console.error('Error:', error); // Manejo de errores
//        alert('Error al guardar los cambios');
//    });
//}


    function llenarSelector(tipos, elementoID) {
        const elemento = document.getElementById(elementoID);
        elemento.innerHTML = ''; // Limpiar opciones existentes

        tipos.forEach(tipo => {
            const opcion = document.createElement('option');
            opcion.value = tipo;
            opcion.textContent = tipo; // Solo para el selector de ejercicios
            elemento.appendChild(opcion);
        });
    }

  

  // Función para manejar el cambio de selección y actualizar la variable ejercicio
  let ejercicio = 'All'; // Variable para almacenar el tipo seleccionado
  function actualizarEjercicio(valorSeleccionado) {
    ejercicio = valorSeleccionado;
    mostrarSiguiente()
    console.log('Ejercicio seleccionado:', ejercicio);
  }


//let indiceActual = -1; // Variable para almacenar el índice del registro actual

// Función para obtener un registro aleatorio que contenga el valor seleccionado como un string completo en la tercera columna
function obtenerRegistroAleatorio(tipoSeleccionado) {
  let registrosFiltrados;

  // Si el tipo seleccionado es "All", no filtra los registros
  if (tipoSeleccionado.trim() === "All") {
    registrosFiltrados = registros;
  } else {
    // Filtra los registros verificando que el tipo seleccionado, sin espacios en blanco,
    // sea exactamente uno de los tipos en la tercera columna después de hacer split por '/'
    registrosFiltrados = registros.filter(registro => {
      if (registro.length > 2) {
        // Divide la tercera columna por '/', elimina espacios en blanco y verifica si el tipo seleccionado está presente
        const tipos = registro[2].split('/').map(tipo => tipo.trim());
        return tipos.includes(tipoSeleccionado.trim());
      }
      return false;
    });
  }

  if (registrosFiltrados.length === 0) {
    return null; // O manejar de otra manera si no hay registros que coincidan
  }
  
  // Selecciona uno de los registros filtrados al azar y lo devuelve
  const indiceAleatorio = Math.floor(Math.random() * registrosFiltrados.length);
//   indiceActual = indiceAleatorio
  return registrosFiltrados[indiceAleatorio];
}


function eliminarRegistro(Pregistro) {
    // Encuentra el índice del registro para eliminar en 'registros' basado solo en los dos primeros campos
    español = Pregistro[0]
    ingles = Pregistro[1]
    let indice = registros.findIndex(registro => 
        registro[0] === español && registro[1] === ingles
    );

    // Verifica si se encontró el índice
    if (indice !== -1) {
        // Elimina el registro de 'registros' usando splice
        registros.splice(indice, 1);
        console.log('Registro eliminado correctamente.');
    } else {
        console.log('Registro no encontrado.');
    }
}





// Función para añadir un grupo a la tercera columna del registro especificado por el índice
function añadirGrupo(registro, grupo) {
  // Verifica que el índice esté dentro del rango de los registros cargados
  if (registro) {
    // Divide la tercera columna por '/', elimina espacios en blanco de los extremos, y verifica si el grupo ya está presente
    let grupos = registro[2].split('/').map(g => g.trim());
    if (!grupos.includes(grupo)) {
      grupos.push(grupo); // Añade el nuevo grupo
      registro[2] = grupos.join('/'); // Reconstruye la tercera columna y actualiza el registro
      return true; // Indica que el grupo fue añadido exitosamente
    } else {
      return false; // Indica que el grupo ya estaba presente
    }
  } else {
    console.error('Índice fuera de rango.');
    return false; // También retorna false si el índice está fuera de rango
  }
}

// Función para eliminar un grupo de la tercera columna del registro especificado por el índice
function eliminaGrupo(registro, grupo) {
  // Verifica que el índice esté dentro del rango de los registros cargados
  if (registro) {
    // Divide la tercera columna por '/', elimina espacios en blanco de los extremos, y verifica si el grupo existe
    let grupos = registro[2].split('/').map(grupo => grupo.trim());
    if (grupos.includes(grupo)) {
      grupos = grupos.filter(g => g !== grupo); // Elimina el grupo
      registro[2] = grupos.join('/'); // Reconstruye la tercera columna y actualiza el registro
      return true; // Indica que el grupo fue eliminado exitosamente
    } else {
      return false; // Indica que el grupo no se encontró
    }
  } else {
    console.error('Índice fuera de rango.');
    return false; // También retorna false si el índice está fuera de rango
  }
}



function arrayToCSVString(arrayDeArrays) {
    // Mapea cada sub-array a un string, uniendo los campos con '#'
    const lineas = arrayDeArrays.map(registro => registro.join('#'));
    // Une todas las líneas con saltos de línea para formar el string completo del CSV
    const csvString = lineas.join('\n');
    return csvString;
}


  // Llama a cargarCSVDesdeServidor al cargar la página
 // window.addEventListener('load', cargarCSVDesdeServidor);



</script>


<form id="inicioForm">
    <div>
        <label for="enunciado">Traduce esto:</label><br>
        <textarea id="enunciado" name="enunciado" rows="1" cols="100"></textarea>
    </div>
    <div>
        <label for="ejercicios">Juegos:</label> <button class="btn-refrescar" onclick="procesarCSV(arrayToCSVString(registros))">Refrescar</button>
<br>
        <select id="ejercicios" onchange="actualizarEjercicio(this.value)">
        <!-- Las opciones se llenarán dinámicamente -->
        </select>
       
    </div>
    <button type="submit">Siguiente!</button>

</form>

<div id="manejoGrupos">
    <!-- Primera fila con botones para añadir a repaso y eliminar del grupo -->
    <div>
        <button id="btnAñadirRepaso">Añadir a repaso</button>
        <button id="btnEliminarRepaso">Eliminar de repaso</button>
        <button id="btnEliminarRegistro">Eliminar registro</button>
        <button id="btnGuardarCambios">Guardar Cambios</button>
    </div>
    
    <p> AÑADIR O ELIMINAR ESTE EJERCICIO DE UN GRUPO: </p>
    <div>
        <select id="nuevoGrupoSelect" onchange="seleccionarGrupo()">
            <!-- Las opciones se llenarán dinámicamente -->
        </select>
        <input type="text" id="nuevoGrupoInput" placeholder="Nuevo grupo o seleccione">
        <button id="btnAñadirNuevoGrupo">Añadir al grupo</button>
        <button id="btnEliminarDelGrupo">Eliminar del grupo</button>

    </div>
</div>

<script>


let registroActual

//document.getElementById('btnEliminarRegistro').addEventListener('click', eliminarRegistro(registroActual));
document.getElementById('btnEliminarRegistro').addEventListener('click', function() {
    eliminarRegistro(registroActual);
});

//document.getElementById('btnGuardarCambios').addEventListener('click', guardarCambios);
//document.getElementById('btnGuardarCambios').addEventListener('click', function() {
//    guardarCambios();
//});



document.getElementById('btnAñadirRepaso').addEventListener('click', function() {
    if (añadirGrupo(registroActual, 'repaso')) {
        alert('Añadido a repaso'); // Si el grupo fue añadido exitosamente
    } else {
        // Si el registro ya estaba en el grupo o hubo algún otro error
        alert('El registro ya está en el grupo repaso o no se pudo añadir');
    }
});

document.getElementById('btnEliminarRepaso').addEventListener('click', function() {
    // Utiliza la función eliminaGrupo para intentar eliminar el registro actual del grupo "repaso"
    if (eliminaGrupo(registroActual, 'repaso')) {
        alert('Eliminado del grupo repaso'); // Si el grupo fue eliminado exitosamente
    } else {
        // Si el registro no estaba en el grupo o hubo algún otro error
        alert('No se ha eliminado del grupo porque, en realidad, no se encontraba en el grupo repaso');
    }
});


document.getElementById('btnEliminarDelGrupo').addEventListener('click', function() {
    // Asume que tienes un selector definido en alguna parte de tu HTML para elegir el grupo
    let grupoSeleccionado = document.getElementById('nuevoGrupoInput').value; // Asegúrate de que el ID coincida
    if (eliminaGrupo(registroActual, grupoSeleccionado)) {
        alert('Eliminado del grupo ' + grupoSeleccionado); // Opcional: feedback al usuario
    } else {
        // Muestra un mensaje de error si el registro no se encontraba en el grupo
        alert(`No se ha eliminado del grupo porque, en realidad, no se encontraba en el grupo ${grupoSeleccionado}`);
    }
});

document.getElementById('btnAñadirNuevoGrupo').addEventListener('click', function() {
    let nuevoGrupo = document.getElementById('nuevoGrupoInput').value.trim();
    if (nuevoGrupo) {
        // La función añadirGrupo ahora devuelve true si el grupo fue añadido con éxito, y false si ya existía
        if (añadirGrupo(registroActual, nuevoGrupo)) {
            alert('Añadido al grupo ' + nuevoGrupo); // Si el grupo fue añadido exitosamente
        } else {
            // Si el registro ya estaba en el grupo
            alert(`El registro ya se encuentra en el grupo ${nuevoGrupo}, por lo que no se ha añadido de nuevo.`);
        }
    } else {
        alert('Por favor, introduce un nombre de grupo válido.'); // Si el input está vacío o es inválido
    }
});


function mostrarSiguiente() {
    // obtener un registro aleatorio
    const enunciadoTextArea = document.getElementById('enunciado');
    const ejercicio = document.getElementById('ejercicios').value;
    registroActual = obtenerRegistroAleatorio(ejercicio);
    enunciadoTextArea.value = registroActual[0];
 
    document.getElementById('responseText').innerText = registroActual[0];
}

function seleccionarGrupo() {
    const select = document.getElementById('nuevoGrupoSelect');
    const input = document.getElementById('nuevoGrupoInput');
    if (select.value) {
        input.value = select.value; // Transfiere el valor seleccionado al input
    }
}

function seleccionarNuevoEjercicioGrupo() {
    const select = document.getElementById('ejercicioNuevoGrupoSelect');
    const input = document.getElementById('campoGrupo');
    if (select.value) {
        input.value = select.value; // Transfiere el valor seleccionado al input
    }
}


document.getElementById('inicioForm').addEventListener('submit', function(e) {
    // Prevenir el comportamiento predeterminado del formulario
    e.preventDefault();

    // obtener un registro aleatorio
    mostrarSiguiente()
  
});

</script>


<div id="audioPlayerContainer"></div> <div id="audioPlayerContainerSol"></div>
<button id="recordButton" style="width: 100%; height: 50px;">Pulsa para grabar/detener</button>


<!-- Estilos para textarea y botón de envío -->
<div id="textButtonContainer" style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4"></textarea>
    <button id="sendTextButton">Enviar Texto</button>
</div>



<div id="responseText" style="height: 250px; margin-top: 20px; border: 1px solid #ddd; padding: 10px; overflow-y:auto;"></div>


<p> AÑADIR UN NUEVO EJERCICIO: </p>
<div id="nuevoEjercicio">
    <div>
        <label for="campoEspañol">Español:</label>
        <input type="text" id="campoEspañol" name="campoEspañol">
    </div>
    <div>
        <label for="campoIngles">Inglés:</label>
        <input type="text" id="campoIngles" name="campoIngles">
    </div>
    <div>
        <label for="campoGrupo">Grupo:</label>
         <select id="ejercicioNuevoGrupoSelect" onchange="seleccionarNuevoEjercicioGrupo()">
            <!-- Las opciones se llenarán dinámicamente -->
        </select>        
        <input type="text" id="campoGrupo" name="campoGrupo">
        
    </div>
    <button id="btnCrearNuevoEjercicio">Crear nuevo ejercicio</button>
</div>


<script>

document.getElementById('btnCrearNuevoEjercicio').addEventListener('click', function() {
    // Obtiene los valores de los campos de texto
    const español = document.getElementById('campoEspañol').value.trim();
    const ingles = document.getElementById('campoIngles').value.trim();
    
    const grupo = document.getElementById('campoGrupo').value.trim();


    // Valida que los campos no estén vacíos (ajusta según necesidades)
    if (español && ingles && grupo) {
        // Inserta los nuevos valores al final del array 'registros'
        registros.push([español, ingles, grupo]);
        console.log('Nuevo ejercicio creado:', español, ingles, grupo);
        console.log('Registros actualizados:', registros);

        // Opcional: Limpia los campos después de la inserción
        document.getElementById('campoEspañol').value = '';
        document.getElementById('campoIngles').value = '';
        document.getElementById('campoGrupo').value = '';
    } else {
        // Opcional: Mensaje de error si algún campo está vacío
        console.error('Todos los campos son requeridos para crear un nuevo ejercicio.');
        alert('Por favor, rellena todos los campos.');
    }
});


let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        console.log('mediaRecorder Detenido!!');
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};



function enviarTexto(texto) {
    updateResponseText(" ---> " + texto + " (sol: " + registroActual[1] + ")" + "\n");
    obtenerYReproducir(texto, false);
    obtenerYReproducir(registroActual[1]);
}




function enviarAudioAlServidor(blob) {
    let formData = new FormData();
    formData.append('file', blob, 'grabacion.ogg');

    fetch('http://localhost:5500/only_transcribe', {
        method: 'POST',
        body: formData,
    })
    .then(response => response.json())
    .then(data => {
        updateResponseText(" ---> " + data.entrada + " (sol: " + registroActual[1] + ")" + "\n");
     

    })
    .catch(error => {
        console.error('Error al enviar el audio:', error);
    });

    obtenerYReproducir(registroActual[1]);

    createAudioPlayerFromBlob(blob)
}



// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTexto(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};




function updateResponseText(text) {
    // document.getElementById('responseText').innerText += text;
    textarea = document.getElementById('responseText')
    textarea.innerText += text;
    textarea.scrollTop = textarea.scrollHeight;
}



function obtenerYReproducir(texto, sol=true) {
    //relleno para que no tenga problemas el generador de voz.
    //if (texto.length < 20) { texto += " . (This phrase was too short)";}
    //else if (texto.length < 25) { texto += " . .";}
    console.log('Obteniendo audio para:', texto);
    fetch('http://localhost:5500/audio', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto, pausa: 'false' })
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {            
            playAudio(data.audio_base64, sol);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}




function playAudio(audioBase64, sol=true) {
    let audioContainer=""
    if (sol) {
        audioContainer = document.getElementById('audioPlayerContainerSol');
    } else {
        audioContainer = document.getElementById('audioPlayerContainer');
    }
    audioContainer.innerHTML = ''; // Limpia el contenedor

    let audioSrc = `data:audio/wav;base64,${audioBase64}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = false;

    console.log('creando audio para solución.');
    audioContainer.appendChild(audioPlayer);
}

function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

function createAudioPlayerFromBlob(blob) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    // Crea una URL de objeto para el Blob
    let audioSrc = URL.createObjectURL(blob);
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = false;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

cargarCSVDesdeServidor()

</script>

Writing juegoTraduccion2.html


# Cliente remoto

Cliente remoto con API separada para proxy inverso con nginx

In [ ]:
%%writefile juegoClienteProxyInverso.html
<head>
  <style>


    body {
        font-family: Arial, sans-serif; /* Mejora la tipografía general */
    }


    #system_prompt {
        height: 150px;
    }


    #inicioForm {
        max-width: 1000px; /* Limita el ancho del formulario */
        margin: 20px auto; /* Centra el formulario */
        padding: 20px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Añade un sombreado ligero */
    }

    #inicioForm div {
        margin-bottom: 15px; /* Añade más espacio entre los campos */
    }

    #inicioForm label {
        font-weight: bold; /* Hace que las etiquetas sean más notables */
        display: block; /* Asegura que la etiqueta esté encima del input */
        margin-bottom: 5px; /* Espacio entre la etiqueta y el campo */
    }

    #inicioForm select, #inicioForm textarea, #inicioForm button {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
    }

    #inicioForm select {
        cursor: pointer; /* Indica que es un elemento interactivo */
        font-size: 16px; /* Aumenta el tamaño del texto */
    }

    #inicioForm textarea {
        resize: vertical; /* Permite al usuario ajustar la altura verticalmente */
    }

    #inicioForm button {
        background-color: #007bff; /* Color de fondo */
        color: white; /* Color del texto */
        border: none; /* Elimina el borde */
        padding: 10px 15px; /* Añade relleno */
        font-size: 18px; /* Aumenta el tamaño del texto */
        cursor: pointer; /* Indica que es un elemento interactivo */
        border-radius: 5px; /* Bordes redondeados */
    }

    #inicioForm button:hover {
        background-color: #0056b3; /* Oscurece el botón al pasar el mouse */
    }

#audioPlayerContainer {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#recordButton {
    background-color: #f44336; /* Color rojo para la grabación */
    color: white;
    border: none;
    padding: 10px 0;
    font-size: 18px;
    border-radius: 5px;
    cursor: pointer;
}

#recordButton:hover {
    background-color: #d32f2f; /* Oscurece el botón al pasar el mouse */
}

/* Estilos para el área de texto y botón de envío */
/* Contenedor del área de texto y el botón */
div#textButtonContainer {
    display: flex; /* Establece el contenedor para usar flexbox */
    justify-content: space-between; /* Espacia los elementos uniformemente */
    align-items: center; /* Alinea los elementos verticalmente en el centro */
}

/* Área de texto */
#textInput {
    flex-grow: 1; /* Permite que el área de texto crezca para ocupar el espacio disponible */
    margin-right: 10px; /* Añade un margen a la derecha para separarlo del botón */
    border: 1px solid #ccc; /* Establece un borde sutil */
    border-radius: 5px; /* Bordes redondeados */
    padding: 8px; /* Añade padding interno */
}

/* Botón */
#sendTextButton {
    padding: 8px 15px; /* Ajusta el padding para dimensionar el botón */
    background-color: #4CAF50; /* Color de fondo */
    color: white; /* Color del texto */
    border: none; /* Elimina el borde */
    border-radius: 5px; /* Bordes redondeados */
    cursor: pointer; /* Cambia el cursor a mano al pasar sobre el botón */
}

#sendTextButton:hover {
    background-color: #388E3C; /* Oscurece el botón al pasar el mouse */
}


#responseText {
    height: 250px;
    margin-top: 20px;
    border: 1px solid #ddd;
    padding: 10px;
    overflow-y: auto; /* Asegura el desplazamiento vertical */
    background-color: #f9f9f9; /* Fondo claro para resaltar el área */
}



</style>
</head>

<script>

window.intervalId = window.intervalId || null; // Asegura que intervalId sea global y única

var URL = 'https://javiergimenez.es/api'; // Inicializa la variable URL


function updateURL() {
    URL = document.getElementById('url').value; // Actualiza la variable URL con el valor del campo de entrada
}

function startInterval() {
    if (window.intervalId === null) {
        window.intervalId = setInterval(() => {
            fetch(URL + '/alive')
                .then(response => response.json())
                .then(data => console.log('Alive:', data))
                .catch(error => console.error('Error fetching alive status:', error));
        }, 30000);
        console.log('Intervalo iniciado.');
    } else {
        console.log('Ya existe un intervalo en ejecución.');
    }
}

function stopInterval() {
    if (window.intervalId !== null) {
        clearInterval(window.intervalId);
        console.log('Intervalo detenido.');
        window.intervalId = null;
    } else {
        console.log('No hay un intervalo para detener.');
    }
}


// startInterval()

// document.getElementById('btnAccesoMic').addEventListener('click', async () => {
//     try {
//         const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
//         // Procesa el stream aquí
//         console.log('Acceso al micrófono concedido');
//     } catch (error) {
//         console.error('Acceso al micrófono denegado:', error);
//     }
// });

document.addEventListener('DOMContentLoaded', function() {
    window.scrollTo(0, 0); // Asegura que la página comience en la parte superior
    document.getElementById('ejercicios').focus(); // Luego establece el foco en el selector
});



// Objeto para mapear los ejercicios a sus strings correspondientes
const ejerciciosStrings = {
    "guessing_game1": {
        systemPrompt: `This is a conversational game in which you have to think in this famous character: #personaje, and the user have to guess this character. You should aswer the user questions about the character. Be concise but give some clues. NEVER say the name of the character until the end. If the user guesses the character then you will say: "Congratulations, you got it right, the character was #personaje". If user give up you will say: "what a shame!, the character was #personaje
Game success example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he #personaje?
assistant: Congratulations, you were right, it was #personaje.
Game give up example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he Benito Perez?
assistant: No, it has nothing to do with.
user: I give up. Who is it?
assistant: what a shame!, the character was #personaje
`,
        saludo: "I'm thinking of a famous fictional character, guess which one it is."
    },
    "guessing_game2": {
        systemPrompt: "You are #personaje, the fictional character. You have to take on the personality of that character and engage in conversations about your events and experiences.",
        saludo: "I'm #personaje, the fictional character. Ask me anything you want to know about me."
    },
    "guessing_game3": {
        systemPrompt: "This is a conversational game in which you have to guess a famous character. You should make questions to the user in order to guess the character that the user have choosen.",
        saludo: "Do you want to play? I will guess your choosen character by asking about it."
    },
    "yes_no_game": {
        systemPrompt: `IMPORTANT: You only can answer "yes" or "no" (nothing more!).
This is a conversational game between you and a the user. The game consists that only at the beggininig you tell the user only a piece of the context and the user having to guess the "key point" of the context from "yes or no questions". The User could ask anything about the story (context) to guess the "key point" of the context but Assistant could only answer "Yes" or "not". If the user asks a question that does not lend itself or cannot be answered with a "yes" or "no" such as "What is the man's name?" then Assistant will respond: "Only "yes or no" questions. When the user guesses the key point of the story you will say: "Congratulations, you have guessed the key to the story.
Game context: A man named Edgar is the lighthouse keeper of Águilas for 30 years. He always turns on the lighthouse at dusk and shortly after sleeps in a small room next to its large lamp. On Edgar's birthday, at dusk after lighting the lighthouse, he decided to go to dinner with an old friend to celebrate his birthday. During dinner he drank more than necessary and they both got drunk. Afther the dinner Edgar accompanied his friend to her house and then went to his house, the lighthouse, where he sleeps every night. Upon entering the lighthouse and going up to his room, due to his drunkenness and the fact that he was very sleepy, he decided to turn off the light (which was actually the light from the main lamp of the lighthouse) to sleep off the drunkenness and did it without realizing or knowing it was dangerous. During the early hours of the morning, a cruise ship full of passengers crashed into the cliff that the lighthouse protected because, when it was turned off, neither the lookout, nor the captain, nor the rest of the crew nor the passengers could see that they were heading against the cliff. An hour later Edgar wakes up, it hasn't dawned yet but you can hear sirens and a lot of noise from the rescuers who are trying to rescue the shipwrecked. Edgar turns on the lamp to illuminate the scene where hundreds of dead shipwrecked people continually crash against the cliff due to the waves. Faced with this heartbreaking reality and his feeling of guilt, Edgar decides to commit suicide by jumping from the top of the lighthouse. The key point of the story that the user must find out is: "Edgar commits suicide because he was the LIGHTHOUSE keeper." or similar but always emphasizing that he was the lighthouse keeper.
IMPORTANT: You only can answer "yes" or "no" or "Only yes or no questions"
Examples of correct answers:
user: What is Edgar's job?
Assistant: Only yes or no questions.
user: Is Edgar a man?
Assistant: yes.`,
        saludo: `This is I can show about the hidden story: Edgar was dazed and comes to his room, turns off the light and lies down on his bed. He wakes up a few hours later, turns on the light, looks out the window and is so horrified that he ends up jumping out of the window and committing suicide.
Guess what happened.
IMPORTANT: From now on I can only answer you "yes" or "no" and nothing more.`
    },
    "English_teacher": {
        systemPrompt: `You are Sofie, an english teacher 29 years old. You will have simple dialogues with the student in your charge. you will only have concise conversations with short sentences so that the student is encouraged to converse.
Also you can offer translations exercises but only from spainish to enlgish. if you do translations exercises about a topic you must always say the sentence to translate in spanish: How do you say 'me gustaría viajar a París'? and then de user will be able response in english. You will not ask for doing translates from english to spanish, only from spanish to english.`,
        saludo: "Good Morning. What is your name?"
    }
    // Añade más ejercicios según sea necesario
};
</script>


<form id="inicioForm">
    <div>
        <label for="system_prompt">System Prompt:</label><br>
        <textarea id="system_prompt" name="system_prompt" rows="10" cols="100"></textarea>
    </div>
    <div>
        <label for="saludo">Saludo:</label><br>
        <textarea id="saludo" name="saludo" rows="4" cols="100"></textarea>
    </div>
    <div>
        <label for="ejercicios">Juegos:</label><br>
        <select id="ejercicios" name="ejercicios">
            <option value="">Selecciona un ejercicio</option>
            <option value="English_teacher">English teacher</option>
            <option value="guessing_game1">You Guess fictional character</option>
            <option value="guessing_game2">You speak with fictional character</option>
            <option value="guessing_game3">Chatbot guess your fictional character</option>
            <!-- <option value="yes_no_game">yes no game</option> -->
        </select>
    </div>
    <div>
        <label for="url">URL:</label>
        <input type="text" id="url" name="url" value="https://javiergimenez.es/api" onchange="updateURL()">
    </div>
    <button type="submit">EMPEZAR!</button>

</form>


<div style="margin-bottom: 20px; display: flex; align-items: center;">
    <button id="downloadButton" style="background-color: #4CAF50; /* Color de fondo */
                                       color: white; /* Color del texto */
                                       padding: 15px 32px; /* Padding alrededor del texto */
                                       text-align: center; /* Alinea el texto al centro */
                                       text-decoration: none; /* Elimina la decoración del texto */
                                       display: inline-block; /* Hace que el botón sea un bloque en línea */
                                       font-size: 16px; /* Tamaño del texto */
                                       margin: 4px 2px; /* Margen alrededor del botón */
                                       cursor: pointer; /* Cambia el cursor a un puntero */
                                       border: none; /* Elimina el borde */
                                       border-radius: 8px; /* Redondea las esquinas del botón */
    ">Descargar Conversación</button>
    <div id="audioPlayerAllContainer"></div>
</div>


<script>
    document.getElementById('downloadButton').addEventListener('click', function() {
        obtenerYReproducirAll(); // Llama a la función en vez de redirigir
    });
</script>


<script>
historico = ""
const userID = Math.floor(Math.random() * (999999));
console.log("userID:" + userID);


document.getElementById('ejercicios').addEventListener('change', function() {
    var selectedKey = this.value; // La clave seleccionada del objeto
    if (selectedKey) {
        // Actualiza los textareas con los valores correspondientes
        document.getElementById('system_prompt').value = ejerciciosStrings[selectedKey].systemPrompt;
        document.getElementById('saludo').value = ejerciciosStrings[selectedKey].saludo;
    }
});


document.getElementById('inicioForm').addEventListener('submit', function(e) {
    // Prevenir el comportamiento predeterminado del formulario
    e.preventDefault();
    fetch(URL + '/alive')
      .then(response => response.json())
      .then(data => console.log('Alive:', data))
      .catch(error => console.error('Error fetching alive status:', error));
    startInterval()
    // Obtener los valores de los campos del formulario
    const system_prompt = document.getElementById('system_prompt').value;
    const saludo = document.getElementById('saludo').value;

    document.getElementById('responseText').innerText = ""
    //obtenerYReproducirAudio(saludo)
    //updateResponseText(saludo + "\n")

    // Crear el cuerpo de la solicitud
    const data = { system_prompt, saludo };

    // Realizar la llamada al servicio Flask
    fetch(URL + '/inicio', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json',
        },
        body: JSON.stringify(data),
    })
    .then(response => response.json())
    .then(data => {
        let saludo = data.message;
        historico = data.historico
        obtenerYReproducirSaludo(saludo)
        updateResponseText(saludo + "\n")

    })
    .catch((error) => {
        console.error('Error:', error);
        alert('VUELVE A DAR AL PLAY!');
    });
    alert('ESPERE UNOS SEGUNDOS HASTA QUE EMPIECE LA CONVERSACIÓN');
});
</script>

<!-- <button id="btnAccesoMic">Permitir acceso al micrófono</button> -->


<div id="audioPlayerContainer"></div>
<button id="recordButton" style="width: 100%; height: 50px;">Pulsa para grabar/detener</button>


<!-- Estilos para textarea y botón de envío -->
<div id="textButtonContainer" style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4"></textarea>
    <button id="sendTextButton">Enviar Texto</button>
</div>



<div id="responseText" style="height: 250px; margin-top: 20px; border: 1px solid #ddd; padding: 10px; overflow-y:auto;"></div>
<script>
let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        console.log('mediaRecorder Detenido!!');
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};


async function enviarAudioAlServidor(blob) {
        let formData = new FormData();
        formData.append('file', blob, 'grabacion.ogg');
        formData.append('historico', JSON.stringify(historico)); // Se asegura de enviar como cadena JSON
        formData.append('userID', userID);

        try {
            const response = await fetch(URL + '/transcribe', {
                method: 'POST',
                body: formData, // Solo se envía formData
            });
            const data = await response.json();
            //return data; // Devuelve los datos procesados
       
            updateResponseText("\nyo: " + data.entrada + "\n\n" + "\n***********************************\n" + "respuesta: ");

            // Reinicia el buffer de audio de manera más eficiente
            audioBuffer = Array(currentAudioIndex).fill({});
            currentAudioIndex = 0;

            historico += data.prompt
            await obtenerPartes();
            } catch (error) {
                console.error('Error al enviar el audio:', error);
                alert("VUELVE A DAR AL PLAY! (después puedes continuar la conversación)");
            }
}

async function obtenerPartes(indice = 0) {
    try {
        const response = await fetch(`${URL}/get_next_part?index=${indice}&userID=${userID}`);
        const partData = await response.json();
        if (partData.output !== "") {
            let trozo = partData.output;
            updateResponseText(trozo);
            historico += trozo

            await obtenerYReproducirAudio(trozo, indice);
            await obtenerPartes(indice + 1); // Llamada recursiva con el índice incrementado
        }
        else{
            historico += "<|eot_id|>"
        }
    } catch (error) {
        console.error('Error al obtener la siguiente parte:', error);
    }
}

// Asumiendo que obtenerYReproducirAudio ya fue optimizado como se mostró anteriormente




// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTextoAlServidor(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};



let audioBuffer = [];
// inicializa audioBuffer con 100 elementos vacíos
for (let i = 0; i < 100; i++) {
    audioBuffer.push({});
}
let milisegundosInicio = Date.now();

let currentAudioIndex = 0; // Para controlar el orden de reproducción


async function enviarTextoAlServidor(texto) {
    try {
        const response = await fetch(URL + '/texto', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({ texto: texto, historico: historico, userID: userID })
        });
        const data = await response.json();
        updateResponseText("\nyo: " + data.entrada + "\n\n" + "\n***********************************\n" + "respuesta: ");

        historico += data.prompt

        // Reinicia el buffer de audio de manera más eficiente
        audioBuffer = Array(currentAudioIndex).fill({});
        currentAudioIndex = 0;

        await obtenerPartes();
    } catch (error) {
        console.error('Error al enviar el texto:', error);
        alert("VUELVE A DAR AL PLAY! (después puedes continuar la conversación)")
    }
}


async function obtenerYReproducirAudio(texto, index) {
    try {
        const response = await fetch(URL + '/audio', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({ texto: texto, pausa: 'true'})
        });
        const data = await response.json();
        if (data.audio_base64) {
            console.log('Enviando audio al buffer de reproducción, tamaño:', data.audio_base64.length, 'índice:', index);
            addAudioClipToBuffer(data.audio_base64, index);
        }
    } catch (error) {
        console.error('Error al obtener el audio:', error);
    }
}



function updateResponseText(text) {
    // document.getElementById('responseText').innerText += text;
    textarea = document.getElementById('responseText')
    textarea.innerText += text;
    textarea.scrollTop = textarea.scrollHeight;
}



function obtenerYReproducirAll() {
    fetch(URL + '/all_conversation', {
        method: 'GET',
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            createAudioAllPlayer(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}


function createAudioAllPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerAllContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

function obtenerYReproducirSaludo(texto) {
    //console.log('Obteniendo audio para:', texto, 'microsegundos:', Date.now() - milisegundosInicio);
    fetch(URL + '/audio', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto, pausa: 'true'})
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
         //   console.log('Obteniendo audio, tamaño:', data.audio_base64.length, 'SALUDO', 'microsegundos:', Date.now()-milisegundosInicio);
            playAudio(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}





function addAudioClipToBuffer(base64Audio, index) {
    audioBuffer[index] = { audio: base64Audio};
    let audioPlayer = document.getElementById('audioPlayerContainer').querySelector('audio');
    //console.log('Despues de meter en Buffer, VIENDO si ejecuto play con indice:', index, 'pausa:',audioPlayer.paused, 'currentAudioIndex', currentAudioIndex,'microsegundos:', Date.now()-milisegundosInicio);
    if (audioPlayer.paused && currentAudioIndex == index) {
        playNextAudioClip();
    }
}

//function esObjetoVacio(obj) {
//  return Object.keys(obj).length === 0;
//}
function esObjetoVacio(obj) {
  try {
    // Intenta obtener las claves del objeto y verifica si su longitud es 0
    return Object.keys(obj).length === 0;
  } catch (error) {
    // Si ocurre un error (por ejemplo, si obj no es un objeto), devuelve true
    //console.error("Se produjo un error porque en esa posición del buffer está vacía: ", error);
    return true;
  }
}



function playNextAudioClip() {
    //console.log('Entrando en playNextAudioClip para ver si reproducimos audio con currentAudioIndex:', currentAudioIndex, 'microsegundos:', Date.now()-milisegundosInicio, 'esObjetoVacio:', esObjetoVacio(audioBuffer[currentAudioIndex]));
    if (!esObjetoVacio(audioBuffer[currentAudioIndex])) {
       // console.log('Iniciando variables para Reproducion audio concurrentAudioIndex:', currentAudioIndex, 'tamaño:', audioBuffer[currentAudioIndex].audio.length, 'microsegundos:', Date.now()-milisegundosInicio);
        const audioData = audioBuffer[currentAudioIndex];
        audioBuffer[currentAudioIndex] = {}; // Limpia el elemento actual
        currentAudioIndex++;

        let audioContainer = document.getElementById('audioPlayerContainer');
        audioContainer.innerHTML = ''; // Limpia el contenedor

        let audioSrc = `data:audio/wav;base64,${audioData.audio}`;
        let audioPlayer = document.createElement('audio');
        audioPlayer.src = audioSrc;
        audioPlayer.controls = true;
        audioPlayer.autoplay = true;

      //  console.log('Justo antes de APPENDCHILD Reproduciendo audio currentAudioIndex:', currentAudioIndex, 'microsegundos:', Date.now()-milisegundosInicio);
        audioContainer.appendChild(audioPlayer);

        audioPlayer.onended = playNextAudioClip;
    }
}

function playAudio(audioBase64) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    audioContainer.innerHTML = ''; // Limpia el contenedor

    let audioSrc = `data:audio/wav;base64,${audioBase64}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;

    //console.log('Reproduciendo saludo! Justo antes de APPENDCHILD', 'microsegundos:', Date.now()-milisegundosInicio);
    audioContainer.appendChild(audioPlayer);
}

function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

obtenerYReproducirSaludo("Wellcom to Conversational Games!")



</script>


Servidor con todo incluido para ejecutarse en segundo plano (producción)

In [1]:
%%writefile server_conversacional.py

#########################################
####    MODEL IN LLAMA_CPP
#########################################
# %%writefile cargar_llama_cpp.py
# from ctransformers import AutoModelForCausalLM, AutoTokenizer

import os
# import accelerate

import subprocess

# Definir las variables de entorno y las rutas
BASE_FOLDER = "./"
REPO = "QuantFactory"
TYPE_MODEL = "Meta-Llama-3-8B-Instruct-GGUF"
MODEL = "Meta-Llama-3-8B-Instruct.Q8_0.gguf"
MODEL_PATH = os.path.join(BASE_FOLDER, MODEL)
CONTEXT_LENGTH = 8192

# Crear el directorio base si no existe
if not os.path.exists(BASE_FOLDER):
    os.mkdir(BASE_FOLDER)
    print("Creado directorio base:", BASE_FOLDER)

# Descargar el modelo si no existe
if not os.path.exists(MODEL_PATH):
    url = f"https://huggingface.co/{REPO}/{TYPE_MODEL}/resolve/main/{MODEL}?download=true"
    cmd = f'curl -L "{url}" -o "{MODEL_PATH}"'
    print("Descargando:", MODEL)
    try:
        result = subprocess.run(cmd, shell=True, check=True)
        print("Descarga completa.")
    except subprocess.CalledProcessError as e:
        print("Error al descargar el archivo:", e)

# %cd {BASE_FOLDER}

from llama_cpp import Llama

model=None
# eos_token_id=None

# Función para cargar el modelo si aún no está cargado
def load_model():
    global model
    # global tokenizer
    if model is None:  # Verifica si model está vacío o no parece ser un modelo válido
        print("Cargando modelo...")
        # model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map='auto', load_in_8bit=True, trust_remote_code=True)
        # model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map='auto', torch_dtype="auto", trust_remote_code=True)
        enable_gpu = True  # offload LLM layers to the GPU (must fit in memory)

        model = Llama(
            model_path=MODEL_PATH,
            n_gpu_layers=-1 if enable_gpu else 0,
            n_ctx=CONTEXT_LENGTH,
            # verbose=False,
        )
        model.verbose=False

        print("Modelo cargado.")
    else:
        print("Modelo ya estaba cargado.")


load_model()

import whisper
modelWhisper = whisper.load_model('medium')


from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface

# Carga el modelo y la configuración
models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/fastspeech2-en-ljspeech",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)

# Asegúrate de que models es una lista
if not isinstance(models, list):
    models = [models]

modelT2S = models[0]
modelT2S = modelT2S.to('cuda:0')

TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

# Aquí, asumimos que task.build_generator puede manejar correctamente el objeto cfg y model
generator = task.build_generator(models, cfg)
#MODELO LO CARGAMOS A PARTE PORQUE TARDA EN CARGARSE

#########################################
####    CHATBOT
#########################################
# %%writefile modelo_llama3.py
# from cargar_llama_cpp import model
LOGGING = False
from threading import Lock

# global model
def encontrar_coincidencia(texto, cadena_busqueda="<|eot_id|>"):
    """
    Esta función busca la primera aparición de una cadena de búsqueda en un texto dado y devuelve el substring
    desde el principio del texto hasta el final de esta coincidencia (incluida).
    
    Parámetros:
    texto (str): El texto en el que se buscará la cadena.
    cadena_busqueda (str): La cadena de caracteres que se buscará en el texto.
    
    Retorna:
    str: El substring desde el inicio hasta el final de la primera coincidencia de la cadena buscada,
    incluyendo la coincidencia. Si no se encuentra ninguna coincidencia, devuelve una cadena vacía.
    """
    # Buscar la posición de la primera coincidencia de la cadena en el texto
    indice = texto.find(cadena_busqueda)
    
    if indice != -1:
        # Devolver el substring desde el inicio hasta el final de la coincidencia
        return texto[:indice + len(cadena_busqueda)]
    else:
        # Devolver una cadena vacía si no hay coincidencia
        return ""


# VENTANA DESLIZANTE
def ajustar_contexto(texto, max_longitud=15000, secuencia="<|start_header_id|>", system_end="<|eot_id|>"):
    system_prompt = encontrar_coincidencia(texto, system_end)
    # Comprobar si la longitud del texto es mayor que el máximo permitido
    if len(texto) > max_longitud:
        indice_secuencia = 0

        while True:
            # Buscar la secuencia de ajuste
            indice_secuencia = texto.find(secuencia, indice_secuencia + 1)

            # Si la secuencia no se encuentra o el texto restante es menor que la longitud máxima
            if indice_secuencia == -1 or len(system_prompt) + len(texto) - indice_secuencia <= max_longitud:
                break

        # Si encontramos una secuencia válida
        if indice_secuencia != -1:
            return system_prompt + texto[indice_secuencia:]

        else:
            # Si no se encuentra ninguna secuencia adecuada, tomar los últimos max_longitud caracteres
            return system_prompt + texto[-max_longitud + len(system_prompt):]
    else:
        return system_prompt + texto



generate_lock = Lock()

def pre_warm_chat(historico, max_additional_tokens=100, stop=["</s>","user:"], short_answer=True, streaming=False, printing=False):
 
    # if short_answer:
    #     # añade como stop el salto de linea
    #     stop.append("\n")

    outputs = ""

    with generate_lock:
        response=model(prompt=historico, max_tokens=max_additional_tokens, temperature=0, top_p=1,
                    top_k=0, repeat_penalty=1,
                    stream=True)


        respuesta = ""

        for chunk in response:
            trozo = chunk['choices'][0]['text']
            # trozo.replace("\n", "")
            # trozo.replace("<|EOT|>", "")
            # for caracter in trozo:
            #     cadena_con_codigos += f"{caracter}({ord(caracter)}) "
            respuesta += trozo
            print(trozo, end="", flush=True)
            # linea += trozo

            # if len(linea)>35:
                # print(linea, end="", flush=True)  # Impresión en consola

                # linea = ""


        outputs = historico + respuesta
        return historico, outputs



import threading

class EstadoGeneracion:
    def __init__(self):
        # lista de 100 partes del texto
        self.parts = [""]*100
        self.top = -1
        self.generando = False
        # self.lock = threading.Lock()


estado_generacion = {}
estado_generacion['anonimo'] = EstadoGeneracion()

# generate_lock = Lock()

def generate_in_file_parts(userID, historico, ai, user, input_text, max_additional_tokens=2000, short_answer=False, streaming=True, printing=True):
    global estado_generacion

    # printf(HISTORICO_LOG, f"generando para USER:{userID}\nhistorico:{historico}\ninput_text:{input_text}")
    # global generate_lock
    if userID not in estado_generacion:
        estado_generacion[userID] = EstadoGeneracion()

    estado_generacion[userID].generando = True
    with generate_lock:
        estado_generacion[userID].top = -1
        print(f"Empezamos a generar ponemos el TOP a {estado_generacion[userID].top} para USER:{userID}!!:", input_text)
        indiceParte = 0
        # estado_generacion[userID].generando = True
        print(f"generando={estado_generacion[userID].generando}; Generando respuesta para USER:{userID}:", input_text)
        # estado_generacion.parts = []  # lista de partes del texto
        parte_actual = ""  # añade la primera parte

        if short_answer:
            # añade como stop el salto de linea
            stop.append("\n")


        prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

        final_prompt = historico + "\n" + prompt


        model_inputs = final_prompt

        outputs = ""
        print(f"{ai}:", end="")


        # outputs = ""
        colchon = (CONTEXT_LENGTH - max_additional_tokens)*3
        print("Longitud:",len(final_prompt), "Colchon:", colchon)
        if len(final_prompt)> colchon: #cuenta la vieja cada token son 3 caracteres (como poco)
            print("Ajustando contexto!!!")
            final_prompt = ajustar_contexto(final_prompt, max_longitud=colchon)
            print(final_prompt)
            #contexto ajustado imprimir los primeros 500 caracteres
            # print(final_prompt[:500])
            #imprimir los 500 últimos caracteres
            # print(final_prompt[-500:])

        response=model(prompt=final_prompt, max_tokens=max_additional_tokens, temperature=0, top_p=1,
                      top_k=0, repeat_penalty=1,
                      stream=True)


        respuesta = ""

        for chunk in response:
            trozo = chunk['choices'][0]['text']
            # trozo.replace("\n", "")
            # trozo.replace("<|EOT|>", "")
            # for caracter in trozo:
            #     cadena_con_codigos += f"{caracter}({ord(caracter)}) "
            respuesta += trozo
            print(trozo, end="", flush=True)

            outputs += trozo
            parte_actual += trozo
            if trozo in ",;:.?!" and len(parte_actual)>28:
                estado_generacion[userID].parts[indiceParte] = parte_actual
                estado_generacion[userID].top = indiceParte
                if LOGGING:
                    print(f"trozo generado para USER: {userID}:", parte_actual)
                    print("se ha generado para entrada de indiceParte (ahora TOP tb vale esto):", indiceParte)
                indiceParte += 1                
                # print("se incrementa indiceParte (pero TOP aun no) a:", indiceParte)
                parte_actual = ""


        if len(parte_actual)>1:
            estado_generacion[userID].parts[indiceParte] = parte_actual
            estado_generacion[userID].top = indiceParte

        all_text = model_inputs + outputs + "<|eot_id|>"
        estado_generacion[userID].generando = False
        if LOGGING:
            print(f"generando={estado_generacion[userID].generando}; Respuesta Terminada. El total generado para {user}:", outputs)

        return all_text, outputs


#########################################
####    SERVER
#########################################
from flask import Flask, request, jsonify, send_file, send_from_directory
from flask_cors import CORS
from threading import Lock
import threading
import os
import torch
from pydub import AudioSegment
import pandas as pd
import random

import time


# import whisper
# modelWhisper = whisper.load_model('medium')



# parts = []  # lista de partes del texto
# generando = False
# global model



# from modelo_llama3 import generate_in_file_parts, pre_warm_chat
if LOGGING:
    print("El modelo es:", model)

ai = "assistant"
user = "user"

contexto = """

"""

system_prompt = """
You are a kind and helpful assistan bot. You are here to help the user to find the best answer to his question.
"""

saludo = "Hello, I am ready to receive and process your input."

idioma = "en"

import sys

# Verifica si el comando tenía flag -s o --short
if "-s" in sys.argv or "--short" in sys.argv:
    short_answer = True
else:
    short_answer = False

# Si encuentra el flag -es cambia el idioma a español
if "-es" in sys.argv:
    idioma = "es"

# Filtra los argumentos para eliminar los flags
args = [arg for arg in sys.argv[1:] if arg not in ["-s", "--short", "-es"]]

# Asigna los valores a system_prompt y saludo basándose en los argumentos restantes
if len(args) > 0:
    system_prompt = args[0]
if len(args) > 1:
    saludo = args[1]


historico = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{saludo}<|eot_id|>"


# load model
# load_model()

if LOGGING:
    print(f"{ai}:", saludo)

# Crea un bloqueo para proteger el código contra la concurrencia a la hora de transcribir
transcribe_lock = Lock()


# generate_lock = Lock()

app = Flask(__name__)
# app.config['MAX_CONTENT_LENGTH'] = 30 * 1024 * 1024  # 30 MB

#import logging
#logging.basicConfig(level=logging.DEBUG)

CORS(app)

output = ""


@app.route('/alive')
def alive():
    return jsonify(True)



def eliminar_archivos_temp(nombre_inicio='temp_sync'):
    # Obtener una lista de todos los archivos en el directorio actual
    archivos = os.listdir('.')
    
    # Filtrar archivos que comienzan con 'temp_sync'
    archivos_temp = [archivo for archivo in archivos if archivo.startswith(nombre_inicio)]
    
    # Iterar sobre la lista de archivos y eliminarlos
    for archivo in archivos_temp:
        try:
            os.remove(archivo)
            print(f"Archivo eliminado: {archivo}")
        except Exception as e:
            print(f"No se pudo eliminar el archivo {archivo}. Razón: {e}")

@app.route('/inicio', methods=['POST'])
def print_strings():
    # global modelo
    # global historico
    eliminar_archivos_temp("received_audio")
    historico = ""

    # Lee el archivo CSV y selecciona un personaje de ficción al azar
    def elegir_personaje_aleatorio():
        df = pd.read_csv('Personajes_ficcion.csv')
        return random.choice(df.iloc[:, 0].tolist())

    # Obtiene los datos del cuerpo de la solicitud
    data = request.json

    # Extrae los strings del objeto JSON
    system_prompt = data.get('system_prompt')
    saludo = data.get('saludo')

    # Preprocesamiento para reemplazar "#personaje" con un personaje aleatorio
    if "#personaje" in system_prompt:
        personaje_aleatorio = elegir_personaje_aleatorio()
        system_prompt = system_prompt.replace("#personaje", personaje_aleatorio)

    # Preprocesamiento para reemplazar "#personaje" con un personaje aleatorio en el saludo
    if "#personaje" in saludo:
        saludo = saludo.replace("#personaje", personaje_aleatorio)

    # Imprime los strings en el log del servidor
    if LOGGING:
        print("INICIALIZANDO CONVERSACIÓN")
    conversation_file = 'conversacion.mp3'
    # si existe el archivo de conversación, lo elimina
    if os.path.exists(conversation_file):
        os.remove(conversation_file)

    if LOGGING:
        print(f"system: {system_prompt}, saludo: {saludo}")

    # if modelo == "mistral":
    #     historico = f"system\n{system_prompt}\nassistant\n{saludo}\n"
    # elif modelo == "zypher":
    #     historico = f"{system_prompt}</s>\n\n{saludo}</s>\n"
    historico = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{saludo}<|eot_id|>"

    pre_warm_chat(historico)

    # Retorna una respuesta para indicar que se recibieron y procesaron los datos
    return jsonify({"message": saludo, "historico": historico}), 200


@app.route('/get-translations-file', methods=['GET'])
def get_translations():
    return send_from_directory(directory='.', path='translations.csv', as_attachment=True)

import csv
import shutil
@app.route('/save-translations-file', methods=['POST'])
def save_translations():
    data = request.json  # Asume que el cliente envía los datos como JSON
    if not data:
        return jsonify({'error': 'No data provided'}), 400

    try:
        # Hace una copia de seguridad del archivo translations.csv antes de modificarlo
        shutil.copy('translations.csv', 'translations.csv.bak')

        # Abre el archivo translations.csv para escribir y actualiza con los datos recibidos
        with open('translations.csv', mode='w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile, delimiter='#')
            for row in data:
                writer.writerow(row)

        return jsonify({'message': 'File successfully saved'}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/all_conversation', methods=['GET'])
def all_conversation():
    # Asegúrate de que el path al archivo sea correcto para tu estructura de proyecto
    filepath = 'conversacion.mp3'
    if not os.path.exists(filepath):
        return jsonify(error="Archivo de conversación no encontrado"), 404

    # Leer el archivo y convertirlo a base64
    with open(filepath, 'rb') as audio_file:
        audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

    return jsonify(audio_base64=audio_base64)



import subprocess

def convert_ogg_to_mp3(source_ogg_path, target_mp3_path):
    """
    Utiliza ffmpeg para convertir un archivo .ogg a .mp3.
    """
    command = ['ffmpeg', '-y' ,'-i', source_ogg_path, target_mp3_path]
    process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Si el comando falla, imprime la salida de error
    if process.returncode != 0:
        print(f"Error al convertir {source_ogg_path} a {target_mp3_path}")
        print("Salida de error de ffmpeg:")
        print(process.stderr.decode())




def convert_wav_to_mp3(source_wav_path, target_mp3_path):
    command = ['ffmpeg', '-i', source_wav_path, target_mp3_path]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


def add_audio_to_conversation_async(source_path, convert_to_mp3=False):
    def task():
        if convert_to_mp3:
            # Convertir de WAV a MP3 si es necesario
            temp_mp3_path = source_path.replace('.wav', '.mp3')
            convert_wav_to_mp3(source_path, temp_mp3_path)
            final_path = temp_mp3_path
        else:
            final_path = source_path

        # Añadir al archivo de conversación
        sound = AudioSegment.from_file(final_path)
        conversation_file = 'conversacion.mp3'
        if os.path.exists(conversation_file):
            conversation_audio = AudioSegment.from_mp3(conversation_file)
            combined_audio = conversation_audio + sound
        else:
            combined_audio = sound
        combined_audio.export(conversation_file, format='mp3')

        # Limpiar archivos temporales
        os.remove(source_path)
        if convert_to_mp3:
            os.remove(temp_mp3_path)

    thread = threading.Thread(target=task)
    thread.start()



def generate_chat_background(userID, entrada, phistorico, ai, user, short_answer):
    # global output  # Indicar que se utilizará la variable global 'output'
    if LOGGING:
        print("OJOOOOOOOO!!!!!!  generate_chat_background USERID:", userID, "entrada:", entrada)
    start_generation_time = time.time()
    # Ejecutar la generación de chat en un hilo aparte
    historico_local, output_local = generate_in_file_parts(userID, phistorico, ai, user, input_text=entrada, max_additional_tokens=2048, short_answer=short_answer, streaming=True, printing=False)
    end_generation_time = time.time()
    generation_duration = end_generation_time - start_generation_time
    if LOGGING:
        print(f"Generación completada en {generation_duration} segundos")

    # Actualizar las variables globales con los resultados obtenidos
    # global historico
    # historico = historico_local
    # output = output_local

@app.route('/transcribe', methods=['POST'])
def transcribe_audio():
    if LOGGING:
        print("Transcribiendo audio...")
    global user
    global ai


    if 'userID' not in request.form:
        return jsonify(error="No se proporcionó userID"), 400
    userID = int(request.form['userID'])

    if 'historico' not in request.form:
        return jsonify(error="No se proporcionó historico"), 400
    historico = request.form['historico']  # Asumiendo que se envía como JSON y necesitará ser parseado en Python

    # Extraer el archivo
    if 'file' not in request.files:
        return jsonify(error="No se proporcionó file"), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify(error="No selected file"), 400

    timestamp = int(time.time() * 1000)
    ogg_filepath = f"received_audio_{timestamp}.ogg"
    file.save(ogg_filepath)

    start_transcribe_time = time.time()
    if LOGGING:
        print("antes del transcribe lock, userID:", userID)
    with transcribe_lock:
        if LOGGING:
            print("después del transcribe lock, userID:", userID)
        transcripcion = modelWhisper.transcribe(ogg_filepath, fp16=False, language=idioma)
        transcripcion = transcripcion["text"]
    end_transcribe_time = time.time()
    transcribe_duration = end_transcribe_time - start_transcribe_time
    if LOGGING:
        print(f"Transcripción completada en {transcribe_duration} segundos")
        print("transcripción:", transcripcion)

    # Iniciar la generación de chat en un hilo aparte
    thread = threading.Thread(target=generate_chat_background, args=(userID, transcripcion, historico, ai, user, short_answer))
    thread.start()

    # Inicia el proceso de adición del audio .ogg en segundo plano, considerando su conversión a .mp3
    add_audio_to_conversation_async(ogg_filepath)

    # La respuesta ya no incluirá 'output' porque se generará en segundo plano
    prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{transcripcion}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    historico += prompt    
    return jsonify(entrada=transcripcion, prompt=prompt, entrada_traducida="")
    # return jsonify(entrada=transcripcion, historico=historico, entrada_traducida="")





@app.route('/only_transcribe', methods=['POST'])
def only_transcribe_audio():
    if LOGGING:
        print("Transcribiendo audio...")
    # global historico
    # global user
    # global ai

    if 'file' not in request.files:
        print("No file part")
        return jsonify(error="No file part"), 400

    file = request.files['file']
    if file.filename == '':
        print("No selected file")
        return jsonify(error="No selected file"), 400

    if LOGGING:
        print("creando fichero ogg audio antes de transcripción...")
    timestamp = int(time.time() * 1000)
    ogg_filepath = f"received_audio_{timestamp}.ogg"
    file.save(ogg_filepath)

    start_transcribe_time = time.time()
    if LOGGING:
        print("antes del transcribe lock")
    with transcribe_lock:
        if LOGGING:
            print("después del transcribe lock")
        transcripcion = modelWhisper.transcribe(ogg_filepath, fp16=False, language=idioma)
        transcripcion = transcripcion["text"]
    end_transcribe_time = time.time()
    transcribe_duration = end_transcribe_time - start_transcribe_time
    if LOGGING:
        print(f"Transcripción completada en {transcribe_duration} segundos")
        print("transcripción:", transcripcion)

    # Iniciar la generación de chat en un hilo aparte
    #thread = threading.Thread(target=generate_chat_background, args=(transcripcion, historico, ai, user, short_answer))
    #thread.start()

    # Inicia el proceso de adición del audio .ogg en segundo plano, considerando su conversión a .mp3
    add_audio_to_conversation_async(ogg_filepath)

    # La respuesta ya no incluirá 'output' porque se generará en segundo plano
    return jsonify(entrada=transcripcion, entrada_traducida="")



@app.route('/get_next_part', methods=['GET'])
def get_next_part():
    global estado_generacion
    if LOGGING:
        print("LAS CLAVES de usuario y sus TOP:")
        for clave in estado_generacion.keys():
            print(clave," TOP:", estado_generacion[clave].top)

    userID = request.args.get('userID', default=0, type=int)
    # Obtener el índice de la solicitud. Si no se proporciona, por defecto es None
    index = request.args.get('index', default=0, type=int)

    if LOGGING:
        print(f"userID:{userID} partes: {estado_generacion[userID].parts}, generando: {estado_generacion[userID].generando}, index: {index}, estado_generacion[userID].top: {estado_generacion[userID].top}")

    while True:
        # if estado_generacion.parts:
            # Verificar si el índice es válido
        if index is not None and index >= 0 and index <= estado_generacion[userID].top:

            part = estado_generacion[userID].parts[index]
            estado_generacion[userID].parts[index] = ""  # Elimina el elemento en el índice dado
            if LOGGING:
                print("con index:", index, "estado_generacion[userID].top:", estado_generacion[userID].top)    
                print(f"Enviando parte: {part}")
            return jsonify(output=part)
            # else:
            #     print("Índice inválido o fuera de límites")
            #     return jsonify(error="Índice inválido o fuera de límites"), 400
        elif estado_generacion[userID].generando:
            if LOGGING:
                print("Esperando a que se generen más partes...")
            time.sleep(0.1)  # Espera 0.1 segundos antes de volver a verificar
        else:
            if LOGGING:
                print("No hay más partes para enviar", "index:", index, "estado_generacion[userID].top:", estado_generacion[userID].top)
            return jsonify(output="") # Si 'generando' es False y 'parts' está vacía, devuelve una cadena vacía



@app.route('/texto', methods=['POST'])
def process_text():
    # global historico
    global user
    global ai

    # Recibe el texto directamente del cuerpo de la solicitud
    data = request.json
    if not data or 'texto' not in data:
        return jsonify(error="No se proporcionó texto"), 400

    texto = data['texto']

    if 'historico' not in data:
        return jsonify(error="No se proporcionó historico"), 400

    historico = data['historico']

    if 'userID' not in data:
        return jsonify(error="No se proporcionó userID"), 400

    userID = data['userID']

    if LOGGING:
        print("HISTORICO!!!:", historico)

    # Utiliza la variable 'idioma' declarada globalmente
    global idioma


    # Generación de respuesta basada en el texto proporcionado
    thread = threading.Thread(target=generate_chat_background, args=(userID, texto, historico, ai, user, short_answer))
    thread.start()


    # si el idioma es español, traduce la respuesta al español

    prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{texto}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    historico += prompt
    return jsonify(entrada=texto, historico=historico, entrada_traducida="")



import torch  # Importamos PyTorch para poder usar la función `to()`

# Función para mover recursivamente todos los tensores en una estructura anidada a un dispositivo
def move_to_device(obj, device):
    if isinstance(obj, torch.Tensor):
        return obj.to(device)
    elif isinstance(obj, dict):
        return {key: move_to_device(value, device) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [move_to_device(item, device) for item in obj]
    else:
        return obj


# PREPARAMOS INSTANCIAS FAIRSEQ

if idioma == "en":
    print("idioma ingles") # ESTO HAY QUE CAMBIARLO 
    # from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    # from fairseq.models.text_to_speech.hub_interface import TTSHubInterface

    # # Carga el modelo y la configuración
    # models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    #     "facebook/fastspeech2-en-ljspeech",
    #     arg_overrides={"vocoder": "hifigan", "fp16": False}
    # )

    # # Asegúrate de que models es una lista
    # if not isinstance(models, list):
    #     models = [models]

    # modelT2S = models[0]
    # modelT2S = modelT2S.to('cuda:0')

    # TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # # Aquí, asumimos que task.build_generator puede manejar correctamente el objeto cfg y model
    # generator = task.build_generator(models, cfg)



elif idioma == "es":
    # El modelo no entiende de números aritméticos. Esta función los convierte a palabras.
    import re
    from num2words import num2words

    def number_to_words(num_str):
        try:
            num = int(num_str)
            return num2words(num, lang='es')
        except ValueError:
            return "Por favor, introduzca un número válido."

    def process_numbers_in_line(line):
        def replace_with_words(match):
            return number_to_words(match.group())

        return re.sub(r'\b\d+\b', replace_with_words, line)

    # Ejemplo de uso
    line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
    new_line = process_numbers_in_line(line)
    print(new_line)
    # Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."


    # Diccionario con las traducciones

    def process_abrev(line):
        translations = {
        'Dr': 'doctor',
        'Sr': 'señor',
        'Sra': 'señora',
        # Añade más traducciones aquí
    }
        for abbr, full in translations.items():
            line = line.replace(f'{abbr}.', full)
            line = line.replace(f'{abbr} ', f'{full} ')
        return line

    def otras_traducciones(line):

        translations = {
        '-': ',',
        '—': ',',
        '%': ' por ciento '
        # Añade más traducciones aquí
        }

        for old, new in translations.items():
            line = line.replace(old, new)
        return line


    def preprocesado_al_modelo(line):
        line_with_numbers = process_numbers_in_line(line)
        line_with_both = process_abrev(line_with_numbers)
        line_with_all = otras_traducciones(line_with_both)
        return line_with_all

    import os
    os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
    print(os.environ['TF_ENABLE_ONEDNN_OPTS'])


    from pydub import AudioSegment, silence

    def quitar_silencios(input_filepath, output_filepath, min_silence_len=1500, new_silence_len=750, silence_thresh=-60):
        """
        Elimina silencios largos de un archivo de audio.

        Parámetros:
        - input_filepath: ruta al archivo de audio de entrada (MP3).
        - output_filepath: ruta al archivo de audio de salida (MP3).
        - min_silence_len: duración mínima del silencio a eliminar (en milisegundos).
        - new_silence_len: duración de los nuevos segmentos de silencio (en milisegundos).
        - silence_thresh: umbral de silencio (en dB).
        """

        # Cargar el archivo de audio
        audio_segment = AudioSegment.from_wav(input_filepath)

        # Encuentra los segmentos de audio separados por silencios
        segments = silence.split_on_silence(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

        # Crear un nuevo segmento de audio con silencios ajustados
        new_audio_segment = AudioSegment.empty()
        silence_chunk = AudioSegment.silent(duration=new_silence_len)  # Chunk de silencio de la nueva duración

        # Añade cada segmento de audio al nuevo audio, intercalando con los nuevos segmentos de silencio
        for segment in segments:
            new_audio_segment += segment + silence_chunk

        # Removemos el último chunk de silencio añadido
        new_audio_segment = new_audio_segment[:-new_silence_len]

        # Guarda el nuevo archivo de audio
        new_audio_segment.export(output_filepath, format="wav")


    # Cargamos el modelo generador fairseq
    from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
    import IPython.display as ipd





    # Cargamos el modelo y la configuración desde el modelo preentrenado de Hugging Face
    models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
        "facebook/tts_transformer-es-css10",
        arg_overrides={"vocoder": "hifigan", "fp16": False}
    )
    modelT2S = models[0]

    # Movemos el modelo al dispositivo GPU
    modelT2S = modelT2S.to('cuda:0')

    # Actualizamos la configuración con los datos del task
    TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # Creamos el generador
    generator = task.build_generator([modelT2S], cfg)


    import torchaudio

    import re

    def dividir_texto_con_minimo_palabras(texto, min_palabras=8):
        partes = re.split(r'([.,;:?!])', texto)
        partes_filtradas = [parte.strip() for parte in partes if parte.strip()]
        partes_combinadas = []
        parte_actual = ''

        for parte in partes_filtradas:
            if parte in '.,;:?!':
                parte_actual += parte
                if len(parte_actual.split()) >= min_palabras:
                    partes_combinadas.append(parte_actual)
                    parte_actual = ''
                else:
                    parte_actual += ' '
            else:
                parte_actual += parte + ' '

        if len(parte_actual.strip()) > 10:
            partes_combinadas.append(parte_actual.strip())

        return partes_combinadas

    def combinar_audios(audios_temporales):
        audio_combinado = "audio_combinado.wav"
        # Cargar el primer archivo de audio para inicializar la concatenación
        wav_total, rate = torchaudio.load(audios_temporales[0])

        # Iterar sobre los archivos restantes y concatenarlos
        for archivo in audios_temporales[1:]:
            wav, _ = torchaudio.load(archivo)
            wav_total = torch.cat((wav_total, wav), 1)

        # Guardar el audio combinado en un archivo final
        torchaudio.save(audio_combinado, wav_total, rate)

        return audio_combinado

    def voz_sintetica_spanish(text):
        text = preprocesado_al_modelo(text)

        lista_dividida = dividir_texto_con_minimo_palabras(text)

        audios_temporales = []

        for parte in lista_dividida:
            # Preparamos los datos de entrada para el modelo
            sample = TTSHubInterface.get_model_input(task, parte)

            # Movemos los datos al dispositivo GPU
            sample = move_to_device(sample, 'cuda:0')

            # Realizamos la predicción
            wav, rate = TTSHubInterface.get_prediction(task, modelT2S, generator, sample)

            if len(wav.shape) == 1:
                wav = wav.unsqueeze(0)

            # temp_file_name = "Temporal.wav"
            temp_file_name = f"temporal_{parte[:10]}.wav"
            torchaudio.save(temp_file_name, wav.to('cpu'), rate)
            audios_temporales.append(temp_file_name)

            combinado = combinar_audios(audios_temporales)
            sin_silencios = "sin_silencios.wav"
            # quitamos silencios
            quitar_silencios(combinado, sin_silencios, min_silence_len=1500, new_silence_len=750, silence_thresh=-60)


        with open(sin_silencios, "rb") as audio_file:
            audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

        return audio_base64



import base64
@app.route('/audio', methods=['POST'])
def generate_audio():
    texto = request.json.get('texto')
    pausa = request.json.get('pausa')
    if LOGGING:
        print('PAUSA!!!!!!!!:', pausa)
        print('TEXTO!!!!!!!!:', texto)
    if not texto:
        return jsonify(error="No se proporcionó texto"), 400

    if idioma == "en":
        audio_base64 = voz_sintetica_english(texto, pausa)
        return jsonify(audio_base64=audio_base64)
    elif idioma == "es":
        audio_base64 = voz_sintetica_spanish(texto)
        return jsonify(audio_base64=audio_base64)

import base64
import io
import soundfile as sf


def add_comma_after_punctuation(text: str) -> str:
    # Lista de caracteres después de los cuales se debe agregar una coma
    punctuation_marks = ['.', '!', '?', '(', ')', ':', '\n']

    # Recorre cada marca de puntuación y añade una coma después de cada ocurrencia
    for mark in punctuation_marks:
        text = text.replace(mark, mark + ',...,')

    return text

# Ejemplo de uso de la función
#example_text = "Hello! How are you? I hope you're doing well. Let's meet tomorrow."
#modified_text = add_comma_after_punctuation(example_text)
#print(modified_text)

import io
import base64
import soundfile as sf
import os
import threading
from pydub import AudioSegment
import subprocess


def add_silence_to_audio(audio_path, duration_ms=3000):
    """Añade un segmento de silencio al final de un archivo de audio."""
    # Carga el audio
    sound = AudioSegment.from_wav(audio_path)
    # Genera el silencio
    silence = AudioSegment.silent(duration=duration_ms)
    # Concatena el audio con el silencio
    combined = sound + silence
    # Guarda el nuevo archivo
    combined.export(audio_path, format='wav')


def voz_sintetica_english(texto, pausa="true"):
    if pausa == "true":
        texto = add_comma_after_punctuation(texto)
    # Preparamos los datos de entrada para el modelo
    sample = TTSHubInterface.get_model_input(task, texto)

    # Movemos los datos al dispositivo GPU
    sample = move_to_device(sample, 'cuda:0')

    # Realizamos la predicción
    wav, rate = TTSHubInterface.get_prediction(task, modelT2S, generator, sample)


        # Convertimos el tensor wav a un buffer de audio en memoria y luego a un archivo temporal
    temp_wav_path = f"temp_synth_audio_{int(time.time() * 1000)}.wav"
    with io.BytesIO() as audio_buffer:
        sf.write(audio_buffer, wav.cpu().numpy(), rate, format='WAV')
        audio_buffer.seek(0)  # Regresamos al inicio del buffer para leerlo
        # Guardar en un archivo temporal
        with open(temp_wav_path, 'wb') as f:
            f.write(audio_buffer.read())

    # Añadir el audio al archivo de conversación en segundo plano
    add_audio_to_conversation_async(temp_wav_path, convert_to_mp3=True)  # Asegúrate de implementar la conversión dentro de esta función si es necesario

    # if len(texto) <= 20:
    #     # Añade silencio al final del archivo de audio
    #     add_silence_to_audio(temp_wav_path, 1000)  # Añade 1 segundo de silencio para que no de problemas en audios cortos

    if len(texto) <= 30:
        print("Añadiendo 1 segundos de silencio")
        add_silence_to_audio(temp_wav_path, 1000)

    # Convertir el buffer a base64 para retornar
    with open(temp_wav_path, 'rb') as f:
        audio_base64 = base64.b64encode(f.read()).decode('utf-8')


    return audio_base64


def print_routes(app):
    print("Endpoints disponibles:")
    for rule in app.url_map.iter_rules():
        methods = ','.join(sorted(rule.methods))
        print(f"{rule.endpoint}: {rule.rule} [{methods}]")


if __name__ == '__main__':
    print_routes(app)
    app.run(host='0.0.0.0', port=5500, threaded=True)
    # app.run(host='0.0.0.0', port=5500, threaded=True, ssl_context=('cert.pem', 'key.pem'))




Overwriting server_conversacional.py


Cliente de juego de traducciones

In [2]:

<head>
    <meta charset="UTF-8">
    <title>Translate Game</title>
  <style>


    body {
        font-family: Arial, sans-serif; /* Mejora la tipografía general */
    }



    #inicioForm {
        max-width: 1000px; /* Limita el ancho del formulario */
        margin: 20px auto; /* Centra el formulario */
        padding: 20px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Añade un sombreado ligero */
    }

    #inicioForm div {
        margin-bottom: 15px; /* Añade más espacio entre los campos */
    }

    #inicioForm label {
        font-weight: bold; /* Hace que las etiquetas sean más notables */
        display: block; /* Asegura que la etiqueta esté encima del input */
        margin-bottom: 5px; /* Espacio entre la etiqueta y el campo */
    }

    #inicioForm select, #inicioForm textarea, #inicioForm button {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
    }

    #inicioForm select {
        cursor: pointer; /* Indica que es un elemento interactivo */
        font-size: 16px; /* Aumenta el tamaño del texto */
    }

    #inicioForm textarea {
        resize: vertical; /* Permite al usuario ajustar la altura verticalmente */
    }

    #inicioForm button {
        background-color: #007bff; /* Color de fondo */
        color: white; /* Color del texto */
        border: none; /* Elimina el borde */
        padding: 10px 15px; /* Añade relleno */
        font-size: 18px; /* Aumenta el tamaño del texto */
        cursor: pointer; /* Indica que es un elemento interactivo */
        border-radius: 5px; /* Bordes redondeados */
    }

    #inicioForm button:hover {
        background-color: #0056b3; /* Oscurece el botón al pasar el mouse */
    }

#audioPlayerContainer {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#audioPlayerContainerSol {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#recordButton {
    background-color: #f44336; /* Color rojo para la grabación */
    color: white;
    border: none;
    padding: 10px 0;
    font-size: 18px;
    border-radius: 5px;
    cursor: pointer;
}

#recordButton:hover {
    background-color: #d32f2f; /* Oscurece el botón al pasar el mouse */
}

/* Estilos para el área de texto y botón de envío */
/* Contenedor del área de texto y el botón */
div#textButtonContainer {
    display: flex; /* Establece el contenedor para usar flexbox */
    justify-content: space-between; /* Espacia los elementos uniformemente */
    align-items: center; /* Alinea los elementos verticalmente en el centro */
}

/* Área de texto */
#textInput {
    flex-grow: 1; /* Permite que el área de texto crezca para ocupar el espacio disponible */
    margin-right: 10px; /* Añade un margen a la derecha para separarlo del botón */
    border: 1px solid #ccc; /* Establece un borde sutil */
    border-radius: 5px; /* Bordes redondeados */
    padding: 8px; /* Añade padding interno */
}

/* Botón */
#sendTextButton {
    padding: 8px 15px; /* Ajusta el padding para dimensionar el botón */
    background-color: #4CAF50; /* Color de fondo */
    color: white; /* Color del texto */
    border: none; /* Elimina el borde */
    border-radius: 5px; /* Bordes redondeados */
    cursor: pointer; /* Cambia el cursor a mano al pasar sobre el botón */
}

#sendTextButton:hover {
    background-color: #388E3C; /* Oscurece el botón al pasar el mouse */
}


#responseText {
    height: 250px;
    margin-top: 20px;
    border: 1px solid #ddd;
    padding: 10px;
    overflow-y: auto; /* Asegura el desplazamiento vertical */
    background-color: #f9f9f9; /* Fondo claro para resaltar el área */
}


#nuevoGrupoSelect, #ejercicioNuevoGrupoSelect {
    width: 100%; /* Aprovecha todo el ancho disponible */
    padding: 8px; /* Añade un relleno para mayor comodidad */
    margin-top: 4px; /* Espacio mínimo superior para separación */
    cursor: pointer; /* Indica que es un elemento interactivo */
    font-size: 16px; /* Aumenta el tamaño del texto */
}

    
        /* Estilo básico para el botón de refrescar */
        .btn-refrescar {
            background-color: #f5f5f5;
            border: none;
            cursor: pointer;
            padding: 10px;
            border-radius: 5px;
            font-size: 16px;
        }
        /* Icono de refrescar utilizando una entidad HTML */
        .btn-refrescar:before {
            content: "\21BB"; /* Símbolo de flecha circular */
            margin-right: 5px;
        }

#campoEspañol, #campoIngles, #campoGrupo {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
}

</style>
</head>

<input type="file" id="fileInput" />
<button onclick="uploadFile()">Subir Fichero de Ejercicios</button>
<button onclick="descargarCSV()">Guardar cambios y Descargar</button>


<script>
function uploadFile() {
  const fileInput = document.getElementById('fileInput');
  if (!fileInput.files.length) {
    alert('Por favor, selecciona un archivo para subir.');
    return;
  }

  const file = fileInput.files[0];
  const reader = new FileReader();

  reader.onload = function(e) {
    // Una vez que el archivo ha sido leído, su contenido se almacena en la variable 'text'.
    const text = e.target.result;
    procesarCSV(text)

    // Aquí puedes hacer lo que necesites con el texto. Por ahora, solo lo mostraremos en una alerta.
    alert("Contenido del archivo cargado:\n" + text.substring(0, 200) + "..."); // Muestra los primeros 200 caracteres
  };

  reader.onerror = function() {
    alert('Hubo un error al leer el archivo');
  };

  // Leer el contenido del archivo como un string.
  reader.readAsText(file);
}


function generarTextoCSV(registros) {
    // Unir cada registro array a un string, separando las columnas con '#', y cada fila con un salto de línea
    return registros.map(fila => fila.join('#')).join('\n');
}

function descargarCSV() {
    const csvText = generarTextoCSV(registros);
    const blob = new Blob([csvText], { type: 'text/csv;charset=utf-8;' });
    const url = URL.createObjectURL(blob);

    // Crear un enlace para la descarga
    const downloadLink = document.createElement("a");
    downloadLink.href = url;
    downloadLink.download = "datos_exportados.csv";

    // Simular un clic en el enlace para iniciar la descarga y luego limpiar
    document.body.appendChild(downloadLink);
    downloadLink.click();
    document.body.removeChild(downloadLink);
    URL.revokeObjectURL(url);
    //guardarCambios();  //para guardar también en el servidor
}

</script>

<script>
let registros = []; // Almacena aquí todos los registros procesados

  // Función para cargar y procesar el archivo CSV
  function cargarCSVDesdeServidor() {
    fetch('https://javiergimenez.es/api/get-translations-file')
      .then(response => response.text())
      .then(text => procesarCSV(text))
      .catch(error => console.error('Error al cargar el archivo CSV:', error));
  }

  // Función para procesar el contenido del CSV y almacenar los registros
  function procesarCSV(csvText) {
    const filas = csvText.split('\n');
    registros = filas.map(fila => {
        const columnas = fila.replace(/\r/g, '').split('#');
        return columnas.map(columna => columna.trim()); 
    });
    const tipos = new Set();
    //tipos.add('All'); // Añadir una opción predeterminada
    tipos.add('repaso'); // Añadir una opción siempre existente

    registros.forEach(registro => {
      if (registro.length > 2) {
        registro[2].split('/').forEach(tipo => tipos.add(tipo));
      }
    });
    llenarSelector(Array.from(tipos), 'nuevoGrupoSelect'); // Actualiza para llenar el <select>
    llenarSelector(Array.from(tipos), 'ejercicioNuevoGrupoSelect'); // Actualiza para llenar el <select>
    //añadir al prinpio de la lista de tipos el valor 'All'
    tipos.add('All'); // Añadir una opción predeterminada
    llenarSelector(Array.from(tipos), 'ejercicios');

  }


//  function guardarCambios() {
//    // Supongamos que 'registros' contiene los datos modificados que queremos guardar
//    fetch('http://localhost:5500/save-translations-file', {
//        method: 'POST',
//        headers: {
//            'Content-Type': 'application/json',
//        },
//        body: JSON.stringify(registros)  // 'registros' debe ser el array de tus datos
//    })
//    .then(response => response.json())
//    .then(data => {
//        console.log(data.message); // Mensaje de éxito
//       alert(data.message);
//    })
//    .catch((error) => {
//        console.error('Error:', error); // Manejo de errores
//        alert('Error al guardar los cambios');
//    });
//}


    function llenarSelector(tipos, elementoID) {
        const elemento = document.getElementById(elementoID);
        elemento.innerHTML = ''; // Limpiar opciones existentes

        tipos.forEach(tipo => {
            const opcion = document.createElement('option');
            opcion.value = tipo;
            opcion.textContent = tipo; // Solo para el selector de ejercicios
            elemento.appendChild(opcion);
        });
    }

  

  // Función para manejar el cambio de selección y actualizar la variable ejercicio
  let ejercicio = 'All'; // Variable para almacenar el tipo seleccionado
  function actualizarEjercicio(valorSeleccionado) {
    ejercicio = valorSeleccionado;
    mostrarSiguiente()
    console.log('Ejercicio seleccionado:', ejercicio);
  }


//let indiceActual = -1; // Variable para almacenar el índice del registro actual

// Función para obtener un registro aleatorio que contenga el valor seleccionado como un string completo en la tercera columna
function obtenerRegistroAleatorio(tipoSeleccionado) {
  let registrosFiltrados;

  // Si el tipo seleccionado es "All", no filtra los registros
  if (tipoSeleccionado.trim() === "All") {
    registrosFiltrados = registros;
  } else {
    // Filtra los registros verificando que el tipo seleccionado, sin espacios en blanco,
    // sea exactamente uno de los tipos en la tercera columna después de hacer split por '/'
    registrosFiltrados = registros.filter(registro => {
      if (registro.length > 2) {
        // Divide la tercera columna por '/', elimina espacios en blanco y verifica si el tipo seleccionado está presente
        const tipos = registro[2].split('/').map(tipo => tipo.trim());
        return tipos.includes(tipoSeleccionado.trim());
      }
      return false;
    });
  }

  if (registrosFiltrados.length === 0) {
    return null; // O manejar de otra manera si no hay registros que coincidan
  }
  
  // Selecciona uno de los registros filtrados al azar y lo devuelve
  const indiceAleatorio = Math.floor(Math.random() * registrosFiltrados.length);
//   indiceActual = indiceAleatorio
  return registrosFiltrados[indiceAleatorio];
}


function eliminarRegistro(Pregistro) {
    // Encuentra el índice del registro para eliminar en 'registros' basado solo en los dos primeros campos
    español = Pregistro[0]
    ingles = Pregistro[1]
    let indice = registros.findIndex(registro => 
        registro[0] === español && registro[1] === ingles
    );

    // Verifica si se encontró el índice
    if (indice !== -1) {
        // Elimina el registro de 'registros' usando splice
        registros.splice(indice, 1);
        console.log('Registro eliminado correctamente.');
    } else {
        console.log('Registro no encontrado.');
    }
}





// Función para añadir un grupo a la tercera columna del registro especificado por el índice
function añadirGrupo(registro, grupo) {
  // Verifica que el índice esté dentro del rango de los registros cargados
  if (registro) {
    // Divide la tercera columna por '/', elimina espacios en blanco de los extremos, y verifica si el grupo ya está presente
    let grupos = registro[2].split('/').map(g => g.trim());
    if (!grupos.includes(grupo)) {
      grupos.push(grupo); // Añade el nuevo grupo
      registro[2] = grupos.join('/'); // Reconstruye la tercera columna y actualiza el registro
      return true; // Indica que el grupo fue añadido exitosamente
    } else {
      return false; // Indica que el grupo ya estaba presente
    }
  } else {
    console.error('Índice fuera de rango.');
    return false; // También retorna false si el índice está fuera de rango
  }
}

// Función para eliminar un grupo de la tercera columna del registro especificado por el índice
function eliminaGrupo(registro, grupo) {
  // Verifica que el índice esté dentro del rango de los registros cargados
  if (registro) {
    // Divide la tercera columna por '/', elimina espacios en blanco de los extremos, y verifica si el grupo existe
    let grupos = registro[2].split('/').map(grupo => grupo.trim());
    if (grupos.includes(grupo)) {
      grupos = grupos.filter(g => g !== grupo); // Elimina el grupo
      registro[2] = grupos.join('/'); // Reconstruye la tercera columna y actualiza el registro
      return true; // Indica que el grupo fue eliminado exitosamente
    } else {
      return false; // Indica que el grupo no se encontró
    }
  } else {
    console.error('Índice fuera de rango.');
    return false; // También retorna false si el índice está fuera de rango
  }
}



function arrayToCSVString(arrayDeArrays) {
    // Mapea cada sub-array a un string, uniendo los campos con '#'
    const lineas = arrayDeArrays.map(registro => registro.join('#'));
    // Une todas las líneas con saltos de línea para formar el string completo del CSV
    const csvString = lineas.join('\n');
    return csvString;
}


  // Llama a cargarCSVDesdeServidor al cargar la página
 // window.addEventListener('load', cargarCSVDesdeServidor);



</script>


<form id="inicioForm">
    <div>
        <label for="enunciado">Traduce esto:</label><br>
        <textarea id="enunciado" name="enunciado" rows="1" cols="100"></textarea>
    </div>
    <div>
        <label for="ejercicios">Juegos:</label> <button class="btn-refrescar" onclick="procesarCSV(arrayToCSVString(registros))">Refrescar</button>
<br>
        <select id="ejercicios" onchange="actualizarEjercicio(this.value)">
        <!-- Las opciones se llenarán dinámicamente -->
        </select>
       
    </div>
    <button type="submit">Siguiente!</button>

</form>

<div id="manejoGrupos">
    <!-- Primera fila con botones para añadir a repaso y eliminar del grupo -->
    <div>
        <button id="btnAñadirRepaso">Añadir a repaso</button>
        <button id="btnEliminarRepaso">Eliminar de repaso</button>
        <button id="btnEliminarRegistro">Eliminar registro</button>
        <button id="btnGuardarCambios">Guardar Cambios</button>
    </div>
    
    <p> AÑADIR O ELIMINAR ESTE EJERCICIO DE UN GRUPO: </p>
    <div>
        <select id="nuevoGrupoSelect" onchange="seleccionarGrupo()">
            <!-- Las opciones se llenarán dinámicamente -->
        </select>
        <input type="text" id="nuevoGrupoInput" placeholder="Nuevo grupo o seleccione">
        <button id="btnAñadirNuevoGrupo">Añadir al grupo</button>
        <button id="btnEliminarDelGrupo">Eliminar del grupo</button>

    </div>
</div>

<script>


let registroActual

//document.getElementById('btnEliminarRegistro').addEventListener('click', eliminarRegistro(registroActual));
document.getElementById('btnEliminarRegistro').addEventListener('click', function() {
    eliminarRegistro(registroActual);
});

//document.getElementById('btnGuardarCambios').addEventListener('click', guardarCambios);
//document.getElementById('btnGuardarCambios').addEventListener('click', function() {
//    guardarCambios();
//});



document.getElementById('btnAñadirRepaso').addEventListener('click', function() {
    if (añadirGrupo(registroActual, 'repaso')) {
        alert('Añadido a repaso'); // Si el grupo fue añadido exitosamente
    } else {
        // Si el registro ya estaba en el grupo o hubo algún otro error
        alert('El registro ya está en el grupo repaso o no se pudo añadir');
    }
});

document.getElementById('btnEliminarRepaso').addEventListener('click', function() {
    // Utiliza la función eliminaGrupo para intentar eliminar el registro actual del grupo "repaso"
    if (eliminaGrupo(registroActual, 'repaso')) {
        alert('Eliminado del grupo repaso'); // Si el grupo fue eliminado exitosamente
    } else {
        // Si el registro no estaba en el grupo o hubo algún otro error
        alert('No se ha eliminado del grupo porque, en realidad, no se encontraba en el grupo repaso');
    }
});


document.getElementById('btnEliminarDelGrupo').addEventListener('click', function() {
    // Asume que tienes un selector definido en alguna parte de tu HTML para elegir el grupo
    let grupoSeleccionado = document.getElementById('nuevoGrupoInput').value; // Asegúrate de que el ID coincida
    if (eliminaGrupo(registroActual, grupoSeleccionado)) {
        alert('Eliminado del grupo ' + grupoSeleccionado); // Opcional: feedback al usuario
    } else {
        // Muestra un mensaje de error si el registro no se encontraba en el grupo
        alert(`No se ha eliminado del grupo porque, en realidad, no se encontraba en el grupo ${grupoSeleccionado}`);
    }
});

document.getElementById('btnAñadirNuevoGrupo').addEventListener('click', function() {
    let nuevoGrupo = document.getElementById('nuevoGrupoInput').value.trim();
    if (nuevoGrupo) {
        // La función añadirGrupo ahora devuelve true si el grupo fue añadido con éxito, y false si ya existía
        if (añadirGrupo(registroActual, nuevoGrupo)) {
            alert('Añadido al grupo ' + nuevoGrupo); // Si el grupo fue añadido exitosamente
        } else {
            // Si el registro ya estaba en el grupo
            alert(`El registro ya se encuentra en el grupo ${nuevoGrupo}, por lo que no se ha añadido de nuevo.`);
        }
    } else {
        alert('Por favor, introduce un nombre de grupo válido.'); // Si el input está vacío o es inválido
    }
});


function mostrarSiguiente() {
    // obtener un registro aleatorio
    const enunciadoTextArea = document.getElementById('enunciado');
    const ejercicio = document.getElementById('ejercicios').value;
    registroActual = obtenerRegistroAleatorio(ejercicio);
    enunciadoTextArea.value = registroActual[0];
 
    document.getElementById('responseText').innerText = registroActual[0];
}

function seleccionarGrupo() {
    const select = document.getElementById('nuevoGrupoSelect');
    const input = document.getElementById('nuevoGrupoInput');
    if (select.value) {
        input.value = select.value; // Transfiere el valor seleccionado al input
    }
}

function seleccionarNuevoEjercicioGrupo() {
    const select = document.getElementById('ejercicioNuevoGrupoSelect');
    const input = document.getElementById('campoGrupo');
    if (select.value) {
        input.value = select.value; // Transfiere el valor seleccionado al input
    }
}


document.getElementById('inicioForm').addEventListener('submit', function(e) {
    // Prevenir el comportamiento predeterminado del formulario
    e.preventDefault();

    // obtener un registro aleatorio
    mostrarSiguiente()
  
});

</script>


<div id="audioPlayerContainer"></div> <div id="audioPlayerContainerSol"></div>
<button id="recordButton" style="width: 100%; height: 50px;">Pulsa para grabar/detener</button>


<!-- Estilos para textarea y botón de envío -->
<div id="textButtonContainer" style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4"></textarea>
    <button id="sendTextButton">Enviar Texto</button>
</div>



<div id="responseText" style="height: 250px; margin-top: 20px; border: 1px solid #ddd; padding: 10px; overflow-y:auto;"></div>


<p> AÑADIR UN NUEVO EJERCICIO: </p>
<div id="nuevoEjercicio">
    <div>
        <label for="campoEspañol">Español:</label>
        <input type="text" id="campoEspañol" name="campoEspañol">
    </div>
    <div>
        <label for="campoIngles">Inglés:</label>
        <input type="text" id="campoIngles" name="campoIngles">
    </div>
    <div>
        <label for="campoGrupo">Grupo:</label>
         <select id="ejercicioNuevoGrupoSelect" onchange="seleccionarNuevoEjercicioGrupo()">
            <!-- Las opciones se llenarán dinámicamente -->
        </select>        
        <input type="text" id="campoGrupo" name="campoGrupo">
        
    </div>
    <button id="btnCrearNuevoEjercicio">Crear nuevo ejercicio</button>
</div>


<script>

document.getElementById('btnCrearNuevoEjercicio').addEventListener('click', function() {
    // Obtiene los valores de los campos de texto
    const español = document.getElementById('campoEspañol').value.trim();
    const ingles = document.getElementById('campoIngles').value.trim();
    
    const grupo = document.getElementById('campoGrupo').value.trim();


    // Valida que los campos no estén vacíos (ajusta según necesidades)
    if (español && ingles && grupo) {
        // Inserta los nuevos valores al final del array 'registros'
        registros.push([español, ingles, grupo]);
        console.log('Nuevo ejercicio creado:', español, ingles, grupo);
        console.log('Registros actualizados:', registros);

        // Opcional: Limpia los campos después de la inserción
        document.getElementById('campoEspañol').value = '';
        document.getElementById('campoIngles').value = '';
        document.getElementById('campoGrupo').value = '';
    } else {
        // Opcional: Mensaje de error si algún campo está vacío
        console.error('Todos los campos son requeridos para crear un nuevo ejercicio.');
        alert('Por favor, rellena todos los campos.');
    }
});


let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        console.log('mediaRecorder Detenido!!');
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};



function enviarTexto(texto) {
    updateResponseText(" ---> " + texto + " (sol: " + registroActual[1] + ")" + "\n");
    obtenerYReproducir(texto, false);
    obtenerYReproducir(registroActual[1]);
}




function enviarAudioAlServidor(blob) {
    let formData = new FormData();
    formData.append('file', blob, 'grabacion.ogg');

    fetch('https://javiergimenez.es/api/only_transcribe', {
        method: 'POST',
        body: formData,
    })
    .then(response => response.json())
    .then(data => {
        updateResponseText(" ---> " + data.entrada + " (sol: " + registroActual[1] + ")" + "\n");
     

    })
    .catch(error => {
        console.error('Error al enviar el audio:', error);
    });

    obtenerYReproducir(registroActual[1]);

    createAudioPlayerFromBlob(blob)
}



// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTexto(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};




function updateResponseText(text) {
    // document.getElementById('responseText').innerText += text;
    textarea = document.getElementById('responseText')
    textarea.innerText += text;
    textarea.scrollTop = textarea.scrollHeight;
}



function obtenerYReproducir(texto, sol=true) {
    //relleno para que no tenga problemas el generador de voz.
    //if (texto.length < 20) { texto += " . (This phrase was too short)";}
    //else if (texto.length < 25) { texto += " . .";}
    console.log('Obteniendo audio para:', texto);
    fetch('https://javiergimenez.es/api/audio', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto, pausa: 'false' })
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {            
            playAudio(data.audio_base64, sol);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}




function playAudio(audioBase64, sol=true) {
    let audioContainer=""
    if (sol) {
        audioContainer = document.getElementById('audioPlayerContainerSol');
    } else {
        audioContainer = document.getElementById('audioPlayerContainer');
    }
    audioContainer.innerHTML = ''; // Limpia el contenedor

    let audioSrc = `data:audio/wav;base64,${audioBase64}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = false;

    console.log('creando audio para solución.');
    audioContainer.appendChild(audioPlayer);
}

function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

function createAudioPlayerFromBlob(blob) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    // Crea una URL de objeto para el Blob
    let audioSrc = URL.createObjectURL(blob);
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = false;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

cargarCSVDesdeServidor()

</script>


Writing juegoTraduccionProxy.html


Juegos conversacionales CSS mejorado para moviles

In [ ]:
%%writefile juegoClienteProxyInversoMovile.html

<head>
    <meta charset="UTF-8">
    <title>Translate Game</title>
  <style>


    body {
        font-family: Arial, sans-serif; /* Mejora la tipografía general */
    }


    #system_prompt {
        height: 150px;
        display: none;
    }


    #inicioForm {
        max-width: 1000px; /* Limita el ancho del formulario */
        margin: 20px auto; /* Centra el formulario */
        padding: 20px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Añade un sombreado ligero */
    }

    #inicioForm div {
        margin-bottom: 15px; /* Añade más espacio entre los campos */
    }

    #inicioForm label {
        font-weight: bold; /* Hace que las etiquetas sean más notables */
        display: block; /* Asegura que la etiqueta esté encima del input */
        margin-bottom: 5px; /* Espacio entre la etiqueta y el campo */
    }

    #inicioForm select, #inicioForm textarea, #inicioForm button {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
    }

    #inicioForm select {
        cursor: pointer; /* Indica que es un elemento interactivo */
        font-size: 66px; /* Aumenta el tamaño del texto */
    }

    #inicioForm textarea {
        resize: vertical; /* Permite al usuario ajustar la altura verticalmente */
        display: none;
    }

    #inicioForm button {
        background-color: #007bff; /* Color de fondo */
        color: white; /* Color del texto */
        border: none; /* Elimina el borde */
        padding: 10px 15px; /* Añade relleno */
        font-size: 58px; /* Aumenta el tamaño del texto */
        cursor: pointer; /* Indica que es un elemento interactivo */
        border-radius: 5px; /* Bordes redondeados */
    }

    #inicioForm button:hover {
        background-color: #0056b3; /* Oscurece el botón al pasar el mouse */
    }

#audioPlayerContainer {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 50px; /* Espacio antes del botón de grabación */
    margin-top: 30px;
}

#recordButton {
    background-color: #f44336; /* Color rojo para la grabación */
    color: white;
    border: none;
    padding: 10px 0;
    font-size: 58px;
    border-radius: 5px;
    cursor: pointer;
}

#recordButton:hover {
    background-color: #d32f2f; /* Oscurece el botón al pasar el mouse */
}

/* Estilos para el área de texto y botón de envío */
/* Contenedor del área de texto y el botón */
div#textButtonContainer {
    display: flex; /* Establece el contenedor para usar flexbox */
    justify-content: space-between; /* Espacia los elementos uniformemente */
    align-items: center; /* Alinea los elementos verticalmente en el centro */
}

/* Área de texto */
#textInput {
    flex-grow: 1; /* Permite que el área de texto crezca para ocupar el espacio disponible */
    margin-right: 10px; /* Añade un margen a la derecha para separarlo del botón */
    border: 1px solid #ccc; /* Establece un borde sutil */
    border-radius: 5px; /* Bordes redondeados */
    padding: 8px; /* Añade padding interno */
    font-size: xx-large; /* Aumenta el tamaño del texto */
}

/* Botón */
#sendTextButton {
    padding: 8px 15px; /* Ajusta el padding para dimensionar el botón */
    background-color: #4CAF50; /* Color de fondo */
    color: white; /* Color del texto */
    border: none; /* Elimina el borde */
    border-radius: 5px; /* Bordes redondeados */
    cursor: pointer; /* Cambia el cursor a mano al pasar sobre el botón */
    font-size: xx-large; /* Aumenta el tamaño del texto */
}

#sendTextButton:hover {
    background-color: #388E3C; /* Oscurece el botón al pasar el mouse */
}


#responseText {
    height: 250px;
    margin-top: 20px;
    border: 1px solid #ddd;
    padding: 10px;
    overflow-y: auto; /* Asegura el desplazamiento vertical */
    background-color: #f9f9f9; /* Fondo claro para resaltar el área */
    font-size: xxx-large;
}



</style>
</head>

<script>

window.intervalId = window.intervalId || null; // Asegura que intervalId sea global y única

var URL = 'https://javiergimenez.es/api'; // Inicializa la variable URL


function updateURL() {
    URL = document.getElementById('url').value; // Actualiza la variable URL con el valor del campo de entrada
}

function startInterval() {
    if (window.intervalId === null) {
        window.intervalId = setInterval(() => {
            fetch(URL + '/alive')
                .then(response => response.json())
                .then(data => console.log('Alive:', data))
                .catch(error => console.error('Error fetching alive status:', error));
        }, 30000);
        console.log('Intervalo iniciado.');
    } else {
        console.log('Ya existe un intervalo en ejecución.');
    }
}

function stopInterval() {
    if (window.intervalId !== null) {
        clearInterval(window.intervalId);
        console.log('Intervalo detenido.');
        window.intervalId = null;
    } else {
        console.log('No hay un intervalo para detener.');
    }
}


// startInterval()

// document.getElementById('btnAccesoMic').addEventListener('click', async () => {
//     try {
//         const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
//         // Procesa el stream aquí
//         console.log('Acceso al micrófono concedido');
//     } catch (error) {
//         console.error('Acceso al micrófono denegado:', error);
//     }
// });

document.addEventListener('DOMContentLoaded', function() {
    window.scrollTo(0, 0); // Asegura que la página comience en la parte superior
    document.getElementById('ejercicios').focus(); // Luego establece el foco en el selector
});



// Objeto para mapear los ejercicios a sus strings correspondientes
const ejerciciosStrings = {
    "guessing_game1": {
        systemPrompt: `This is a conversational game in which you have to think in this famous character: #personaje, and the user have to guess this character. You should aswer the user questions about the character. Be concise but give some clues. NEVER say the name of the character until the end. If the user guesses the character then you will say: "Congratulations, you got it right, the character was #personaje". If user give up you will say: "what a shame!, the character was #personaje
Game success example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he #personaje?
assistant: Congratulations, you were right, it was #personaje.
Game give up example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he Benito Perez?
assistant: No, it has nothing to do with.
user: I give up. Who is it?
assistant: what a shame!, the character was #personaje
`,
        saludo: "I'm thinking of a famous fictional character, guess which one it is."
    },
    "guessing_game2": {
        systemPrompt: "You are #personaje, the fictional character. You have to take on the personality of that character and engage in conversations about your events and experiences.",
        saludo: "I'm #personaje, the fictional character. Ask me anything you want to know about me."
    },
    "guessing_game3": {
        systemPrompt: "This is a conversational game in which you have to guess a famous character. You should make questions to the user in order to guess the character that the user have choosen.",
        saludo: "Do you want to play? I will guess your choosen character by asking about it."
    },
    "yes_no_game": {
        systemPrompt: `IMPORTANT: You only can answer "yes" or "no" (nothing more!).
This is a conversational game between you and a the user. The game consists that only at the beggininig you tell the user only a piece of the context and the user having to guess the "key point" of the context from "yes or no questions". The User could ask anything about the story (context) to guess the "key point" of the context but Assistant could only answer "Yes" or "not". If the user asks a question that does not lend itself or cannot be answered with a "yes" or "no" such as "What is the man's name?" then Assistant will respond: "Only "yes or no" questions. When the user guesses the key point of the story you will say: "Congratulations, you have guessed the key to the story.
Game context: A man named Edgar is the lighthouse keeper of Águilas for 30 years. He always turns on the lighthouse at dusk and shortly after sleeps in a small room next to its large lamp. On Edgar's birthday, at dusk after lighting the lighthouse, he decided to go to dinner with an old friend to celebrate his birthday. During dinner he drank more than necessary and they both got drunk. Afther the dinner Edgar accompanied his friend to her house and then went to his house, the lighthouse, where he sleeps every night. Upon entering the lighthouse and going up to his room, due to his drunkenness and the fact that he was very sleepy, he decided to turn off the light (which was actually the light from the main lamp of the lighthouse) to sleep off the drunkenness and did it without realizing or knowing it was dangerous. During the early hours of the morning, a cruise ship full of passengers crashed into the cliff that the lighthouse protected because, when it was turned off, neither the lookout, nor the captain, nor the rest of the crew nor the passengers could see that they were heading against the cliff. An hour later Edgar wakes up, it hasn't dawned yet but you can hear sirens and a lot of noise from the rescuers who are trying to rescue the shipwrecked. Edgar turns on the lamp to illuminate the scene where hundreds of dead shipwrecked people continually crash against the cliff due to the waves. Faced with this heartbreaking reality and his feeling of guilt, Edgar decides to commit suicide by jumping from the top of the lighthouse. The key point of the story that the user must find out is: "Edgar commits suicide because he was the LIGHTHOUSE keeper." or similar but always emphasizing that he was the lighthouse keeper.
IMPORTANT: You only can answer "yes" or "no" or "Only yes or no questions"
Examples of correct answers:
user: What is Edgar's job?
Assistant: Only yes or no questions.
user: Is Edgar a man?
Assistant: yes.`,
        saludo: `This is I can show about the hidden story: Edgar was dazed and comes to his room, turns off the light and lies down on his bed. He wakes up a few hours later, turns on the light, looks out the window and is so horrified that he ends up jumping out of the window and committing suicide.
Guess what happened.
IMPORTANT: From now on I can only answer you "yes" or "no" and nothing more.`
    },
    "English_teacher": {
        systemPrompt: `You are Sofie, an english teacher 29 years old. You will have simple dialogues with the student in your charge. you will only have concise conversations with short sentences so that the student is encouraged to converse.
Also you can offer translations exercises but only from spainish to enlgish. if you do translations exercises about a topic you must always say the sentence to translate in spanish: How do you say 'me gustaría viajar a París'? and then de user will be able response in english. You will not ask for doing translates from english to spanish, only from spanish to english.`,
        saludo: "Good Morning. What is your name?"
    }
    // Añade más ejercicios según sea necesario
};
</script>


<form id="inicioForm">
    <div>
       <!-- <label for="system_prompt">System Prompt:</label><br>-->
        <textarea id="system_prompt" name="system_prompt" rows="10" cols="100"></textarea>
    </div>
    <div>
        <!--<label for="saludo">Saludo:</label><br>-->
        <textarea id="saludo" name="saludo" rows="4" cols="100"></textarea>
    </div>
    <div>
        <!--<label for="ejercicios">Juegos:</label><br>-->
        <select id="ejercicios" name="ejercicios">
            <option value="">Selecciona un Juego</option>
            <option value="English_teacher">English teacher</option>
            <option value="guessing_game1">You Guess fictional character</option>
            <option value="guessing_game2">You speak with fictional character</option>
            <option value="guessing_game3">Chatbot guess your fictional character</option>
            <!-- <option value="yes_no_game">yes no game</option> -->
        </select>
    </div>
<!--    <div>
        <label for="url">URL:</label>
        <input type="text" id="url" name="url" value="https://javiergimenez.es/api" onchange="updateURL()">
    </div>-->
    <button type="submit">EMPEZAR!</button>

</form>


<div style="margin-bottom: 20px; display: flex; align-items: center;">
    <button id="downloadButton" style="background-color: #4CAF50; /* Color de fondo */
                                       color: white; /* Color del texto */
                                       padding: 15px 32px; /* Padding alrededor del texto */
                                       text-align: center; /* Alinea el texto al centro */
                                       text-decoration: none; /* Elimina la decoración del texto */
                                       display: inline-block; /* Hace que el botón sea un bloque en línea */
                                       font-size: 26px; /* Tamaño del texto */
                                       margin: 4px 2px; /* Margen alrededor del botón */
                                       cursor: pointer; /* Cambia el cursor a un puntero */
                                       border: none; /* Elimina el borde */
                                       border-radius: 8px; /* Redondea las esquinas del botón */
    ">Descargar Conversación</button>
    <div id="audioPlayerAllContainer"></div>
</div>


<script>
    document.getElementById('downloadButton').addEventListener('click', function() {
        obtenerYReproducirAll(); // Llama a la función en vez de redirigir
    });
</script>


<script>
historico = ""
const userID = Math.floor(Math.random() * (999999));
console.log("userID:" + userID);


document.getElementById('ejercicios').addEventListener('change', function() {
    var selectedKey = this.value; // La clave seleccionada del objeto
    if (selectedKey) {
        // Actualiza los textareas con los valores correspondientes
        document.getElementById('system_prompt').value = ejerciciosStrings[selectedKey].systemPrompt;
        document.getElementById('saludo').value = ejerciciosStrings[selectedKey].saludo;
    }
});


document.getElementById('inicioForm').addEventListener('submit', function(e) {
    // Prevenir el comportamiento predeterminado del formulario
    e.preventDefault();
    fetch(URL + '/alive')
      .then(response => response.json())
      .then(data => console.log('Alive:', data))
      .catch(error => console.error('Error fetching alive status:', error));
    startInterval()
    // Obtener los valores de los campos del formulario
    const system_prompt = document.getElementById('system_prompt').value;
    const saludo = document.getElementById('saludo').value;

    document.getElementById('responseText').innerText = ""
    //obtenerYReproducirAudio(saludo)
    //updateResponseText(saludo + "\n")

    // Crear el cuerpo de la solicitud
    const data = { system_prompt, saludo };

    // Realizar la llamada al servicio Flask
    fetch(URL + '/inicio', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json',
        },
        body: JSON.stringify(data),
    })
    .then(response => response.json())
    .then(data => {
        let saludo = data.message;
        historico = data.historico
        obtenerYReproducirSaludo(saludo)
        updateResponseText(saludo + "\n")

    })
    .catch((error) => {
        console.error('Error:', error);
        alert('VUELVE A DAR AL PLAY!');
    });
    alert('ESPERE UNOS SEGUNDOS HASTA QUE EMPIECE LA CONVERSACIÓN');
});
</script>

<!-- <button id="btnAccesoMic">Permitir acceso al micrófono</button> -->


<div id="audioPlayerContainer"></div>
<button id="recordButton" style="width: 100%; height: 100px;">Pulsa para grabar/detener</button>


<!-- Estilos para textarea y botón de envío -->
<div id="textButtonContainer" style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4"></textarea>
    <button id="sendTextButton">Enviar Texto</button>
</div>



<div id="responseText" style="height: 750px; margin-top: 20px; border: 1px solid #ddd; padding: 10px; overflow-y:auto;"></div>
<script>
let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        console.log('mediaRecorder Detenido!!');
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};


async function enviarAudioAlServidor(blob) {
        let formData = new FormData();
        formData.append('file', blob, 'grabacion.ogg');
        formData.append('historico', JSON.stringify(historico)); // Se asegura de enviar como cadena JSON
        formData.append('userID', userID);

        try {
            const response = await fetch(URL + '/transcribe', {
                method: 'POST',
                body: formData, // Solo se envía formData
            });
            const data = await response.json();
            //return data; // Devuelve los datos procesados
       
            updateResponseText("\nyo: " + data.entrada + "\n\n" + "\n***********************************\n" + "respuesta: ");

            // Reinicia el buffer de audio de manera más eficiente
            audioBuffer = Array(currentAudioIndex).fill({});
            currentAudioIndex = 0;

            historico += data.prompt
            await obtenerPartes();
            } catch (error) {
                console.error('Error al enviar el audio:', error);
                alert("VUELVE A DAR AL PLAY! (después puedes continuar la conversación)");
            }
}

async function obtenerPartes(indice = 0) {
    try {
        const response = await fetch(`${URL}/get_next_part?index=${indice}&userID=${userID}`);
        const partData = await response.json();
        if (partData.output !== "") {
            let trozo = partData.output;
            updateResponseText(trozo);
            historico += trozo

            await obtenerYReproducirAudio(trozo, indice);
            await obtenerPartes(indice + 1); // Llamada recursiva con el índice incrementado
        }
        else{
            historico += "<|eot_id|>"
        }
    } catch (error) {
        console.error('Error al obtener la siguiente parte:', error);
    }
}

// Asumiendo que obtenerYReproducirAudio ya fue optimizado como se mostró anteriormente




// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTextoAlServidor(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};



let audioBuffer = [];
// inicializa audioBuffer con 100 elementos vacíos
for (let i = 0; i < 100; i++) {
    audioBuffer.push({});
}
let milisegundosInicio = Date.now();

let currentAudioIndex = 0; // Para controlar el orden de reproducción


async function enviarTextoAlServidor(texto) {
    try {
        const response = await fetch(URL + '/texto', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({ texto: texto, historico: historico, userID: userID })
        });
        const data = await response.json();
        updateResponseText("\nyo: " + data.entrada + "\n\n" + "\n***********************************\n" + "respuesta: ");

        historico += data.prompt

        // Reinicia el buffer de audio de manera más eficiente
        audioBuffer = Array(currentAudioIndex).fill({});
        currentAudioIndex = 0;

        await obtenerPartes();
    } catch (error) {
        console.error('Error al enviar el texto:', error);
        alert("VUELVE A DAR AL PLAY! (después puedes continuar la conversación)")
    }
}


async function obtenerYReproducirAudio(texto, index) {
    try {
        const response = await fetch(URL + '/audio', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({ texto: texto, pausa: 'true'})
        });
        const data = await response.json();
        if (data.audio_base64) {
            console.log('Enviando audio al buffer de reproducción, tamaño:', data.audio_base64.length, 'índice:', index);
            addAudioClipToBuffer(data.audio_base64, index);
        }
    } catch (error) {
        console.error('Error al obtener el audio:', error);
    }
}



function updateResponseText(text) {
    // document.getElementById('responseText').innerText += text;
    textarea = document.getElementById('responseText')
    textarea.innerText += text;
    textarea.scrollTop = textarea.scrollHeight;
}



function obtenerYReproducirAll() {
    fetch(URL + '/all_conversation', {
        method: 'GET',
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            createAudioAllPlayer(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}


function createAudioAllPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerAllContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

function obtenerYReproducirSaludo(texto) {
    //console.log('Obteniendo audio para:', texto, 'microsegundos:', Date.now() - milisegundosInicio);
    fetch(URL + '/audio', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto, pausa: 'true'})
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
         //   console.log('Obteniendo audio, tamaño:', data.audio_base64.length, 'SALUDO', 'microsegundos:', Date.now()-milisegundosInicio);
            playAudio(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}





function addAudioClipToBuffer(base64Audio, index) {
    audioBuffer[index] = { audio: base64Audio};
    let audioPlayer = document.getElementById('audioPlayerContainer').querySelector('audio');
    //console.log('Despues de meter en Buffer, VIENDO si ejecuto play con indice:', index, 'pausa:',audioPlayer.paused, 'currentAudioIndex', currentAudioIndex,'microsegundos:', Date.now()-milisegundosInicio);
    if (audioPlayer.paused && currentAudioIndex == index) {
        playNextAudioClip();
    }
}

//function esObjetoVacio(obj) {
//  return Object.keys(obj).length === 0;
//}
function esObjetoVacio(obj) {
  try {
    // Intenta obtener las claves del objeto y verifica si su longitud es 0
    return Object.keys(obj).length === 0;
  } catch (error) {
    // Si ocurre un error (por ejemplo, si obj no es un objeto), devuelve true
    //console.error("Se produjo un error porque en esa posición del buffer está vacía: ", error);
    return true;
  }
}



function playNextAudioClip() {
    //console.log('Entrando en playNextAudioClip para ver si reproducimos audio con currentAudioIndex:', currentAudioIndex, 'microsegundos:', Date.now()-milisegundosInicio, 'esObjetoVacio:', esObjetoVacio(audioBuffer[currentAudioIndex]));
    if (!esObjetoVacio(audioBuffer[currentAudioIndex])) {
       // console.log('Iniciando variables para Reproducion audio concurrentAudioIndex:', currentAudioIndex, 'tamaño:', audioBuffer[currentAudioIndex].audio.length, 'microsegundos:', Date.now()-milisegundosInicio);
        const audioData = audioBuffer[currentAudioIndex];
        audioBuffer[currentAudioIndex] = {}; // Limpia el elemento actual
        currentAudioIndex++;

        let audioContainer = document.getElementById('audioPlayerContainer');
        audioContainer.innerHTML = ''; // Limpia el contenedor

        let audioSrc = `data:audio/wav;base64,${audioData.audio}`;
        let audioPlayer = document.createElement('audio');
        audioPlayer.src = audioSrc;
        audioPlayer.controls = true;
        audioPlayer.autoplay = true;

      //  console.log('Justo antes de APPENDCHILD Reproduciendo audio currentAudioIndex:', currentAudioIndex, 'microsegundos:', Date.now()-milisegundosInicio);
        audioContainer.appendChild(audioPlayer);

        audioPlayer.onended = playNextAudioClip;
    }
}

function playAudio(audioBase64) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    audioContainer.innerHTML = ''; // Limpia el contenedor

    let audioSrc = `data:audio/wav;base64,${audioBase64}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;

    //console.log('Reproduciendo saludo! Justo antes de APPENDCHILD', 'microsegundos:', Date.now()-milisegundosInicio);
    audioContainer.appendChild(audioPlayer);
}

function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

obtenerYReproducirSaludo("Wellcom to Conversational Games!")



</script>


traducciones mejoradas para moviles

In [ ]:

<head>
    <meta charset="UTF-8">
    <title>Translate Game</title>
  <style>


    body {
        font-family: Arial, sans-serif; /* Mejora la tipografía general */
    }



    #inicioForm {
        max-width: 1000px; /* Limita el ancho del formulario */
        margin: 20px auto; /* Centra el formulario */
        padding: 20px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Añade un sombreado ligero */
    }

    #inicioForm div {
        margin-bottom: 15px; /* Añade más espacio entre los campos */
    }

    #inicioForm label {
        font-weight: bold; /* Hace que las etiquetas sean más notables */
        display: block; /* Asegura que la etiqueta esté encima del input */
        margin-bottom: 5px; /* Espacio entre la etiqueta y el campo */
    }

    #inicioForm select, #inicioForm textarea, #inicioForm button {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
    }

    #inicioForm select {
        cursor: pointer; /* Indica que es un elemento interactivo */
        font-size: 66px; /* Aumenta el tamaño del texto */
    }

    #inicioForm textarea {
        resize: vertical; /* Permite al usuario ajustar la altura verticalmente */
        font-size: xx-large; /* Aumenta el tamaño del texto */
    }

    #inicioForm button {
        background-color: #007bff; /* Color de fondo */
        color: white; /* Color del texto */
        border: none; /* Elimina el borde */
        padding: 10px 15px; /* Añade relleno */
        font-size: 58px; /* Aumenta el tamaño del texto */
        cursor: pointer; /* Indica que es un elemento interactivo */
        border-radius: 5px; /* Bordes redondeados */
    }

    #inicioForm button:hover {
        background-color: #0056b3; /* Oscurece el botón al pasar el mouse */
    }

#audioPlayerContainer {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#audioPlayerContainerSol {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#recordButton {
    background-color: #f44336; /* Color rojo para la grabación */
    color: white;
    border: none;
    padding: 10px 0;
    font-size: 68px;
    border-radius: 5px;
    cursor: pointer;
}

#recordButton:hover {
    background-color: #d32f2f; /* Oscurece el botón al pasar el mouse */
}

/* Estilos para el área de texto y botón de envío */
/* Contenedor del área de texto y el botón */
div#textButtonContainer {
    display: flex; /* Establece el contenedor para usar flexbox */
    justify-content: space-between; /* Espacia los elementos uniformemente */
    align-items: center; /* Alinea los elementos verticalmente en el centro */
}

/* Área de texto */
#textInput {
    flex-grow: 1; /* Permite que el área de texto crezca para ocupar el espacio disponible */
    margin-right: 10px; /* Añade un margen a la derecha para separarlo del botón */
    border: 1px solid #ccc; /* Establece un borde sutil */
    border-radius: 5px; /* Bordes redondeados */
    padding: 8px; /* Añade padding interno */
    font-size: xx-large; /* Aumenta el tamaño del texto */

}

/* Botón */
#sendTextButton {
    padding: 8px 15px; /* Ajusta el padding para dimensionar el botón */
    background-color: #4CAF50; /* Color de fondo */
    color: white; /* Color del texto */
    border: none; /* Elimina el borde */
    border-radius: 5px; /* Bordes redondeados */
    cursor: pointer; /* Cambia el cursor a mano al pasar sobre el botón */
}

#sendTextButton:hover {
    background-color: #388E3C; /* Oscurece el botón al pasar el mouse */
}


#responseText {
    height: 250px;
    margin-top: 20px;
    border: 1px solid #ddd;
    padding: 10px;
    overflow-y: auto; /* Asegura el desplazamiento vertical */
    background-color: #f9f9f9; /* Fondo claro para resaltar el área */
    font-size: xxx-large; /* Aumenta el tamaño del texto */
}


#nuevoGrupoSelect, #ejercicioNuevoGrupoSelect {
    width: 100%; /* Aprovecha todo el ancho disponible */
    padding: 8px; /* Añade un relleno para mayor comodidad */
    margin-top: 4px; /* Espacio mínimo superior para separación */
    cursor: pointer; /* Indica que es un elemento interactivo */
    font-size: 36px; /* Aumenta el tamaño del texto */
}

    
        /* Estilo básico para el botón de refrescar */
        .btn-refrescar {
            background-color: #f5f5f5;
            border: none;
            cursor: pointer;
            padding: 10px;
            border-radius: 5px;
            font-size: 16px;
        }
        /* Icono de refrescar utilizando una entidad HTML */
        .btn-refrescar:before {
            content: "\21BB"; /* Símbolo de flecha circular */
            margin-right: 5px;
        }

#campoEspañol, #campoIngles, #campoGrupo {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
}

</style>
</head>

<input type="file" id="fileInput" style="font-size: xxx-large;"/>
<button onclick="uploadFile()" style="font-size: xxx-large;">Subir Fichero de Ejercicios</button>
<button onclick="descargarCSV()" style="font-size: xxx-large;">Guardar cambios y Descargar</button>


<script>
function uploadFile() {
  const fileInput = document.getElementById('fileInput');
  if (!fileInput.files.length) {
    alert('Por favor, selecciona un archivo para subir.');
    return;
  }

  const file = fileInput.files[0];
  const reader = new FileReader();

  reader.onload = function(e) {
    // Una vez que el archivo ha sido leído, su contenido se almacena en la variable 'text'.
    const text = e.target.result;
    procesarCSV(text)

    // Aquí puedes hacer lo que necesites con el texto. Por ahora, solo lo mostraremos en una alerta.
    alert("Contenido del archivo cargado:\n" + text.substring(0, 200) + "..."); // Muestra los primeros 200 caracteres
  };

  reader.onerror = function() {
    alert('Hubo un error al leer el archivo');
  };

  // Leer el contenido del archivo como un string.
  reader.readAsText(file);
}


function generarTextoCSV(registros) {
    // Unir cada registro array a un string, separando las columnas con '#', y cada fila con un salto de línea
    return registros.map(fila => fila.join('#')).join('\n');
}

function descargarCSV() {
    const csvText = generarTextoCSV(registros);
    const blob = new Blob([csvText], { type: 'text/csv;charset=utf-8;' });
    const url = URL.createObjectURL(blob);

    // Crear un enlace para la descarga
    const downloadLink = document.createElement("a");
    downloadLink.href = url;
    downloadLink.download = "datos_exportados.csv";

    // Simular un clic en el enlace para iniciar la descarga y luego limpiar
    document.body.appendChild(downloadLink);
    downloadLink.click();
    document.body.removeChild(downloadLink);
    URL.revokeObjectURL(url);
    //guardarCambios();  //para guardar también en el servidor
}

</script>

<script>
let registros = []; // Almacena aquí todos los registros procesados

  // Función para cargar y procesar el archivo CSV
  function cargarCSVDesdeServidor() {
    fetch('https://javiergimenez.es/api/get-translations-file')
      .then(response => response.text())
      .then(text => procesarCSV(text))
      .catch(error => console.error('Error al cargar el archivo CSV:', error));
  }

  // Función para procesar el contenido del CSV y almacenar los registros
  function procesarCSV(csvText) {
    const filas = csvText.split('\n');
    registros = filas.map(fila => {
        const columnas = fila.replace(/\r/g, '').split('#');
        return columnas.map(columna => columna.trim()); 
    });
    const tipos = new Set();
    //tipos.add('All'); // Añadir una opción predeterminada
    tipos.add('repaso'); // Añadir una opción siempre existente

    registros.forEach(registro => {
      if (registro.length > 2) {
        registro[2].split('/').forEach(tipo => tipos.add(tipo));
      }
    });
    llenarSelector(Array.from(tipos), 'nuevoGrupoSelect'); // Actualiza para llenar el <select>
    llenarSelector(Array.from(tipos), 'ejercicioNuevoGrupoSelect'); // Actualiza para llenar el <select>
    //añadir al prinpio de la lista de tipos el valor 'All'
    tipos.add('All'); // Añadir una opción predeterminada
    llenarSelector(Array.from(tipos), 'ejercicios');

  }


//  function guardarCambios() {
//    // Supongamos que 'registros' contiene los datos modificados que queremos guardar
//    fetch('http://localhost:5500/save-translations-file', {
//        method: 'POST',
//        headers: {
//            'Content-Type': 'application/json',
//        },
//        body: JSON.stringify(registros)  // 'registros' debe ser el array de tus datos
//    })
//    .then(response => response.json())
//    .then(data => {
//        console.log(data.message); // Mensaje de éxito
//       alert(data.message);
//    })
//    .catch((error) => {
//        console.error('Error:', error); // Manejo de errores
//        alert('Error al guardar los cambios');
//    });
//}


    function llenarSelector(tipos, elementoID) {
        const elemento = document.getElementById(elementoID);
        elemento.innerHTML = ''; // Limpiar opciones existentes

        tipos.forEach(tipo => {
            const opcion = document.createElement('option');
            opcion.value = tipo;
            opcion.textContent = tipo; // Solo para el selector de ejercicios
            elemento.appendChild(opcion);
        });
    }

  

  // Función para manejar el cambio de selección y actualizar la variable ejercicio
  let ejercicio = 'All'; // Variable para almacenar el tipo seleccionado
  function actualizarEjercicio(valorSeleccionado) {
    ejercicio = valorSeleccionado;
    mostrarSiguiente()
    console.log('Ejercicio seleccionado:', ejercicio);
  }


//let indiceActual = -1; // Variable para almacenar el índice del registro actual

// Función para obtener un registro aleatorio que contenga el valor seleccionado como un string completo en la tercera columna
function obtenerRegistroAleatorio(tipoSeleccionado) {
  let registrosFiltrados;

  // Si el tipo seleccionado es "All", no filtra los registros
  if (tipoSeleccionado.trim() === "All") {
    registrosFiltrados = registros;
  } else {
    // Filtra los registros verificando que el tipo seleccionado, sin espacios en blanco,
    // sea exactamente uno de los tipos en la tercera columna después de hacer split por '/'
    registrosFiltrados = registros.filter(registro => {
      if (registro.length > 2) {
        // Divide la tercera columna por '/', elimina espacios en blanco y verifica si el tipo seleccionado está presente
        const tipos = registro[2].split('/').map(tipo => tipo.trim());
        return tipos.includes(tipoSeleccionado.trim());
      }
      return false;
    });
  }

  if (registrosFiltrados.length === 0) {
    return null; // O manejar de otra manera si no hay registros que coincidan
  }
  
  // Selecciona uno de los registros filtrados al azar y lo devuelve
  const indiceAleatorio = Math.floor(Math.random() * registrosFiltrados.length);
//   indiceActual = indiceAleatorio
  return registrosFiltrados[indiceAleatorio];
}


function eliminarRegistro(Pregistro) {
    // Encuentra el índice del registro para eliminar en 'registros' basado solo en los dos primeros campos
    español = Pregistro[0]
    ingles = Pregistro[1]
    let indice = registros.findIndex(registro => 
        registro[0] === español && registro[1] === ingles
    );

    // Verifica si se encontró el índice
    if (indice !== -1) {
        // Elimina el registro de 'registros' usando splice
        registros.splice(indice, 1);
        console.log('Registro eliminado correctamente.');
    } else {
        console.log('Registro no encontrado.');
    }
}





// Función para añadir un grupo a la tercera columna del registro especificado por el índice
function añadirGrupo(registro, grupo) {
  // Verifica que el índice esté dentro del rango de los registros cargados
  if (registro) {
    // Divide la tercera columna por '/', elimina espacios en blanco de los extremos, y verifica si el grupo ya está presente
    let grupos = registro[2].split('/').map(g => g.trim());
    if (!grupos.includes(grupo)) {
      grupos.push(grupo); // Añade el nuevo grupo
      registro[2] = grupos.join('/'); // Reconstruye la tercera columna y actualiza el registro
      return true; // Indica que el grupo fue añadido exitosamente
    } else {
      return false; // Indica que el grupo ya estaba presente
    }
  } else {
    console.error('Índice fuera de rango.');
    return false; // También retorna false si el índice está fuera de rango
  }
}

// Función para eliminar un grupo de la tercera columna del registro especificado por el índice
function eliminaGrupo(registro, grupo) {
  // Verifica que el índice esté dentro del rango de los registros cargados
  if (registro) {
    // Divide la tercera columna por '/', elimina espacios en blanco de los extremos, y verifica si el grupo existe
    let grupos = registro[2].split('/').map(grupo => grupo.trim());
    if (grupos.includes(grupo)) {
      grupos = grupos.filter(g => g !== grupo); // Elimina el grupo
      registro[2] = grupos.join('/'); // Reconstruye la tercera columna y actualiza el registro
      return true; // Indica que el grupo fue eliminado exitosamente
    } else {
      return false; // Indica que el grupo no se encontró
    }
  } else {
    console.error('Índice fuera de rango.');
    return false; // También retorna false si el índice está fuera de rango
  }
}



function arrayToCSVString(arrayDeArrays) {
    // Mapea cada sub-array a un string, uniendo los campos con '#'
    const lineas = arrayDeArrays.map(registro => registro.join('#'));
    // Une todas las líneas con saltos de línea para formar el string completo del CSV
    const csvString = lineas.join('\n');
    return csvString;
}


  // Llama a cargarCSVDesdeServidor al cargar la página
 // window.addEventListener('load', cargarCSVDesdeServidor);



</script>


<form id="inicioForm">
    <div>
        <!-- <label for="enunciado">Traduce esto:</label><br> -->
        <textarea id="enunciado" name="enunciado" rows="4" cols="100"></textarea>
    </div>
    <div>
        <!-- <label for="ejercicios">Juegos:</label>  -->
        <!-- <button class="btn-refrescar" onclick="procesarCSV(arrayToCSVString(registros))">Refrescar</button> -->
<br>
        <select id="ejercicios" onchange="actualizarEjercicio(this.value)">
        <!-- Las opciones se llenarán dinámicamente -->
        </select>
       
    </div>
    <button type="submit">Siguiente!</button>

</form>

<div id="manejoGrupos">
    <!-- Primera fila con botones para añadir a repaso y eliminar del grupo -->
    <div>
        <button id="btnAñadirRepaso" style="font-size: xxx-large; margin: 10px;">Añadir a repaso</button>
        <button id="btnEliminarRepaso" style="font-size: xx-large;">Eliminar de repaso</button>
        <button id="btnEliminarRegistro" style="font-size: xx-large;">Eliminar registro</button>
        <!-- <button id="btnGuardarCambios">Guardar Cambios</button> -->
    </div>
    
    <p> AÑADIR O ELIMINAR ESTE EJERCICIO DE UN GRUPO: </p>
    <div>
        <select id="nuevoGrupoSelect" onchange="seleccionarGrupo()">
            <!-- Las opciones se llenarán dinámicamente -->
        </select>
        <input type="text" id="nuevoGrupoInput" placeholder="Nuevo grupo" style="font-size: x-large; margin: 10px;">
        <button id="btnAñadirNuevoGrupo" style="font-size: x-large; margin: 10px;">Añadir al grupo</button>
        <button id="btnEliminarDelGrupo" style="font-size: x-large; margin: 10px;">Eliminar del grupo</button>
        <button style="font-size: x-large; margin: 10px;" onclick="procesarCSV(arrayToCSVString(registros))">Refrescar</button>

    </div>
</div>

<script>


let registroActual

//document.getElementById('btnEliminarRegistro').addEventListener('click', eliminarRegistro(registroActual));
document.getElementById('btnEliminarRegistro').addEventListener('click', function() {
    eliminarRegistro(registroActual);
});

//document.getElementById('btnGuardarCambios').addEventListener('click', guardarCambios);
//document.getElementById('btnGuardarCambios').addEventListener('click', function() {
//    guardarCambios();
//});



document.getElementById('btnAñadirRepaso').addEventListener('click', function() {
    if (añadirGrupo(registroActual, 'repaso')) {
        alert('Añadido a repaso'); // Si el grupo fue añadido exitosamente
    } else {
        // Si el registro ya estaba en el grupo o hubo algún otro error
        alert('El registro ya está en el grupo repaso o no se pudo añadir');
    }
});

document.getElementById('btnEliminarRepaso').addEventListener('click', function() {
    // Utiliza la función eliminaGrupo para intentar eliminar el registro actual del grupo "repaso"
    if (eliminaGrupo(registroActual, 'repaso')) {
        alert('Eliminado del grupo repaso'); // Si el grupo fue eliminado exitosamente
    } else {
        // Si el registro no estaba en el grupo o hubo algún otro error
        alert('No se ha eliminado del grupo porque, en realidad, no se encontraba en el grupo repaso');
    }
});


document.getElementById('btnEliminarDelGrupo').addEventListener('click', function() {
    // Asume que tienes un selector definido en alguna parte de tu HTML para elegir el grupo
    let grupoSeleccionado = document.getElementById('nuevoGrupoInput').value; // Asegúrate de que el ID coincida
    if (eliminaGrupo(registroActual, grupoSeleccionado)) {
        alert('Eliminado del grupo ' + grupoSeleccionado); // Opcional: feedback al usuario
    } else {
        // Muestra un mensaje de error si el registro no se encontraba en el grupo
        alert(`No se ha eliminado del grupo porque, en realidad, no se encontraba en el grupo ${grupoSeleccionado}`);
    }
});

document.getElementById('btnAñadirNuevoGrupo').addEventListener('click', function() {
    let nuevoGrupo = document.getElementById('nuevoGrupoInput').value.trim();
    if (nuevoGrupo) {
        // La función añadirGrupo ahora devuelve true si el grupo fue añadido con éxito, y false si ya existía
        if (añadirGrupo(registroActual, nuevoGrupo)) {
            alert('Añadido al grupo ' + nuevoGrupo); // Si el grupo fue añadido exitosamente
        } else {
            // Si el registro ya estaba en el grupo
            alert(`El registro ya se encuentra en el grupo ${nuevoGrupo}, por lo que no se ha añadido de nuevo.`);
        }
    } else {
        alert('Por favor, introduce un nombre de grupo válido.'); // Si el input está vacío o es inválido
    }
});


function mostrarSiguiente() {
    // obtener un registro aleatorio
    const enunciadoTextArea = document.getElementById('enunciado');
    const ejercicio = document.getElementById('ejercicios').value;
    registroActual = obtenerRegistroAleatorio(ejercicio);
    enunciadoTextArea.value = registroActual[0];
 
    document.getElementById('responseText').innerText = registroActual[0];
}

function seleccionarGrupo() {
    const select = document.getElementById('nuevoGrupoSelect');
    const input = document.getElementById('nuevoGrupoInput');
    if (select.value) {
        input.value = select.value; // Transfiere el valor seleccionado al input
    }
}

function seleccionarNuevoEjercicioGrupo() {
    const select = document.getElementById('ejercicioNuevoGrupoSelect');
    const input = document.getElementById('campoGrupo');
    if (select.value) {
        input.value = select.value; // Transfiere el valor seleccionado al input
    }
}


document.getElementById('inicioForm').addEventListener('submit', function(e) {
    // Prevenir el comportamiento predeterminado del formulario
    e.preventDefault();

    // obtener un registro aleatorio
    mostrarSiguiente()
  
});

</script>


<div id="audioPlayerContainer"></div> <div id="audioPlayerContainerSol"></div>
<button id="recordButton" style="width: 100%; height: 120px;">Pulsa para grabar/detener</button>


<!-- Estilos para textarea y botón de envío -->
<div id="textButtonContainer" style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4"></textarea>
    <button id="sendTextButton">Enviar Texto</button>
</div>



<div id="responseText" style="height: 450px; margin-top: 20px; border: 1px solid #ddd; padding: 10px; overflow-y:auto;"></div>


<p> AÑADIR UN NUEVO EJERCICIO: </p>
<div id="nuevoEjercicio">
    <div>
        <label for="campoEspañol">Español:</label>
        <input type="text" id="campoEspañol" name="campoEspañol">
    </div>
    <div>
        <label for="campoIngles">Inglés:</label>
        <input type="text" id="campoIngles" name="campoIngles">
    </div>
    <div>
        <label for="campoGrupo">Grupo:</label>
         <select id="ejercicioNuevoGrupoSelect" onchange="seleccionarNuevoEjercicioGrupo()">
            <!-- Las opciones se llenarán dinámicamente -->
        </select>        
        <input type="text" id="campoGrupo" name="campoGrupo">
        
    </div>
    <button id="btnCrearNuevoEjercicio">Crear nuevo ejercicio</button>
</div>


<script>

document.getElementById('btnCrearNuevoEjercicio').addEventListener('click', function() {
    // Obtiene los valores de los campos de texto
    const español = document.getElementById('campoEspañol').value.trim();
    const ingles = document.getElementById('campoIngles').value.trim();
    
    const grupo = document.getElementById('campoGrupo').value.trim();


    // Valida que los campos no estén vacíos (ajusta según necesidades)
    if (español && ingles && grupo) {
        // Inserta los nuevos valores al final del array 'registros'
        registros.push([español, ingles, grupo]);
        console.log('Nuevo ejercicio creado:', español, ingles, grupo);
        console.log('Registros actualizados:', registros);

        // Opcional: Limpia los campos después de la inserción
        document.getElementById('campoEspañol').value = '';
        document.getElementById('campoIngles').value = '';
        document.getElementById('campoGrupo').value = '';
    } else {
        // Opcional: Mensaje de error si algún campo está vacío
        console.error('Todos los campos son requeridos para crear un nuevo ejercicio.');
        alert('Por favor, rellena todos los campos.');
    }
});


let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        console.log('mediaRecorder Detenido!!');
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};



function enviarTexto(texto) {
    updateResponseText(" --->\nR:" + texto + "\nS: " + registroActual[1]  + "\n");
    obtenerYReproducir(texto, false);
    obtenerYReproducir(registroActual[1]);
}




function enviarAudioAlServidor(blob) {
    let formData = new FormData();
    formData.append('file', blob, 'grabacion.ogg');

    fetch('https://javiergimenez.es/api/only_transcribe', {
        method: 'POST',
        body: formData,
    })
    .then(response => response.json())
    .then(data => {
        updateResponseText(" --->\nR:" + data.entrada + "\nS: " + registroActual[1] + "\n");
     

    })
    .catch(error => {
        console.error('Error al enviar el audio:', error);
    });

    obtenerYReproducir(registroActual[1]);

    createAudioPlayerFromBlob(blob)
}



// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTexto(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};




function updateResponseText(text) {
    // document.getElementById('responseText').innerText += text;
    textarea = document.getElementById('responseText')
    textarea.innerText += text;
    textarea.scrollTop = textarea.scrollHeight;
}



function obtenerYReproducir(texto, sol=true) {
    //relleno para que no tenga problemas el generador de voz.
    //if (texto.length < 20) { texto += " . (This phrase was too short)";}
    //else if (texto.length < 25) { texto += " . .";}
    console.log('Obteniendo audio para:', texto);
    fetch('https://javiergimenez.es/api/audio', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto, pausa: 'false' })
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {            
            playAudio(data.audio_base64, sol);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}




function playAudio(audioBase64, sol=true) {
    let audioContainer=""
    if (sol) {
        audioContainer = document.getElementById('audioPlayerContainerSol');
    } else {
        audioContainer = document.getElementById('audioPlayerContainer');
    }
    audioContainer.innerHTML = ''; // Limpia el contenedor

    let audioSrc = `data:audio/wav;base64,${audioBase64}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = false;

    console.log('creando audio para solución.');
    audioContainer.appendChild(audioPlayer);
}

function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

function createAudioPlayerFromBlob(blob) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    // Crea una URL de objeto para el Blob
    let audioSrc = URL.createObjectURL(blob);
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = false;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

cargarCSVDesdeServidor()

</script>


# server rápido

In [28]:
# %%writefile server_conversacional.py

#########################################
####    MODEL IN LLAMA_CPP
#########################################

#MODELO LO CARGAMOS A PARTE PORQUE TARDA EN CARGARSE

#########################################
####    CHATBOT
#########################################
# %%writefile modelo_llama3.py
# from cargar_llama_cpp import model
LOGGING = False
from threading import Lock

# global model
def encontrar_coincidencia(texto, cadena_busqueda="<|eot_id|>"):
    """
    Esta función busca la primera aparición de una cadena de búsqueda en un texto dado y devuelve el substring
    desde el principio del texto hasta el final de esta coincidencia (incluida).
    
    Parámetros:
    texto (str): El texto en el que se buscará la cadena.
    cadena_busqueda (str): La cadena de caracteres que se buscará en el texto.
    
    Retorna:
    str: El substring desde el inicio hasta el final de la primera coincidencia de la cadena buscada,
    incluyendo la coincidencia. Si no se encuentra ninguna coincidencia, devuelve una cadena vacía.
    """
    # Buscar la posición de la primera coincidencia de la cadena en el texto
    indice = texto.find(cadena_busqueda)
    
    if indice != -1:
        # Devolver el substring desde el inicio hasta el final de la coincidencia
        return texto[:indice + len(cadena_busqueda)]
    else:
        # Devolver una cadena vacía si no hay coincidencia
        return ""


# VENTANA DESLIZANTE
def ajustar_contexto(texto, max_longitud=15000, secuencia="<|start_header_id|>", system_end="<|eot_id|>"):
    system_prompt = encontrar_coincidencia(texto, system_end)
    # Comprobar si la longitud del texto es mayor que el máximo permitido
    if len(texto) > max_longitud:
        indice_secuencia = 0

        while True:
            # Buscar la secuencia de ajuste
            indice_secuencia = texto.find(secuencia, indice_secuencia + 1)

            # Si la secuencia no se encuentra o el texto restante es menor que la longitud máxima
            if indice_secuencia == -1 or len(system_prompt) + len(texto) - indice_secuencia <= max_longitud:
                break

        # Si encontramos una secuencia válida
        if indice_secuencia != -1:
            return system_prompt + texto[indice_secuencia:]

        else:
            # Si no se encuentra ninguna secuencia adecuada, tomar los últimos max_longitud caracteres
            return system_prompt + texto[-max_longitud + len(system_prompt):]
    else:
        return system_prompt + texto



generate_lock = Lock()

def pre_warm_chat(historico, max_additional_tokens=100, stop=["</s>","user:"], short_answer=True, streaming=False, printing=False):
 
    # if short_answer:
    #     # añade como stop el salto de linea
    #     stop.append("\n")

    outputs = ""

    with generate_lock:
        response=model(prompt=historico, max_tokens=max_additional_tokens, temperature=0, top_p=1,
                    top_k=0, repeat_penalty=1,
                    stream=True)


        respuesta = ""

        for chunk in response:
            trozo = chunk['choices'][0]['text']
            # trozo.replace("\n", "")
            # trozo.replace("<|EOT|>", "")
            # for caracter in trozo:
            #     cadena_con_codigos += f"{caracter}({ord(caracter)}) "
            respuesta += trozo
            print(trozo, end="", flush=True)
            # linea += trozo

            # if len(linea)>35:
                # print(linea, end="", flush=True)  # Impresión en consola

                # linea = ""


        outputs = historico + respuesta
        return historico, outputs



import threading

class EstadoGeneracion:
    def __init__(self):
        # lista de 100 partes del texto
        self.parts = [""]*100
        self.top = -1
        self.generando = False
        self.primer_audio = ""
        # self.lock = threading.Lock()


estado_generacion = {}
estado_generacion['anonimo'] = EstadoGeneracion()

# generate_lock = Lock()

def generate_in_file_parts(userID, historico, ai, user, input_text, max_additional_tokens=2000, short_answer=False, streaming=True, printing=True):
    global estado_generacion

    printf(HISTORICO_LOG, f"generando para USER:{userID}\nhistorico:{historico}\ninput_text:{input_text}")
    # global generate_lock
    if userID not in estado_generacion:
        estado_generacion[userID] = EstadoGeneracion()

    estado_generacion[userID].generando = True
    with generate_lock:
        estado_generacion[userID].top = -1
        print(f"Empezamos a generar ponemos el TOP a {estado_generacion[userID].top} para USER:{userID}!!:", input_text)
        indiceParte = 0
        # estado_generacion[userID].generando = True
        print(f"generando={estado_generacion[userID].generando}; Generando respuesta para USER:{userID}:", input_text)
        # estado_generacion.parts = []  # lista de partes del texto
        parte_actual = ""  # añade la primera parte

        if short_answer:
            # añade como stop el salto de linea
            stop.append("\n")


        prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

        final_prompt = historico + "\n" + prompt


        model_inputs = final_prompt

        outputs = ""
        print(f"{ai}:", end="")


        # outputs = ""
        colchon = (CONTEXT_LENGTH - max_additional_tokens)*3
        print("Longitud:",len(final_prompt), "Colchon:", colchon)
        if len(final_prompt)> colchon: #cuenta la vieja cada token son 3 caracteres (como poco)
            print("Ajustando contexto!!!")
            final_prompt = ajustar_contexto(final_prompt, max_longitud=colchon)
            print(final_prompt)
            #contexto ajustado imprimir los primeros 500 caracteres
            # print(final_prompt[:500])
            #imprimir los 500 últimos caracteres
            # print(final_prompt[-500:])

        response=model(prompt=final_prompt, max_tokens=max_additional_tokens, temperature=0, top_p=1,
                      top_k=0, repeat_penalty=1,
                      stream=True)


        respuesta = ""
        estado_generacion[userID].primer_audio = "wait"

        for chunk in response:
            trozo = chunk['choices'][0]['text']
            # trozo.replace("\n", "")
            # trozo.replace("<|EOT|>", "")
            # for caracter in trozo:
            #     cadena_con_codigos += f"{caracter}({ord(caracter)}) "
            respuesta += trozo
            print(trozo, end="", flush=True)

            outputs += trozo
            parte_actual += trozo
            if trozo in ",;:.?!" and len(parte_actual)>44 or trozo in "." and len(parte_actual)>4:

                if indiceParte == 0: #creamos primer audio rápido para rápida respuesta
                    estado_generacion[userID].primer_audio = voz_sintetica_english(parte_actual, "true")
                    
                estado_generacion[userID].parts[indiceParte] = parte_actual
                estado_generacion[userID].top = indiceParte
                if LOGGING:
                    print(f"trozo generado para USER: {userID}:", parte_actual)
                    print("se ha generado para entrada de indiceParte (ahora TOP tb vale esto):", indiceParte)
                indiceParte += 1                
                # print("se incrementa indiceParte (pero TOP aun no) a:", indiceParte)
                parte_actual = ""


        if len(parte_actual)>1:
            estado_generacion[userID].parts[indiceParte] = parte_actual
            estado_generacion[userID].top = indiceParte

        all_text = model_inputs + outputs + "<|eot_id|>"
        estado_generacion[userID].generando = False
        if LOGGING:
            print(f"generando={estado_generacion[userID].generando}; Respuesta Terminada. El total generado para {user}:", outputs)

        return all_text, outputs


#########################################
####    SERVER
#########################################
from flask import Flask, request, jsonify, send_file, send_from_directory
from flask_cors import CORS
from threading import Lock
import threading
import os
import torch
from pydub import AudioSegment
import pandas as pd
import random

import time


# import whisper
# modelWhisper = whisper.load_model('medium')



# parts = []  # lista de partes del texto
# generando = False
# global model



# from modelo_llama3 import generate_in_file_parts, pre_warm_chat
if LOGGING:
    print("El modelo es:", model)

ai = "assistant"
user = "user"

contexto = """

"""

system_prompt = """
You are a kind and helpful assistan bot. You are here to help the user to find the best answer to his question.
"""

saludo = "Hello, I am ready to receive and process your input."

idioma = "en"

import sys

# Verifica si el comando tenía flag -s o --short
if "-s" in sys.argv or "--short" in sys.argv:
    short_answer = True
else:
    short_answer = False

# Si encuentra el flag -es cambia el idioma a español
if "-es" in sys.argv:
    idioma = "es"

# Filtra los argumentos para eliminar los flags
args = [arg for arg in sys.argv[1:] if arg not in ["-s", "--short", "-es"]]

# Asigna los valores a system_prompt y saludo basándose en los argumentos restantes
if len(args) > 0:
    system_prompt = args[0]
if len(args) > 1:
    saludo = args[1]


historico = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{saludo}<|eot_id|>"


# load model
# load_model()

if LOGGING:
    print(f"{ai}:", saludo)

# Crea un bloqueo para proteger el código contra la concurrencia a la hora de transcribir
transcribe_lock = Lock()


# generate_lock = Lock()

app = Flask(__name__)
# app.config['MAX_CONTENT_LENGTH'] = 30 * 1024 * 1024  # 30 MB

#import logging
#logging.basicConfig(level=logging.DEBUG)

CORS(app)

output = ""


@app.route('/alive')
def alive():
    return jsonify(True)



def eliminar_archivos_temp(nombre_inicio='temp_sync'):
    # Obtener una lista de todos los archivos en el directorio actual
    archivos = os.listdir('.')
    
    # Filtrar archivos que comienzan con 'temp_sync'
    archivos_temp = [archivo for archivo in archivos if archivo.startswith(nombre_inicio)]
    
    # Iterar sobre la lista de archivos y eliminarlos
    for archivo in archivos_temp:
        try:
            os.remove(archivo)
            print(f"Archivo eliminado: {archivo}")
        except Exception as e:
            print(f"No se pudo eliminar el archivo {archivo}. Razón: {e}")

@app.route('/inicio', methods=['POST'])
def print_strings():
    # global modelo
    # global historico
    eliminar_archivos_temp("received_audio")
    historico = ""

    # Lee el archivo CSV y selecciona un personaje de ficción al azar
    def elegir_personaje_aleatorio():
        df = pd.read_csv('Personajes_ficcion.csv')
        return random.choice(df.iloc[:, 0].tolist())

    # Obtiene los datos del cuerpo de la solicitud
    data = request.json

    # Extrae los strings del objeto JSON
    system_prompt = data.get('system_prompt')
    saludo = data.get('saludo')

    # Preprocesamiento para reemplazar "#personaje" con un personaje aleatorio
    if "#personaje" in system_prompt:
        personaje_aleatorio = elegir_personaje_aleatorio()
        system_prompt = system_prompt.replace("#personaje", personaje_aleatorio)

    # Preprocesamiento para reemplazar "#personaje" con un personaje aleatorio en el saludo
    if "#personaje" in saludo:
        saludo = saludo.replace("#personaje", personaje_aleatorio)

    # Imprime los strings en el log del servidor
    if LOGGING:
        print("INICIALIZANDO CONVERSACIÓN")
    conversation_file = 'conversacion.mp3'
    # si existe el archivo de conversación, lo elimina
    if os.path.exists(conversation_file):
        os.remove(conversation_file)

    if LOGGING:
        print(f"system: {system_prompt}, saludo: {saludo}")

    # if modelo == "mistral":
    #     historico = f"system\n{system_prompt}\nassistant\n{saludo}\n"
    # elif modelo == "zypher":
    #     historico = f"{system_prompt}</s>\n\n{saludo}</s>\n"
    historico = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{saludo}<|eot_id|>"

    pre_warm_chat(historico)

    # Retorna una respuesta para indicar que se recibieron y procesaron los datos
    return jsonify({"message": saludo, "historico": historico}), 200


@app.route('/get-translations-file', methods=['GET'])
def get_translations():
    return send_from_directory(directory='.', path='translations.csv', as_attachment=True)

import csv
import shutil
@app.route('/save-translations-file', methods=['POST'])
def save_translations():
    data = request.json  # Asume que el cliente envía los datos como JSON
    if not data:
        return jsonify({'error': 'No data provided'}), 400

    try:
        # Hace una copia de seguridad del archivo translations.csv antes de modificarlo
        shutil.copy('translations.csv', 'translations.csv.bak')

        # Abre el archivo translations.csv para escribir y actualiza con los datos recibidos
        with open('translations.csv', mode='w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile, delimiter='#')
            for row in data:
                writer.writerow(row)

        return jsonify({'message': 'File successfully saved'}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/all_conversation', methods=['GET'])
def all_conversation():
    # Asegúrate de que el path al archivo sea correcto para tu estructura de proyecto
    filepath = 'conversacion.mp3'
    if not os.path.exists(filepath):
        return jsonify(error="Archivo de conversación no encontrado"), 404

    # Leer el archivo y convertirlo a base64
    with open(filepath, 'rb') as audio_file:
        audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

    return jsonify(audio_base64=audio_base64)



import subprocess

def convert_ogg_to_mp3(source_ogg_path, target_mp3_path):
    """
    Utiliza ffmpeg para convertir un archivo .ogg a .mp3.
    """
    command = ['ffmpeg', '-y' ,'-i', source_ogg_path, target_mp3_path]
    process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Si el comando falla, imprime la salida de error
    if process.returncode != 0:
        print(f"Error al convertir {source_ogg_path} a {target_mp3_path}")
        print("Salida de error de ffmpeg:")
        print(process.stderr.decode())




def convert_wav_to_mp3(source_wav_path, target_mp3_path):
    command = ['ffmpeg', '-i', source_wav_path, target_mp3_path]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


def add_audio_to_conversation_async(source_path, convert_to_mp3=False):
    def task():
        if convert_to_mp3:
            # Convertir de WAV a MP3 si es necesario
            temp_mp3_path = source_path.replace('.wav', '.mp3')
            convert_wav_to_mp3(source_path, temp_mp3_path)
            final_path = temp_mp3_path
        else:
            final_path = source_path

        # Añadir al archivo de conversación
        sound = AudioSegment.from_file(final_path)
        conversation_file = 'conversacion.mp3'
        if os.path.exists(conversation_file):
            conversation_audio = AudioSegment.from_mp3(conversation_file)
            combined_audio = conversation_audio + sound
        else:
            combined_audio = sound
        combined_audio.export(conversation_file, format='mp3')

        # Limpiar archivos temporales
        os.remove(source_path)
        if convert_to_mp3:
            os.remove(temp_mp3_path)

    thread = threading.Thread(target=task)
    thread.start()



def generate_chat_background(userID, entrada, phistorico, ai, user, short_answer):
    # global output  # Indicar que se utilizará la variable global 'output'
    if LOGGING:
        print("OJOOOOOOOO!!!!!!  generate_chat_background USERID:", userID, "entrada:", entrada)
    start_generation_time = time.time()
    # Ejecutar la generación de chat en un hilo aparte
    historico_local, output_local = generate_in_file_parts(userID, phistorico, ai, user, input_text=entrada, max_additional_tokens=2048, short_answer=short_answer, streaming=True, printing=False)
    end_generation_time = time.time()
    generation_duration = end_generation_time - start_generation_time
    if LOGGING:
        print(f"Generación completada en {generation_duration} segundos")

    # Actualizar las variables globales con los resultados obtenidos
    # global historico
    # historico = historico_local
    # output = output_local

@app.route('/transcribe', methods=['POST'])
def transcribe_audio():
    if LOGGING:
        print("Transcribiendo audio...")
    global user
    global ai


    if 'userID' not in request.form:
        return jsonify(error="No se proporcionó userID"), 400
    userID = int(request.form['userID'])

    if 'historico' not in request.form:
        return jsonify(error="No se proporcionó historico"), 400
    historico = request.form['historico']  # Asumiendo que se envía como JSON y necesitará ser parseado en Python

    # Extraer el archivo
    if 'file' not in request.files:
        return jsonify(error="No se proporcionó file"), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify(error="No selected file"), 400

    timestamp = int(time.time() * 1000)
    ogg_filepath = f"received_audio_{timestamp}.ogg"
    file.save(ogg_filepath)

    start_transcribe_time = time.time()
    if LOGGING:
        print("antes del transcribe lock, userID:", userID)
    with transcribe_lock:
        if LOGGING:
            print("después del transcribe lock, userID:", userID)
        transcripcion = modelWhisper.transcribe(ogg_filepath, fp16=False, language=idioma)
        transcripcion = transcripcion["text"]
    end_transcribe_time = time.time()
    transcribe_duration = end_transcribe_time - start_transcribe_time
    if LOGGING:
        print(f"Transcripción completada en {transcribe_duration} segundos")
        print("transcripción:", transcripcion)

    # Iniciar la generación de chat en un hilo aparte
    thread = threading.Thread(target=generate_chat_background, args=(userID, transcripcion, historico, ai, user, short_answer))
    thread.start()

    # Inicia el proceso de adición del audio .ogg en segundo plano, considerando su conversión a .mp3
    add_audio_to_conversation_async(ogg_filepath)

    # La respuesta ya no incluirá 'output' porque se generará en segundo plano
    prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{transcripcion}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    historico += prompt    
    return jsonify(entrada=transcripcion, prompt=prompt, entrada_traducida="")
    # return jsonify(entrada=transcripcion, historico=historico, entrada_traducida="")





@app.route('/only_transcribe', methods=['POST'])
def only_transcribe_audio():
    if LOGGING:
        print("Transcribiendo audio...")
    # global historico
    # global user
    # global ai

    if 'file' not in request.files:
        print("No file part")
        return jsonify(error="No file part"), 400

    file = request.files['file']
    if file.filename == '':
        print("No selected file")
        return jsonify(error="No selected file"), 400

    if LOGGING:
        print("creando fichero ogg audio antes de transcripción...")
    timestamp = int(time.time() * 1000)
    ogg_filepath = f"received_audio_{timestamp}.ogg"
    file.save(ogg_filepath)

    start_transcribe_time = time.time()
    if LOGGING:
        print("antes del transcribe lock")
    with transcribe_lock:
        if LOGGING:
            print("después del transcribe lock")
        transcripcion = modelWhisper.transcribe(ogg_filepath, fp16=False, language=idioma)
        transcripcion = transcripcion["text"]
    end_transcribe_time = time.time()
    transcribe_duration = end_transcribe_time - start_transcribe_time
    if LOGGING:
        print(f"Transcripción completada en {transcribe_duration} segundos")
        print("transcripción:", transcripcion)

    # Iniciar la generación de chat en un hilo aparte
    #thread = threading.Thread(target=generate_chat_background, args=(transcripcion, historico, ai, user, short_answer))
    #thread.start()

    # Inicia el proceso de adición del audio .ogg en segundo plano, considerando su conversión a .mp3
    add_audio_to_conversation_async(ogg_filepath)

    # La respuesta ya no incluirá 'output' porque se generará en segundo plano
    return jsonify(entrada=transcripcion, entrada_traducida="")



@app.route('/get_next_part', methods=['GET'])
def get_next_part():
    global estado_generacion
    if LOGGING:
        print("LAS CLAVES de usuario y sus TOP:")
        for clave in estado_generacion.keys():
            print(clave," TOP:", estado_generacion[clave].top)

    userID = request.args.get('userID', default=0, type=int)
    # Obtener el índice de la solicitud. Si no se proporciona, por defecto es None
    index = request.args.get('index', default=0, type=int)

    if LOGGING:
        print(f"userID:{userID} partes: {estado_generacion[userID].parts}, generando: {estado_generacion[userID].generando}, index: {index}, estado_generacion[userID].top: {estado_generacion[userID].top}")

    while True:
        # if estado_generacion.parts:
            # Verificar si el índice es válido
        if index is not None and index >= 0 and index <= estado_generacion[userID].top:

            part = estado_generacion[userID].parts[index]
            estado_generacion[userID].parts[index] = ""  # Elimina el elemento en el índice dado
            if LOGGING:
                print("con index:", index, "estado_generacion[userID].top:", estado_generacion[userID].top)    
                print(f"Enviando parte: {part}")
            return jsonify(output=part)
            # else:
            #     print("Índice inválido o fuera de límites")
            #     return jsonify(error="Índice inválido o fuera de límites"), 400
        elif estado_generacion[userID].generando:
            if LOGGING:
                print("Esperando a que se generen más partes...")
            time.sleep(0.1)  # Espera 0.1 segundos antes de volver a verificar
        else:
            if LOGGING:
                print("No hay más partes para enviar", "index:", index, "estado_generacion[userID].top:", estado_generacion[userID].top)
            return jsonify(output="") # Si 'generando' es False y 'parts' está vacía, devuelve una cadena vacía



@app.route('/texto', methods=['POST'])
def process_text():
    # global historico
    global user
    global ai

    # Recibe el texto directamente del cuerpo de la solicitud
    data = request.json
    if not data or 'texto' not in data:
        return jsonify(error="No se proporcionó texto"), 400

    texto = data['texto']

    if 'historico' not in data:
        return jsonify(error="No se proporcionó historico"), 400

    historico = data['historico']

    if 'userID' not in data:
        return jsonify(error="No se proporcionó userID"), 400

    userID = data['userID']

    if LOGGING:
        print("HISTORICO!!!:", historico)

    # Utiliza la variable 'idioma' declarada globalmente
    global idioma


    # Generación de respuesta basada en el texto proporcionado
    thread = threading.Thread(target=generate_chat_background, args=(userID, texto, historico, ai, user, short_answer))
    thread.start()


    # si el idioma es español, traduce la respuesta al español

    prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{texto}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    historico += prompt
    return jsonify(entrada=texto, historico=historico, entrada_traducida="")



import torch  # Importamos PyTorch para poder usar la función `to()`

# Función para mover recursivamente todos los tensores en una estructura anidada a un dispositivo
def move_to_device(obj, device):
    if isinstance(obj, torch.Tensor):
        return obj.to(device)
    elif isinstance(obj, dict):
        return {key: move_to_device(value, device) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [move_to_device(item, device) for item in obj]
    else:
        return obj


# PREPARAMOS INSTANCIAS FAIRSEQ

if idioma == "en":
    print("idioma ingles") # ESTO HAY QUE CAMBIARLO 
    # from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    # from fairseq.models.text_to_speech.hub_interface import TTSHubInterface

    # # Carga el modelo y la configuración
    # models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    #     "facebook/fastspeech2-en-ljspeech",
    #     arg_overrides={"vocoder": "hifigan", "fp16": False}
    # )

    # # Asegúrate de que models es una lista
    # if not isinstance(models, list):
    #     models = [models]

    # modelT2S = models[0]
    # modelT2S = modelT2S.to('cuda:0')

    # TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # # Aquí, asumimos que task.build_generator puede manejar correctamente el objeto cfg y model
    # generator = task.build_generator(models, cfg)



elif idioma == "es":
    # El modelo no entiende de números aritméticos. Esta función los convierte a palabras.
    import re
    from num2words import num2words

    def number_to_words(num_str):
        try:
            num = int(num_str)
            return num2words(num, lang='es')
        except ValueError:
            return "Por favor, introduzca un número válido."

    def process_numbers_in_line(line):
        def replace_with_words(match):
            return number_to_words(match.group())

        return re.sub(r'\b\d+\b', replace_with_words, line)

    # Ejemplo de uso
    line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
    new_line = process_numbers_in_line(line)
    print(new_line)
    # Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."


    # Diccionario con las traducciones

    def process_abrev(line):
        translations = {
        'Dr': 'doctor',
        'Sr': 'señor',
        'Sra': 'señora',
        # Añade más traducciones aquí
    }
        for abbr, full in translations.items():
            line = line.replace(f'{abbr}.', full)
            line = line.replace(f'{abbr} ', f'{full} ')
        return line

    def otras_traducciones(line):

        translations = {
        '-': ',',
        '—': ',',
        '%': ' por ciento '
        # Añade más traducciones aquí
        }

        for old, new in translations.items():
            line = line.replace(old, new)
        return line


    def preprocesado_al_modelo(line):
        line_with_numbers = process_numbers_in_line(line)
        line_with_both = process_abrev(line_with_numbers)
        line_with_all = otras_traducciones(line_with_both)
        return line_with_all

    import os
    os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
    print(os.environ['TF_ENABLE_ONEDNN_OPTS'])


    from pydub import AudioSegment, silence

    def quitar_silencios(input_filepath, output_filepath, min_silence_len=1500, new_silence_len=750, silence_thresh=-60):
        """
        Elimina silencios largos de un archivo de audio.

        Parámetros:
        - input_filepath: ruta al archivo de audio de entrada (MP3).
        - output_filepath: ruta al archivo de audio de salida (MP3).
        - min_silence_len: duración mínima del silencio a eliminar (en milisegundos).
        - new_silence_len: duración de los nuevos segmentos de silencio (en milisegundos).
        - silence_thresh: umbral de silencio (en dB).
        """

        # Cargar el archivo de audio
        audio_segment = AudioSegment.from_wav(input_filepath)

        # Encuentra los segmentos de audio separados por silencios
        segments = silence.split_on_silence(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

        # Crear un nuevo segmento de audio con silencios ajustados
        new_audio_segment = AudioSegment.empty()
        silence_chunk = AudioSegment.silent(duration=new_silence_len)  # Chunk de silencio de la nueva duración

        # Añade cada segmento de audio al nuevo audio, intercalando con los nuevos segmentos de silencio
        for segment in segments:
            new_audio_segment += segment + silence_chunk

        # Removemos el último chunk de silencio añadido
        new_audio_segment = new_audio_segment[:-new_silence_len]

        # Guarda el nuevo archivo de audio
        new_audio_segment.export(output_filepath, format="wav")


    # Cargamos el modelo generador fairseq
    from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
    import IPython.display as ipd





    # Cargamos el modelo y la configuración desde el modelo preentrenado de Hugging Face
    models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
        "facebook/tts_transformer-es-css10",
        arg_overrides={"vocoder": "hifigan", "fp16": False}
    )
    modelT2S = models[0]

    # Movemos el modelo al dispositivo GPU
    modelT2S = modelT2S.to('cuda:0')

    # Actualizamos la configuración con los datos del task
    TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # Creamos el generador
    generator = task.build_generator([modelT2S], cfg)


    import torchaudio

    import re

    def dividir_texto_con_minimo_palabras(texto, min_palabras=8):
        partes = re.split(r'([.,;:?!])', texto)
        partes_filtradas = [parte.strip() for parte in partes if parte.strip()]
        partes_combinadas = []
        parte_actual = ''

        for parte in partes_filtradas:
            if parte in '.,;:?!':
                parte_actual += parte
                if len(parte_actual.split()) >= min_palabras:
                    partes_combinadas.append(parte_actual)
                    parte_actual = ''
                else:
                    parte_actual += ' '
            else:
                parte_actual += parte + ' '

        if len(parte_actual.strip()) > 10:
            partes_combinadas.append(parte_actual.strip())

        return partes_combinadas

    def combinar_audios(audios_temporales):
        audio_combinado = "audio_combinado.wav"
        # Cargar el primer archivo de audio para inicializar la concatenación
        wav_total, rate = torchaudio.load(audios_temporales[0])

        # Iterar sobre los archivos restantes y concatenarlos
        for archivo in audios_temporales[1:]:
            wav, _ = torchaudio.load(archivo)
            wav_total = torch.cat((wav_total, wav), 1)

        # Guardar el audio combinado en un archivo final
        torchaudio.save(audio_combinado, wav_total, rate)

        return audio_combinado

    def voz_sintetica_spanish(text):
        text = preprocesado_al_modelo(text)

        lista_dividida = dividir_texto_con_minimo_palabras(text)

        audios_temporales = []

        for parte in lista_dividida:
            # Preparamos los datos de entrada para el modelo
            sample = TTSHubInterface.get_model_input(task, parte)

            # Movemos los datos al dispositivo GPU
            sample = move_to_device(sample, 'cuda:0')

            # Realizamos la predicción
            wav, rate = TTSHubInterface.get_prediction(task, modelT2S, generator, sample)

            if len(wav.shape) == 1:
                wav = wav.unsqueeze(0)

            # temp_file_name = "Temporal.wav"
            temp_file_name = f"temporal_{parte[:10]}.wav"
            torchaudio.save(temp_file_name, wav.to('cpu'), rate)
            audios_temporales.append(temp_file_name)

            combinado = combinar_audios(audios_temporales)
            sin_silencios = "sin_silencios.wav"
            # quitamos silencios
            quitar_silencios(combinado, sin_silencios, min_silence_len=1500, new_silence_len=750, silence_thresh=-60)


        with open(sin_silencios, "rb") as audio_file:
            audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

        return audio_base64



import base64
@app.route('/audio', methods=['POST'])
def generate_audio():
    texto = request.json.get('texto')
    pausa = request.json.get('pausa')
    if LOGGING:
        print('PAUSA!!!!!!!!:', pausa)
        print('TEXTO!!!!!!!!:', texto)
    if not texto:
        return jsonify(error="No se proporcionó texto"), 400

    if idioma == "en":
        audio_base64 = voz_sintetica_english(texto, pausa)
        return jsonify(audio_base64=audio_base64)
    elif idioma == "es":
        audio_base64 = voz_sintetica_spanish(texto)
        return jsonify(audio_base64=audio_base64)


import base64
@app.route('/primer_audio', methods=['GET'])
def primer_audio():
    # if LOGGING:
    

    userID = request.args.get('userID', default=0, type=int)
   
    if idioma == "en":
        while estado_generacion[userID].primer_audio == "wait":
            time.sleep(0.1)
            print("esperando primer audio")
        audio_base64 = estado_generacion[userID].primer_audio
        print('RECUPERANDO PRIMER AUDIO!!!!!!!!:', audio_base64)
        return jsonify(audio_base64=audio_base64)
    elif idioma == "es":
        audio_base64 = voz_sintetica_spanish(texto)
        return jsonify(audio_base64=audio_base64)


import io
import soundfile as sf


def add_comma_after_punctuation(text: str) -> str:
    # Lista de caracteres después de los cuales se debe agregar una coma
    punctuation_marks = ['.', '!', '?', '(', ')', ':', '\n']

    # Recorre cada marca de puntuación y añade una coma después de cada ocurrencia
    for mark in punctuation_marks:
        text = text.replace(mark, mark + ',...,')

    return text

# Ejemplo de uso de la función
#example_text = "Hello! How are you? I hope you're doing well. Let's meet tomorrow."
#modified_text = add_comma_after_punctuation(example_text)
#print(modified_text)

import io
import base64
import soundfile as sf
import os
import threading
from pydub import AudioSegment
import subprocess


def add_silence_to_audio(audio_path, duration_ms=3000):
    """Añade un segmento de silencio al final de un archivo de audio."""
    # Carga el audio
    sound = AudioSegment.from_wav(audio_path)
    # Genera el silencio
    silence = AudioSegment.silent(duration=duration_ms)
    # Concatena el audio con el silencio
    combined = sound + silence
    # Guarda el nuevo archivo
    combined.export(audio_path, format='wav')


def voz_sintetica_english(texto, pausa="true"):
    if pausa == "true":
        texto = add_comma_after_punctuation(texto)
    # Preparamos los datos de entrada para el modelo
    sample = TTSHubInterface.get_model_input(task, texto)

    # Movemos los datos al dispositivo GPU
    sample = move_to_device(sample, 'cuda:0')

    # Realizamos la predicción
    wav, rate = TTSHubInterface.get_prediction(task, modelT2S, generator, sample)


        # Convertimos el tensor wav a un buffer de audio en memoria y luego a un archivo temporal
    temp_wav_path = f"temp_synth_audio_{int(time.time() * 1000)}.wav"
    with io.BytesIO() as audio_buffer:
        sf.write(audio_buffer, wav.cpu().numpy(), rate, format='WAV')
        audio_buffer.seek(0)  # Regresamos al inicio del buffer para leerlo
        # Guardar en un archivo temporal
        with open(temp_wav_path, 'wb') as f:
            f.write(audio_buffer.read())

  

    # if len(texto) <= 20:
    #     # Añade silencio al final del archivo de audio
    #     add_silence_to_audio(temp_wav_path, 1000)  # Añade 1 segundo de silencio para que no de problemas en audios cortos

    if len(texto) <= 30:
        print("Añadiendo 700 milisegundos de silencio")
        add_silence_to_audio(temp_wav_path, 700)

    elif len(texto) <= 44:
        print("Añadiendo 0.5 segundos de silencio")
        add_silence_to_audio(temp_wav_path, 500)

    # Añadir el audio al archivo de conversación en segundo plano
    add_audio_to_conversation_async(temp_wav_path, convert_to_mp3=True)  # Asegúrate de implementar la conversión dentro de esta función si es necesario


    # Convertir el buffer a base64 para retornar
    with open(temp_wav_path, 'rb') as f:
        audio_base64 = base64.b64encode(f.read()).decode('utf-8')


    return audio_base64


def print_routes(app):
    print("Endpoints disponibles:")
    for rule in app.url_map.iter_rules():
        methods = ','.join(sorted(rule.methods))
        print(f"{rule.endpoint}: {rule.rule} [{methods}]")


if __name__ == '__main__':
    print_routes(app)
    app.run(host='0.0.0.0', port=5500, threaded=True)
    # app.run(host='0.0.0.0', port=5500, threaded=True, ssl_context=('cert.pem', 'key.pem'))




idioma ingles
Endpoints disponibles:
static: /static/<path:filename> [GET,HEAD,OPTIONS]
alive: /alive [GET,HEAD,OPTIONS]
print_strings: /inicio [OPTIONS,POST]
get_translations: /get-translations-file [GET,HEAD,OPTIONS]
save_translations: /save-translations-file [OPTIONS,POST]
all_conversation: /all_conversation [GET,HEAD,OPTIONS]
transcribe_audio: /transcribe [OPTIONS,POST]
only_transcribe_audio: /only_transcribe [OPTIONS,POST]
get_next_part: /get_next_part [GET,HEAD,OPTIONS]
process_text: /texto [OPTIONS,POST]
generate_audio: /audio [OPTIONS,POST]
primer_audio: /primer_audio [GET,HEAD,OPTIONS]
 * Serving Flask app '__main__'
 * Debug mode: off


2024-05-10 15:13:54 | INFO | werkzeug | WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5500
 * Running on http://172.25.241.247:5500
2024-05-10 15:13:54 | INFO | werkzeug | Press CTRL+C to quit


2024-05-10 15:14:00 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 15:14:00] "OPTIONS /audio HTTP/1.1" 200 -
2024-05-10 15:14:01 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 15:14:01] "POST /audio HTTP/1.1" 200 -


Añadiendo 0.5 segundos de silencio


2024-05-10 15:14:05 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 15:14:05] "GET /alive HTTP/1.1" 200 -
2024-05-10 15:14:05 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 15:14:05] "OPTIONS /inicio HTTP/1.1" 200 -


...

2024-05-10 15:25:15 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 15:25:15] "POST /transcribe HTTP/1.1" 200 -


Empezamos a generar ponemos el TOP a -1 para USER:128694!!:  Good morning, my name is Javier.
generando=True; Generando respuesta para USER:128694:  Good morning, my name is Javier.
assistant:Longitud: 635 Colchon: 18432
Nice to meet you, Javier! How are you today?esperando primer audio
esperando primer audio
esperando primer audio


2024-05-10 15:25:16 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 15:25:16] "GET /primer_audio?userID=128694 HTTP/1.1" 200 -


esperando primer audio
RECUPERANDO PRIMER AUDIO!!!!!!!!: ...


2024-05-10 15:25:16 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 15:25:16] "GET /get_next_part?index=0&userID=128694 HTTP/1.1" 200 -
2024-05-10 15:25:17 | INFO | werkzeug | 172.25.240.1 - - [10/May/2024 15:25:17] "GET /get_next_part?index=1&userID=128694 HTTP/1.1" 200 -


cliente rápido

In [25]:
%%writefile JuegosRapidos.html


<head>
    <meta charset="UTF-8">
    <title>Conversational Games</title>
  <style>


    body {
        font-family: Arial, sans-serif; /* Mejora la tipografía general */
    }


    #system_prompt {
        height: 150px;
        display: none;
    }


    #inicioForm {
        max-width: 1000px; /* Limita el ancho del formulario */
        margin: 20px auto; /* Centra el formulario */
        padding: 20px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Añade un sombreado ligero */
    }

    #inicioForm div {
        margin-bottom: 15px; /* Añade más espacio entre los campos */
    }

    #inicioForm label {
        font-weight: bold; /* Hace que las etiquetas sean más notables */
        display: block; /* Asegura que la etiqueta esté encima del input */
        margin-bottom: 5px; /* Espacio entre la etiqueta y el campo */
    }

    #inicioForm select, #inicioForm textarea, #inicioForm button {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
    }

    #inicioForm select {
        cursor: pointer; /* Indica que es un elemento interactivo */
        font-size: 66px; /* Aumenta el tamaño del texto */
        text-align: right; /* Alinea el texto a la derecha */
    }

    #inicioForm textarea {
        resize: vertical; /* Permite al usuario ajustar la altura verticalmente */
        display: none;
    }

    #inicioForm button {
        background-color: #007bff; /* Color de fondo */
        color: white; /* Color del texto */
        border: none; /* Elimina el borde */
        padding: 10px 15px; /* Añade relleno */
        font-size: 58px; /* Aumenta el tamaño del texto */
        cursor: pointer; /* Indica que es un elemento interactivo */
        border-radius: 5px; /* Bordes redondeados */
    }

    #inicioForm button:hover {
        background-color: #0056b3; /* Oscurece el botón al pasar el mouse */
    }

#audioPlayerContainer {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 50px; /* Espacio antes del botón de grabación */
    margin-top: 30px;
}



#recordButton {
    background-color: #f44336; /* Color rojo para la grabación */
    color: white;
    border: none;
    padding: 10px 0;
    font-size: 58px;
    border-radius: 5px;
    cursor: pointer;
}

#recordButton:hover {
    background-color: #d32f2f; /* Oscurece el botón al pasar el mouse */
}

/* Estilos para el área de texto y botón de envío */
/* Contenedor del área de texto y el botón */
div#textButtonContainer {
    display: flex; /* Establece el contenedor para usar flexbox */
    justify-content: space-between; /* Espacia los elementos uniformemente */
    align-items: center; /* Alinea los elementos verticalmente en el centro */
}

/* Área de texto */
#textInput {
    flex-grow: 1; /* Permite que el área de texto crezca para ocupar el espacio disponible */
    margin-right: 10px; /* Añade un margen a la derecha para separarlo del botón */
    border: 1px solid #ccc; /* Establece un borde sutil */
    border-radius: 5px; /* Bordes redondeados */
    padding: 8px; /* Añade padding interno */
    font-size: xx-large; /* Aumenta el tamaño del texto */
}

/* Botón */
#sendTextButton {
    padding: 30px 15px; /* Ajusta el padding para dimensionar el botón */
    background-color: #4CAF50; /* Color de fondo */
    color: white; /* Color del texto */
    border: none; /* Elimina el borde */
    border-radius: 5px; /* Bordes redondeados */
    cursor: pointer; /* Cambia el cursor a mano al pasar sobre el botón */
    font-size: xx-large; /* Aumenta el tamaño del texto */
}

#sendTextButton:hover {
    background-color: #388E3C; /* Oscurece el botón al pasar el mouse */
}


#responseText {
    height: 250px;
    margin-top: 20px;
    border: 1px solid #ddd;
    padding: 10px;
    overflow-y: auto; /* Asegura el desplazamiento vertical */
    background-color: #f9f9f9; /* Fondo claro para resaltar el área */
    font-size: xxx-large;
}



</style>
</head>

<script>

window.intervalId = window.intervalId || null; // Asegura que intervalId sea global y única

var URL = 'https://javiergimenez.es/api'; // Inicializa la variable URL


function updateURL() {
    URL = document.getElementById('url').value; // Actualiza la variable URL con el valor del campo de entrada
}

function startInterval() {
    if (window.intervalId === null) {
        window.intervalId = setInterval(() => {
            fetch(URL + '/alive')
                .then(response => response.json())
                .then(data => console.log('Alive:', data))
                .catch(error => console.error('Error fetching alive status:', error));
        }, 30000);
        console.log('Intervalo iniciado.');
    } else {
        console.log('Ya existe un intervalo en ejecución.');
    }
}

function stopInterval() {
    if (window.intervalId !== null) {
        clearInterval(window.intervalId);
        console.log('Intervalo detenido.');
        window.intervalId = null;
    } else {
        console.log('No hay un intervalo para detener.');
    }
}


// startInterval()

// document.getElementById('btnAccesoMic').addEventListener('click', async () => {
//     try {
//         const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
//         // Procesa el stream aquí
//         console.log('Acceso al micrófono concedido');
//     } catch (error) {
//         console.error('Acceso al micrófono denegado:', error);
//     }
// });

document.addEventListener('DOMContentLoaded', function() {
    window.scrollTo(0, 0); // Asegura que la página comience en la parte superior
    document.getElementById('ejercicios').focus(); // Luego establece el foco en el selector
});



// Objeto para mapear los ejercicios a sus strings correspondientes
const ejerciciosStrings = {
    "guessing_game1": {
        systemPrompt: `This is a conversational game in which you have to think in this famous character: #personaje, and the user have to guess this character. You should aswer the user questions about the character. Be concise but give some clues. NEVER say the name of the character until the end. If the user guesses the character then you will say: "Congratulations, you got it right, the character was #personaje". If user give up you will say: "what a shame!, the character was #personaje
Game success example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he #personaje?
assistant: Congratulations, you were right, it was #personaje.
Game give up example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he Benito Perez?
assistant: No, it has nothing to do with.
user: I give up. Who is it?
assistant: what a shame!, the character was #personaje
`,
        saludo: "I'm thinking of a famous fictional character, guess which one it is."
    },
    "guessing_game2": {
        systemPrompt: "You are #personaje, the fictional character. You have to take on the personality of that character and engage in conversations about your events and experiences.",
        saludo: "I'm #personaje, the fictional character. Ask me anything you want to know about me."
    },
    "guessing_game3": {
        systemPrompt: "This is a conversational game in which you have to guess a famous character. You should make questions to the user in order to guess the character that the user have choosen.",
        saludo: "Do you want to play? I will guess your choosen character by asking about it."
    },
    "yes_no_game": {
        systemPrompt: `IMPORTANT: You only can answer "yes" or "no" (nothing more!).
This is a conversational game between you and a the user. The game consists that only at the beggininig you tell the user only a piece of the context and the user having to guess the "key point" of the context from "yes or no questions". The User could ask anything about the story (context) to guess the "key point" of the context but Assistant could only answer "Yes" or "not". If the user asks a question that does not lend itself or cannot be answered with a "yes" or "no" such as "What is the man's name?" then Assistant will respond: "Only "yes or no" questions. When the user guesses the key point of the story you will say: "Congratulations, you have guessed the key to the story.
Game context: A man named Edgar is the lighthouse keeper of Águilas for 30 years. He always turns on the lighthouse at dusk and shortly after sleeps in a small room next to its large lamp. On Edgar's birthday, at dusk after lighting the lighthouse, he decided to go to dinner with an old friend to celebrate his birthday. During dinner he drank more than necessary and they both got drunk. Afther the dinner Edgar accompanied his friend to her house and then went to his house, the lighthouse, where he sleeps every night. Upon entering the lighthouse and going up to his room, due to his drunkenness and the fact that he was very sleepy, he decided to turn off the light (which was actually the light from the main lamp of the lighthouse) to sleep off the drunkenness and did it without realizing or knowing it was dangerous. During the early hours of the morning, a cruise ship full of passengers crashed into the cliff that the lighthouse protected because, when it was turned off, neither the lookout, nor the captain, nor the rest of the crew nor the passengers could see that they were heading against the cliff. An hour later Edgar wakes up, it hasn't dawned yet but you can hear sirens and a lot of noise from the rescuers who are trying to rescue the shipwrecked. Edgar turns on the lamp to illuminate the scene where hundreds of dead shipwrecked people continually crash against the cliff due to the waves. Faced with this heartbreaking reality and his feeling of guilt, Edgar decides to commit suicide by jumping from the top of the lighthouse. The key point of the story that the user must find out is: "Edgar commits suicide because he was the LIGHTHOUSE keeper." or similar but always emphasizing that he was the lighthouse keeper.
IMPORTANT: You only can answer "yes" or "no" or "Only yes or no questions"
Examples of correct answers:
user: What is Edgar's job?
Assistant: Only yes or no questions.
user: Is Edgar a man?
Assistant: yes.`,
        saludo: `This is I can show about the hidden story: Edgar was dazed and comes to his room, turns off the light and lies down on his bed. He wakes up a few hours later, turns on the light, looks out the window and is so horrified that he ends up jumping out of the window and committing suicide.
Guess what happened.
IMPORTANT: From now on I can only answer you "yes" or "no" and nothing more.`
    },
    "English_teacher": {
        systemPrompt: `You are Sofie, an english teacher 29 years old. You will have simple dialogues with the student in your charge. you will only have concise conversations with short sentences so that the student is encouraged to converse. You will always speak in English, never in Spanish. If your student has grammatical errors, you will let them know.`,
        saludo: "Good Morning. What is your name?"
    }
    // Añade más ejercicios según sea necesario
};
</script>


<form id="inicioForm">
    <div>
       <!-- <label for="system_prompt">System Prompt:</label><br>-->
        <textarea id="system_prompt" name="system_prompt" rows="10" cols="100"></textarea>
    </div>
    <div>
        <!--<label for="saludo">Saludo:</label><br>-->
        <textarea id="saludo" name="saludo" rows="4" cols="100"></textarea>
    </div>
    <div>
        <!--<label for="ejercicios">Juegos:</label><br>-->
        <select id="ejercicios" name="ejercicios">
            <option value="">Selecciona un Juego  ▼</option>
            <option value="English_teacher">Sofie, English teacher .</option>
            <option value="guessing_game1">You Guess fictional character</option>
            <option value="guessing_game2">You speak with fictional character</option>
            <option value="guessing_game3">Chatbot guess your fictional character</option>
            <!-- <option value="yes_no_game">yes no game</option> -->
        </select>
    </div>
<!--    <div>
        <label for="url">URL:</label>
        <input type="text" id="url" name="url" value="https://javiergimenez.es/api" onchange="updateURL()">
    </div>-->
    <button type="submit">EMPEZAR!</button>

</form>


<div style="margin-bottom: 20px; display: flex; align-items: center;">
    <button id="downloadButton" style="background-color: #4CAF50; /* Color de fondo */
                                       color: white; /* Color del texto */
                                       padding: 15px 32px; /* Padding alrededor del texto */
                                       text-align: center; /* Alinea el texto al centro */
                                       text-decoration: none; /* Elimina la decoración del texto */
                                       display: inline-block; /* Hace que el botón sea un bloque en línea */
                                       font-size: 26px; /* Tamaño del texto */
                                       margin: 4px 2px; /* Margen alrededor del botón */
                                       cursor: pointer; /* Cambia el cursor a un puntero */
                                       border: none; /* Elimina el borde */
                                       border-radius: 8px; /* Redondea las esquinas del botón */
    ">Descargar Conversación</button>
    <div id="audioPlayerAllContainer"></div>
</div>


<script>
    document.getElementById('downloadButton').addEventListener('click', function() {
        obtenerYReproducirAll(); // Llama a la función en vez de redirigir
    });
</script>


<script>
historico = ""
const userID = Math.floor(Math.random() * (999999));
console.log("userID:" + userID);


document.getElementById('ejercicios').addEventListener('change', function() {
    var selectedKey = this.value; // La clave seleccionada del objeto
    if (selectedKey) {
        // Actualiza los textareas con los valores correspondientes
        document.getElementById('system_prompt').value = ejerciciosStrings[selectedKey].systemPrompt;
        document.getElementById('saludo').value = ejerciciosStrings[selectedKey].saludo;
    }
});


document.getElementById('inicioForm').addEventListener('submit', function(e) {
    // Prevenir el comportamiento predeterminado del formulario
    e.preventDefault();
    fetch(URL + '/alive')
      .then(response => response.json())
      .then(data => console.log('Alive:', data))
      .catch(error => console.error('Error fetching alive status:', error));
    // startInterval()
    // Obtener los valores de los campos del formulario
    const system_prompt = document.getElementById('system_prompt').value;
    const saludo = document.getElementById('saludo').value;

    document.getElementById('responseText').innerText = ""
    //obtenerYReproducirAudio(saludo)
    //updateResponseText(saludo + "\n")

    // Crear el cuerpo de la solicitud
    const data = { system_prompt, saludo };

    // Realizar la llamada al servicio Flask
    fetch(URL + '/inicio', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json',
        },
        body: JSON.stringify(data),
    })
    .then(response => response.json())
    .then(data => {
        let saludo = data.message;
        historico = data.historico
        obtenerYReproducirSaludo(saludo)
        updateResponseText("************ GREET: ************\n" +saludo + "\n")

    })
    .catch((error) => {
        console.error('Error:', error);
        alert('VUELVE A DAR AL PLAY!');
    });
    alert('ESPERE UNOS SEGUNDOS HASTA QUE EMPIECE LA CONVERSACIÓN');
});
</script>

<!-- <button id="btnAccesoMic">Permitir acceso al micrófono</button> -->


<div id="audioPlayerContainer"></div>
<button id="recordButton" style="width: 100%; height: 100px;">Pulsa para grabar/detener</button>


<!-- Estilos para textarea y botón de envío -->
<div id="textButtonContainer" style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4"></textarea>
    <button id="sendTextButton">Enviar Texto</button>
</div>



<div id="responseText" style="height: 750px; margin-top: 20px; border: 1px solid #ddd; padding: 10px; overflow-y:auto;"></div>
<script>
let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        console.log('mediaRecorder Detenido!!');
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};


async function enviarAudioAlServidor(blob) {
        let formData = new FormData();
        formData.append('file', blob, 'grabacion.ogg');
        formData.append('historico', JSON.stringify(historico)); // Se asegura de enviar como cadena JSON
        formData.append('userID', userID);

        try {
            const response = await fetch(URL + '/transcribe', {
                method: 'POST',
                body: formData, // Solo se envía formData
            });
            const data = await response.json();
            //return data; // Devuelve los datos procesados
       
            updateResponseText("\n************** ME: **************\n" + data.entrada + "\n********** RESPONSE: **********\n");

            // Reinicia el buffer de audio de manera más eficiente
            audioBuffer = Array(currentAudioIndex).fill({});
            currentAudioIndex = 0;

            historico += data.prompt
            await obtenerPartes();
            } catch (error) {
                console.error('Error al enviar el audio:', error);
                alert("VUELVE A DAR AL PLAY! (después puedes continuar la conversación)");
            }
}

async function obtenerPartes(indice = 0) {
    try {
        if (indice == 0) {
            console.log("recuperando primer audio!!");
            const response_audio = await fetch(`${URL}/primer_audio?userID=${userID}`);
            const data = await response_audio.json();
            if (data.audio_base64) {
                console.log('Enviando audio al buffer de reproducción, tamaño:', data.audio_base64.length, 'índice:', indice);
                addAudioClipToBuffer(data.audio_base64, indice);
            }
            const response = await fetch(`${URL}/get_next_part?index=${indice}&userID=${userID}`);
            const partData = await response.json();
            if (partData.output !== "") {
                let trozo = partData.output;
                updateResponseText(trozo);
                historico += trozo
                await obtenerPartes(indice + 1); // Llamada recursiva con el índice incrementado          
            }
             else{
                historico += "<|eot_id|>"
            }           
        }
        else{
            const response = await fetch(`${URL}/get_next_part?index=${indice}&userID=${userID}`);
            const partData = await response.json();
            if (partData.output !== "") {
                let trozo = partData.output;
                updateResponseText(trozo);
                historico += trozo

                await obtenerYReproducirAudio(trozo, indice);
                await obtenerPartes(indice + 1); // Llamada recursiva con el índice incrementado
            }
            else{
                historico += "<|eot_id|>"
            }
        }
    } catch (error) {
        console.error('Error al obtener la siguiente parte:', error);
    }
}

// Asumiendo que obtenerYReproducirAudio ya fue optimizado como se mostró anteriormente




// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTextoAlServidor(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};



let audioBuffer = [];
// inicializa audioBuffer con 100 elementos vacíos
for (let i = 0; i < 100; i++) {
    audioBuffer.push({});
}
let milisegundosInicio = Date.now();

let currentAudioIndex = 0; // Para controlar el orden de reproducción


async function enviarTextoAlServidor(texto) {
    try {
        const response = await fetch(URL + '/texto', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({ texto: texto, historico: historico, userID: userID })
        });
        const data = await response.json();
        // updateResponseText("\nyo: " + data.entrada + "\n\n" + "\n***********************************\n" + "respuesta: ");
        updateResponseText("\n************** ME: **************\n" + data.entrada + "\n********** RESPONSE: **********\n");

        historico += data.prompt

        // Reinicia el buffer de audio de manera más eficiente
        audioBuffer = Array(currentAudioIndex).fill({});
        currentAudioIndex = 0;

        await obtenerPartes();
    } catch (error) {
        console.error('Error al enviar el texto:', error);
        alert("VUELVE A DAR AL PLAY! (después puedes continuar la conversación)")
    }
}


async function obtenerYReproducirAudio(texto, index) {
    try {
        const response = await fetch(URL + '/audio', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({ texto: texto, pausa: 'true'})
        });
        const data = await response.json();
        if (data.audio_base64) {
            console.log('Enviando audio al buffer de reproducción, tamaño:', data.audio_base64.length, 'índice:', index);
            addAudioClipToBuffer(data.audio_base64, index);
        }
    } catch (error) {
        console.error('Error al obtener el audio:', error);
    }
}



function updateResponseText(text) {
    // document.getElementById('responseText').innerText += text;
    textarea = document.getElementById('responseText')
    textarea.innerText += text;
    textarea.scrollTop = textarea.scrollHeight;
}



function obtenerYReproducirAll() {
    fetch(URL + '/all_conversation', {
        method: 'GET',
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            createAudioAllPlayer(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}


function createAudioAllPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerAllContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

function obtenerYReproducirSaludo(texto) {
    //console.log('Obteniendo audio para:', texto, 'microsegundos:', Date.now() - milisegundosInicio);
    fetch(URL + '/audio', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto, pausa: 'true'})
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
         //   console.log('Obteniendo audio, tamaño:', data.audio_base64.length, 'SALUDO', 'microsegundos:', Date.now()-milisegundosInicio);
            playAudio(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}





function addAudioClipToBuffer(base64Audio, index) {
    if (index == 0) {
        console.log('Iniciando buffer de reproducción, tamaño:', base64Audio.length, 'índice:', index);
    }
    audioBuffer[index] = { audio: base64Audio};
    let audioPlayer = document.getElementById('audioPlayerContainer').querySelector('audio');
    //console.log('Despues de meter en Buffer, VIENDO si ejecuto play con indice:', index, 'pausa:',audioPlayer.paused, 'currentAudioIndex', currentAudioIndex,'microsegundos:', Date.now()-milisegundosInicio);
    if (audioPlayer.paused && currentAudioIndex == index) {
        playNextAudioClip();
    }
}

//function esObjetoVacio(obj) {
//  return Object.keys(obj).length === 0;
//}
function esObjetoVacio(obj) {
  try {
    // Intenta obtener las claves del objeto y verifica si su longitud es 0
    return Object.keys(obj).length === 0;
  } catch (error) {
    // Si ocurre un error (por ejemplo, si obj no es un objeto), devuelve true
    //console.error("Se produjo un error porque en esa posición del buffer está vacía: ", error);
    return true;
  }
}



function playNextAudioClip() {
    //console.log('Entrando en playNextAudioClip para ver si reproducimos audio con currentAudioIndex:', currentAudioIndex, 'microsegundos:', Date.now()-milisegundosInicio, 'esObjetoVacio:', esObjetoVacio(audioBuffer[currentAudioIndex]));
    if (!esObjetoVacio(audioBuffer[currentAudioIndex])) {
       // console.log('Iniciando variables para Reproducion audio concurrentAudioIndex:', currentAudioIndex, 'tamaño:', audioBuffer[currentAudioIndex].audio.length, 'microsegundos:', Date.now()-milisegundosInicio);
        const audioData = audioBuffer[currentAudioIndex];
        audioBuffer[currentAudioIndex] = {}; // Limpia el elemento actual
        currentAudioIndex++;

        let audioContainer = document.getElementById('audioPlayerContainer');
        audioContainer.innerHTML = ''; // Limpia el contenedor

        let audioSrc = `data:audio/wav;base64,${audioData.audio}`;
        let audioPlayer = document.createElement('audio');
        audioPlayer.src = audioSrc;
        audioPlayer.controls = true;
        audioPlayer.autoplay = true;

        audioContainer.appendChild(audioPlayer);

        audioPlayer.onended = playNextAudioClip;
    }
}

function playAudio(audioBase64) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    audioContainer.innerHTML = ''; // Limpia el contenedor

    let audioSrc = `data:audio/wav;base64,${audioBase64}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;

    //console.log('Reproduciendo saludo! Justo antes de APPENDCHILD', 'microsegundos:', Date.now()-milisegundosInicio);
    audioContainer.appendChild(audioPlayer);
}

function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

obtenerYReproducirSaludo("Wellcom to Conversational Games!")



</script>


Overwriting JuegosRapidos.html
